In [154]:
library(mgcv)
library(dplyr)
library(gdata)
library(epitools)

In [194]:
df<- read.csv("csv_folder/eicu_clean_19Aug.csv", stringsAsFactors = TRUE)
df$age<- as.integer(df$age)
df$gender<- factor(df$gender)
df$ethnicity<- factor(df$ethnicity)
cols <- c(18:29)
df[cols] <- lapply(df[cols], factor)  
df_eicu<- df[,-c(1,7,8,10,12,39,41)]
df_eicu$hosp_mort <-ifelse(df_eicu$hosp_mort=='ALIVE',0,1)
df_eicu$icu_mort<-ifelse(df_eicu$icu_mort=='ALIVE',0,1)
df_eicu$gender <-ifelse(df_eicu$gender=='Female','F','M')
#df_eicu$gender<-df_eicu$Gender
df_eicu <- df_eicu %>% rename(vasso_duration_24h = X24hrs_hr, 
                              vasso_duration_48h = X48hrs_hr,
                              vasso_duration_72h = X72hrs_hr,
                              mechanical_ventilation = ventilation_duration_days,
                              sepsis = sepsis_binary,
                              hospital_expire_flag = hosp_mort,
                              icu_mortality = icu_mort,
                              stroke = cebrovascular_accident,
                              afib = atrial_fibrillation
                                )
df_eicu$use_vasopressor<- ifelse(df_eicu$vasso_duration_24h>0,1,0)


df_eicu_aki <-df_eicu[df_eicu$mortality_7d == 'ALIVE',]
eicu_prop <- read.csv("csv_folder/eicu_prop_24h.csv",stringsAsFactors = TRUE)
df_eicu_prop <- merge(x = df_eicu, y = eicu_prop[ ,  -1], by = "patientunitstayid", all.x=TRUE)
df_eicu_prop_AKI <-df_eicu_prop
df_eicu_prop_AKI$prop_map_24_AKI<- 1- df_eicu_prop_AKI$propBelow_map_24
df_eicu_prop_AKI$prop_asp_24_AKI<- 1- df_eicu_prop_AKI$propBelow_asp_24
df_eicu_prop_AKI$prop_adp_24_AKI<- 1- df_eicu_prop_AKI$propBelow_adp_24
df_eicu_prop_AKI<-df_eicu_prop_AKI[df_eicu_prop_AKI$mortality_7d == 'ALIVE',]

eicu_prop_48 <- read.csv("csv_folder/eicu_prop_48h.csv",stringsAsFactors = TRUE)
df_eicu_prop_48 <- merge(x = df_eicu, y = eicu_prop_48[ ,  -1], by = "patientunitstayid", all.x=TRUE)
df_eicu_prop_AKI_48 <-df_eicu_prop_48
df_eicu_prop_AKI_48$prop_map_48_AKI<- 1- df_eicu_prop_AKI_48$propBelow_map_48
df_eicu_prop_AKI_48$prop_asp_48_AKI<- 1- df_eicu_prop_AKI_48$propBelow_asp_48
df_eicu_prop_AKI_48$prop_adp_48_AKI<- 1- df_eicu_prop_AKI_48$propBelow_adp_48
df_eicu_prop_AKI_48<-df_eicu_prop_AKI_48[df_eicu_prop_AKI_48$mortality_7d == 'ALIVE',]

eicu_prop_72 <- read.csv("csv_folder/eicu_prop_72h.csv",stringsAsFactors = TRUE)
df_eicu_prop_72 <- merge(x = df_eicu, y = eicu_prop_72[ ,  -1], by = "patientunitstayid", all.x=TRUE)
df_eicu_prop_AKI_72 <-df_eicu_prop_72
df_eicu_prop_AKI_72$prop_map_72_AKI<- 1- df_eicu_prop_AKI_72$propBelow_map_72
df_eicu_prop_AKI_72$prop_asp_72_AKI<- 1- df_eicu_prop_AKI_72$propBelow_asp_72
df_eicu_prop_AKI_72$prop_adp_72_AKI<- 1- df_eicu_prop_AKI_72$propBelow_adp_72
df_eicu_prop_AKI_72<-df_eicu_prop_AKI_72[df_eicu_prop_AKI_72$mortality_7d == 'ALIVE',]

head(df_eicu_prop_AKI, n = 2)

patientunitstayid,age,gender,ethnicity,unittype,icu_mortality,hospital_expire_flag,apache_iv,sofatotal,vasso_duration_24h,⋯,prop_map_24,propBelow_asp_24,propAbove_asp_24,prop_asp_24,propBelow_adp_24,propAbove_adp_24,prop_adp_24,prop_map_24_AKI,prop_asp_24_AKI,prop_adp_24_AKI
141194,68,M,Caucasian,CTICU,0,0,70,3,0.00,⋯,0.04166667,0.7500000,0.04166667,0.2083333,1.0000000,0.000000000,0.00000000,0.04166667,0.2500000,0.00000000
141233,81,F,Caucasian,CTICU,0,0,66,12,23.97,⋯,0.50467290,0.3271028,0.45794393,0.2149533,0.9065421,0.009345794,0.08411215,0.65420561,0.6728972,0.09345794


In [195]:
df1 <- read.csv("csv_folder/mimic_clean_19Aug.csv", 
               stringsAsFactors = TRUE)
df1$age<- as.integer(df1$age)
df1$icu_mortality<- factor(df1$icu_mortality)

cols <- c(17:28,39,40 )
df1[cols] <- lapply(df1[cols], factor)  
df_mimic<- df1[,-c(1,7,8,10,12)]
df_mimic <- df_mimic %>% rename(vasso_duration_24h = first_24_vasso_hours, 
                                vasso_duration_48h = first_48_vasso_hours,
                                vasso_duration_72h = first_72_vasso_hours,
                                unittype = first_careunit,
                                ethnicity = ethnicity_new, 
                                AKI_7d =aki_7day,
                                sofatotal = sofa,
                                mechanical_ventilation = vent_duration_days,
                                chf = congestive_heart_failure,
                                stroke = cereb)
df_mimic$use_vasopressor <- ifelse(df_mimic$vasso_duration_24h>0,1,0)

df_mimic_aki <-df_mimic[df_mimic$mortality_7d == 0,]
mimic_prop <- read.csv("csv_folder/mimic_prop_24h.csv",stringsAsFactors = TRUE)
# mimic_prop_AKI <- read.csv("csv_folder/mimic_prop_map_48h_72h.csv",stringsAsFactors = TRUE)

df_mimic_prop <- merge(x = df_mimic, y = mimic_prop[ ,  -1], by = "icustay_id", all.x=TRUE)
df_mimic_prop_AKI <- df_mimic_prop
df_mimic_prop_AKI$prop_map_24_AKI<-1-df_mimic_prop_AKI$propBelow_map_24
df_mimic_prop_AKI$prop_asp_24_AKI<-1-df_mimic_prop_AKI$propBelow_asp_24
df_mimic_prop_AKI$prop_adp_24_AKI<-1-df_mimic_prop_AKI$propBelow_adp_24
df_mimic_prop_AKI<-df_mimic_prop_AKI[df_mimic_prop_AKI$mortality_7d == 0,]

mimic_prop_48 <- read.csv("csv_folder/mimic_prop_48h.csv",stringsAsFactors = TRUE)
df_mimic_prop_48 <- merge(x = df_mimic, y = mimic_prop_48[ ,  -1], by = "icustay_id", all.x=TRUE)
df_mimic_prop_AKI_48 <-df_mimic_prop_48
df_mimic_prop_AKI_48$prop_map_48_AKI<- 1- df_mimic_prop_AKI_48$propBelow_map_48
df_mimic_prop_AKI_48$prop_asp_48_AKI<- 1- df_mimic_prop_AKI_48$propBelow_asp_48
df_mimic_prop_AKI_48$prop_adp_48_AKI<- 1- df_mimic_prop_AKI_48$propBelow_adp_48
df_mimic_prop_AKI_48<-df_mimic_prop_AKI_48[df_mimic_prop_AKI_48$mortality_7d == 0,]

mimic_prop_72 <- read.csv("csv_folder/mimic_prop_72h.csv",stringsAsFactors = TRUE)
df_mimic_prop_72 <- merge(x = df_mimic, y = mimic_prop_72[ ,  -1], by = "icustay_id", all.x=TRUE)
df_mimic_prop_AKI_72 <-df_mimic_prop_72
df_mimic_prop_AKI_72$prop_map_72_AKI<- 1- df_mimic_prop_AKI_72$propBelow_map_72
df_mimic_prop_AKI_72$prop_asp_72_AKI<- 1- df_mimic_prop_AKI_72$propBelow_asp_72
df_mimic_prop_AKI_72$prop_adp_72_AKI<- 1- df_mimic_prop_AKI_72$propBelow_adp_72
df_mimic_prop_AKI_72<-df_mimic_prop_AKI_72[df_mimic_prop_AKI_72$mortality_7d == 0,]

head(df_mimic_prop_AKI, n = 2)

icustay_id,age,gender,ethnicity,unittype,icu_mortality,hospital_expire_flag,sofatotal,vasso_duration_24h,vasso_duration_48h,⋯,prop_map_24,propBelow_asp_24,propAbove_asp_24,prop_asp_24,propBelow_adp_24,propAbove_adp_24,prop_adp_24,prop_map_24_AKI,prop_asp_24_AKI,prop_adp_24_AKI
200003,48,M,WHITE,SICU,0,0,6,1.084444,17.33444,⋯,0.4705882,0.4375000,0.1875000,0.3750000,0.06250000,0.7500000,0.1875000,0.9411765,0.5625000,0.9375000
200009,47,F,WHITE,CSRU,0,0,3,0.000000,0.00000,⋯,0.1935484,0.7096774,0.1612903,0.1290323,0.03225806,0.3548387,0.6129032,0.4516129,0.2903226,0.9677419


In [157]:
# Model fitting
plot_width <- 17.2 * 0.393701 # The textwidth of our Word manuscript in inches.
#plot_width <- 4.2126 # The Lancet asks for plots to have a width of 107mm.
font <- "Times" # Text in figures should be Times New Roman according to the Lancet.
point_size <- 10 # Per Lancet instructions
logistic <- function(x) 1/(1+exp(-x))



# Function that adds labels to subfigures, edited from:
# https://logfc.wordpress.com/2017/03/15/adding-figure-labels-a-b-c-in-the-top-left-corner-of-the-plotting-region/
fig_label <- function(label) {
    cex <- 2
    ds <- dev.size("in")
    # xy coordinates of device corners in user coordinates
    x <- grconvertX(c(0, ds[1]), from="in", to="user")
    y <- grconvertY(c(0, ds[2]), from="in", to="user")
    
    # fragment of the device we use to plot
      # account for the fragment of the device that 
      # the figure is using
      fig <- par("fig")
      dx <- (x[2] - x[1])
      dy <- (y[2] - y[1])
      x <- x[1] + dx * fig[1:2]
      y <- y[1] + dy * fig[3:4]
  
  
  sw <- strwidth(label, cex=cex) * 60/100
  sh <- strheight(label, cex=cex) * 60/100
  
  x1 <- x[1] + sw
  
  y1 <- y[2] - sh
  
  old.par <- par(xpd=NA)
  on.exit(par(old.par))
  
  text(x1, y1, label, cex=cex)
}

In [158]:
gam_plotMed <- function(
                    gamfitMed,
                    main = "Median of measurements",
                    xRange = c(55, 95),
                    yRange = c(0, .20),
                    label,
                    xlabel
) {
# print(paste("Number of cases:", summary(gamfitMed)$n))

xlab <- xlabel


if(colnames(gamfitMed$model)[1] == "hospital_expire_flag") {
    ylab <- "Probability of hospital mortality"
  }
    else if(colnames(gamfitMed$model)[1] == "AKI_7d") {
    ylab <- "Probability of acute kidney injury (AKI)"
  } 
    else {
    ylab <- "Probability of ICU mortality"
  }

xName <- colnames(gamfitMed$model)[grep("ibp", colnames(gamfitMed$model))]
# vasso_Name <- colnames(gamfitMed$model)[grep("vasso", colnames(gamfitMed$model))]

# Color for dotted line
colD <- "Black"
# colD <- "Blue"

    plot(1, type = 'n', xlim = xRange, ylim = yRange,
         ylab = ylab,
         xlab = xlab, main = main, yaxs = 'i', xaxs = 'i', yaxt = 'n', xaxt = 'n')
    
    att <- pretty(yRange)
if(!isTRUE(all.equal(att, round(att, digits = 2)))) {
  axis(2, at = att, lab = paste0(sprintf('%.1f', att*100), '%'), las = TRUE)
} else axis(2, at = att, lab = paste0(att*100, '%'), las = TRUE)
    
    att <- pretty(xRange)
    axis(1, at = att, lab = paste0(att), las = TRUE)
#     axis(1, at = att, lab = paste0(att, '%'), las = TRUE)

    
    eval(parse(text = paste(c('predictionsPlusCI <- predict(gamfitMed, newdata = data.frame(',
                              xName, ' = gamfitMed$model$', xName, ", gender = 'F', age = median(gamfitMed$model$age), 
                            bmi = median(gamfitMed$model$bmi),",
                      ifelse(
                        "high_vent_proportion" %in% colnames(gamfitMed$model),
                        "high_vent_proportion = median(gamfitMed$model$high_vent_proportion),",
                        ""
                      ),
                      ifelse(
                        "ethnicity" %in% colnames(gamfitMed$model),
                        "ethnicity = gamfitMed$model$ethnicity,",
                        ""
                      ),
                      ifelse(
                        "apsiii" %in% colnames(gamfitMed$model),
                        "apsiii = median(gamfitMed$model$apsiii),",
                        "sofatotal = median(gamfitMed$model$sofatotal),"
                        
                      ),
                      ifelse(
                        "vasso_duration_24h" %in% colnames(gamfitMed$model),
                        "vasso_duration_24h = median(gamfitMed$model$vasso_duration_24h),",
                        ""
                      ),
                      ifelse(
                        "vasso_duration_48h" %in% colnames(gamfitMed$model),
                        "vasso_duration_48h = median(gamfitMed$model$vasso_duration_48h),",
                        ""
                      ),
                      ifelse(
                        "vasso_duration_72h" %in% colnames(gamfitMed$model),
                        "vasso_duration_72h = median(gamfitMed$model$vasso_duration_72h),",
                        ""
                      ),
                      "hospital_id = 264), type = 'link', se.fit = T)"), collapse = "")))

  
  # We have to use the data on which GAM was fit for confidence region as the GAM does not provide standard errors for 'new' input
    eval(parse(text = paste0('xx <- gamfitMed$model$', xName)))
    ord.index <- order(xx)
    xx <- xx[ord.index]
    i<-2
    j<-1
    index <-1
    bp_val <-list()
    bp_grad<-list()
    current_list <-list()
    next_list<-list()
    while (i <=length(ord.index)){
        eval(parse(text = paste(c('x2 <- gamfitMed$model$', xName,"[ord.index[",i,"]]"), collapse = "")))
        eval(parse(text = paste(c('x1 <- gamfitMed$model$', xName,"[ord.index[",j,"]]"), collapse = "")))
#       x2 <- b$model$ibp_mean_24h[ord.index[i]]
#       x1 <-b$model$ibp_mean_24h[ord.index[j]]

        if((x2-x1)>=1.0){
            current_val <- logistic(predictionsPlusCI$fit[ord.index[j]])
            right_val <- logistic(predictionsPlusCI$fit[ord.index[i]])
            grad <-(right_val-current_val)*100/(x2-x1)
            bp_val[[index]] <- x1
            bp_grad[[index]]<-unlist(grad,use.names=FALSE)
            current_list[[index]]<-current_val
            next_list[[index]]<-right_val
            index <-index+1
            j <- i
            i <- i+1

        } else {
            i <-i+1
        }

        }
    df_gradient <- data.frame("bp_value" = unlist(bp_val, use.names=FALSE), 
                        "gradient" = unlist(bp_grad, use.names=FALSE)
                       , "pre_val_left" = unlist(current_list, use.names=FALSE)
                       , "pre_val_right" = unlist(next_list, use.names=FALSE))
    
    by_type <- colnames(gamfitMed$model)[grep("ibp", colnames(gamfitMed$model))]
    if(startsWith(xName, 'ibp_mean', ignore.case=TRUE)){
        bp_value_left <-60
        bp_value_right<-90
    } else if (startsWith(xName, 'ibp_systolic', ignore.case=TRUE)){
        bp_value_left <-100
        bp_value_right<-135
        
    } else {
        bp_value_left <-40
        bp_value_right<-70
    }
    
    
    grad_0.2_left<-min(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient< 0.2)
                                   & (df_gradient$bp_value< bp_value_right) & (df_gradient$bp_value> bp_value_left), "bp_value"])
    grad_0.2_right<-max(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient< 0.2)
                               & (df_gradient$bp_value< bp_value_right) & (df_gradient$bp_value> bp_value_left), "bp_value"])
    grad_0.1_left<-min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient< 0.1)
                                   & (df_gradient$bp_value< bp_value_right) & (df_gradient$bp_value> bp_value_left), "bp_value"])
    grad_0.1_right<-max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient< 0.1)
                               & (df_gradient$bp_value< bp_value_right) & (df_gradient$bp_value> bp_value_left), "bp_value"])
    grad_0.2_left <-ifelse(grad_0.2_left>=43, grad_0.2_left,40)
    grad_0.1_left <-ifelse(grad_0.1_left>=43, grad_0.1_left,41)
    grad_0.2_right<-ifelse(grad_0.2_right<=137, grad_0.2_right,139)
    grad_0.1_right<-ifelse(grad_0.1_right<=137, grad_0.1_right,138)

    if(gamfitMed$family$link == 'logit') {
        lcl <- logistic(predictionsPlusCI$fit[ord.index] - 1.96*predictionsPlusCI$se.fit[ord.index])
        ucl <- logistic(predictionsPlusCI$fit[ord.index] + 1.96*predictionsPlusCI$se.fit[ord.index])

        lines(x = xx, y = lcl, lty = 2, lwd = 2, col = colD)
        lines(x = xx, y = ucl, lty = 2, lwd = 2, col = colD)
        lines(xx, logistic(predictionsPlusCI$fit[ord.index]), lwd = 3)
        abline(v=grad_0.2_left, col = "Red")
        abline(v=grad_0.2_right, col = "Red")
        abline(v=grad_0.1_left, col = "Blue")
        abline(v=grad_0.1_right, col = "Blue")
    } else {
        lcl <- predictionsPlusCI$fit[ord.index] - 1.96*predictionsPlusCI$se.fit[ord.index]
        ucl <- predictionsPlusCI$fit[ord.index] + 1.96*predictionsPlusCI$se.fit[ord.index]

        lines(x = xx, y = lcl, lty = 2, lwd = 2)
        lines(x = xx, y = ucl, lty = 2, lwd = 2)
        lines(xx, predictionsPlusCI$fit[ord.index], lwd = 3)
        abline(v=grad_0.2_left, col = "Red")
        abline(v=grad_0.2_right, col = "Red")
        abline(v=grad_0.1_left, col = "Blue")
        abline(v=grad_0.1_right, col = "Blue")
    }

    if(!missing(label)) fig_label(label)
}


In [159]:
find_gradient <- function(b)
{
xName <- colnames(b$model)[grep("ibp", colnames(b$model))]
eval(parse(text = paste(c('newdata = data.frame(',
                          xName, ' = b$model$', xName, ", gender = 'F', age = median(b$model$age), bmi = median(b$model$bmi),",
                  ifelse(
                    "high_vent_proportion" %in% colnames(b$model),
                    "high_vent_proportion = median(b$model$high_vent_proportion),",
                    ""
                  ),
                  ifelse(
                    "apsiii" %in% colnames(b$model),
                    "apsiii = median(b$model$apsiii),",
                    "sofatotal = median(b$model$sofatotal),"
                  ),
                  ifelse(
                    "ethnicity" %in% colnames(b$model),
                    "ethnicity = b$model$ethnicity,",
                    ""
                  ),
                  ifelse(
                    "apsiii" %in% colnames(b$model),
                    "apsiii = median(b$model$apsiii),",
                    "sofatotal = median(b$model$sofatotal),"

                  ),
                  ifelse(
                    "vasso_duration_24h" %in% colnames(b$model),
                    "vasso_duration_24h = median(b$model$vasso_duration_24h),",
                    ""
                  ),
                  ifelse(
                    "vasso_duration_48h" %in% colnames(b$model),
                    "vasso_duration_48h = median(b$model$vasso_duration_48h),",
                    ""
                  ),
                  ifelse(
                    "vasso_duration_72h" %in% colnames(b$model),
                    "vasso_duration_72h = median(b$model$vasso_duration_72h),",
                    ""
                  ),
                  "hospital_id = 264)"), collapse = "")))
# newd <- newdata
pred<-predict(b,newdata,type="link",se.fit = T)
eval(parse(text = paste0('xx <- b$model$', xName)))
# xx <- b$model$ibp_mean_24h
ord.index <- order(xx)
i<-2
j<-1
index <-1
bp_val <-list()
bp_grad<-list()
current_list <-list()
next_list<-list()
while (i <=length(ord.index)){
    eval(parse(text = paste(c('x2 <- b$model$', xName,"[ord.index[",i,"]]"), collapse = "")))
    eval(parse(text = paste(c('x1 <- b$model$', xName,"[ord.index[",j,"]]"), collapse = "")))
#       x2 <- b$model$ibp_mean_24h[ord.index[i]]
#       x1 <-b$model$ibp_mean_24h[ord.index[j]]
    
    if((x2-x1)>=1.0){
        current_val <- logistic(pred$fit[ord.index[j]])
        right_val <- logistic(pred$fit[ord.index[i]])
        grad <-(right_val-current_val)*100/(x2-x1)
        bp_val[[index]] <- x1
        bp_grad[[index]]<-unlist(grad,use.names=FALSE)
        current_list[[index]]<-current_val
        next_list[[index]]<-right_val
        index <-index+1
        j <- i
        i <- i+1
        
      } else {
          i <-i+1
      }
      
    }
df_gradient <- data.frame("bp_value" = unlist(bp_val, use.names=FALSE), 
                          "gradient" = unlist(bp_grad, use.names=FALSE)
                         , "pre_val_left" = unlist(current_list, use.names=FALSE)
                         , "pre_val_right" = unlist(next_list, use.names=FALSE))
return (df_gradient)
}

In [160]:
gam_plotMed_noGradient <- function(
                    gamfitMed,
                    main = "Median of measurements",
                    xRange = c(55, 95),
                    yRange = c(0, .20),
                    label,
                    xlabel
) {
# print(paste("Number of cases:", summary(gamfitMed)$n))

xlab <- xlabel


if(colnames(gamfitMed$model)[1] == "hospital_expire_flag") {
    ylab <- "Probability of hospital mortality"
  }
    else if(colnames(gamfitMed$model)[1] == "AKI_7d") {
    ylab <- "Probability of acute kidney injury (AKI)"
  } 
    else {
    ylab <- "Probability of ICU mortality"
  }

xName <- colnames(gamfitMed$model)[grep("ibp", colnames(gamfitMed$model))]
vasso_Name <- colnames(gamfitMed$model)[grep("vasso", colnames(gamfitMed$model))]

# Color for dotted line
colD <- "Black"

    plot(1, type = 'n', xlim = xRange, ylim = yRange,
         ylab = ylab,
         xlab = xlab, main = main, yaxs = 'i', xaxs = 'i', yaxt = 'n', xaxt = 'n')
    
    att <- pretty(yRange)
if(!isTRUE(all.equal(att, round(att, digits = 2)))) {
  axis(2, at = att, lab = paste0(sprintf('%.1f', att*100), '%'), las = TRUE)
} else axis(2, at = att, lab = paste0(att*100, '%'), las = TRUE)
    
    att <- pretty(xRange)
    axis(1, at = att, lab = paste0(att), las = TRUE)
#     axis(1, at = att, lab = paste0(att, '%'), las = TRUE)

    
    eval(parse(text = paste(c('predictionsPlusCI <- predict(gamfitMed, newdata = data.frame(',
                              xName, ' = gamfitMed$model$', xName, ", gender = 'F', age = median(gamfitMed$model$age), 
                            bmi = median(gamfitMed$model$bmi),",
                      ifelse(
                        "high_vent_proportion" %in% colnames(gamfitMed$model),
                        "high_vent_proportion = median(gamfitMed$model$high_vent_proportion),",
                        ""
                      ),
                      ifelse(
                        "ethnicity" %in% colnames(gamfitMed$model),
                        "ethnicity = gamfitMed$model$ethnicity,",
                        ""
                      ),
                      ifelse(
                        "apsiii" %in% colnames(gamfitMed$model),
                        "apsiii = median(gamfitMed$model$apsiii),",
                        "sofatotal = median(gamfitMed$model$sofatotal),"
                        
                      ),
                      ifelse(
                        "vasso_duration_24h" %in% colnames(gamfitMed$model),
                        "vasso_duration_24h = median(gamfitMed$model$vasso_duration_24h),",
                        ""
                      ),
                      ifelse(
                        "vasso_duration_48h" %in% colnames(gamfitMed$model),
                        "vasso_duration_48h = median(gamfitMed$model$vasso_duration_48h),",
                        ""
                      ),
                      ifelse(
                        "vasso_duration_72h" %in% colnames(gamfitMed$model),
                        "vasso_duration_72h = median(gamfitMed$model$vasso_duration_72h),",
                        ""
                      ),
                      "hospital_id = 264), type = 'link', se.fit = T)"), collapse = "")))

  
  # We have to use the data on which GAM was fit for confidence region as the GAM does not provide standard errors for 'new' input
  eval(parse(text = paste0('xx <- gamfitMed$model$', xName)))
  ord.index <- order(xx)
  xx <- xx[ord.index]
  
  if(gamfitMed$family$link == 'logit') {
    lcl <- logistic(predictionsPlusCI$fit[ord.index] - 1.96*predictionsPlusCI$se.fit[ord.index])
    ucl <- logistic(predictionsPlusCI$fit[ord.index] + 1.96*predictionsPlusCI$se.fit[ord.index])
    
    lines(x = xx, y = lcl, lty = 2, lwd = 2, col = colD)
    lines(x = xx, y = ucl, lty = 2, lwd = 2, col = colD)
    lines(xx, logistic(predictionsPlusCI$fit[ord.index]), lwd = 3)
  } else {
    lcl <- predictionsPlusCI$fit[ord.index] - 1.96*predictionsPlusCI$se.fit[ord.index]
    ucl <- predictionsPlusCI$fit[ord.index] + 1.96*predictionsPlusCI$se.fit[ord.index]
    
    lines(x = xx, y = lcl, lty = 2, lwd = 2)
    lines(x = xx, y = ucl, lty = 2, lwd = 2)
    lines(xx, predictionsPlusCI$fit[ord.index], lwd = 3)
  }
  
  if(!missing(label)) fig_label(label)
}


## Vasopressor 24h, 48h, 72h: mean vs hospital mortality

### eICU

In [439]:
df_vasso1_24 <-df_eicu_prop[df_eicu_prop$vasso_duration_24h >0,]
df_vasso0_24 <-df_eicu_prop[df_eicu_prop$vasso_duration_24h ==0,]
df_vasso1_48 <-df_eicu_prop[df_eicu_prop$vasso_duration_48h >0,]
df_vasso0_48 <-df_eicu_prop[df_eicu_prop$vasso_duration_48h ==0,]
df_vasso1_72 <-df_eicu_prop[df_eicu_prop$vasso_duration_72h >0,]
df_vasso0_72 <-df_eicu_prop[df_eicu_prop$vasso_duration_72h ==0,]

### MAP

In [431]:
# hospital mortality
gamfitMed_eICU_vasso1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_24, family = binomial)
gamfitMed_eICU_vasso0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_24, family = binomial)

gamfitMed_eICU_vasso1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_48, family = binomial)
gamfitMed_eICU_vasso0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_48, family = binomial)

gamfitMed_eICU_vasso1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_72, family = binomial)
gamfitMed_eICU_vasso0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_72, family = binomial)

# icu mortality
gamfitMed_eICU_vasso1_24h_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_24, family = binomial)
gamfitMed_eICU_vasso0_24h_icu  <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_24, family = binomial)

gamfitMed_eICU_vasso1_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_48, family = binomial)
gamfitMed_eICU_vasso0_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_48, family = binomial)

gamfitMed_eICU_vasso1_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_72, family = binomial)
gamfitMed_eICU_vasso0_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_72, family = binomial)


# AKI
gamfitMed_eICU_vasso1_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_24, family = binomial)
gamfitMed_eICU_vasso0_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_24, family = binomial)

gamfitMed_eICU_vasso1_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_48, family = binomial)
gamfitMed_eICU_vasso0_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_48, family = binomial)

gamfitMed_eICU_vasso1_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_72, family = binomial)
gamfitMed_eICU_vasso0_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_72, family = binomial)


In [434]:
pdf(file = "figure/subgroup/Vasso_eICU_MAP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(55, 95)
# hospital mortality
gam_plotMed(gamfitMed_eICU_vasso1_24h_hosp, main = "Vasopressor", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_vasso1_48h_hosp, main = "Vasopressor", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_vasso1_72h_hosp, main = "Vasopressor", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_vasso0_48h_hosp, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_vasso0_24h_hosp, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_vasso0_72h_hosp, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# icu mortality
gam_plotMed(gamfitMed_eICU_vasso1_24h_icu, main = "Vasopressor", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_vasso1_48h_icu, main = "Vasopressor", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_vasso1_72h_icu, main = "Vasopressor", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_vasso0_48h_icu, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_vasso0_24h_icu, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_vasso0_72h_icu, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# AKI
x_range = c(55, 95)
y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_eICU_vasso1_24h_aki, main = "Vasopressor", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_vasso1_48h_aki, main = "Vasopressor", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_vasso1_72h_aki, main = "Vasopressor", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_vasso0_48h_aki, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_vasso0_24h_aki, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_vasso0_72h_aki, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

dev.off()

[1] "Number of cases: 6304"
[1] "Number of cases: 6731"
[1] "Number of cases: 6876"
[1] "Number of cases: 5560"
[1] "Number of cases: 5987"
[1] "Number of cases: 5415"
[1] "Number of cases: 6304"
[1] "Number of cases: 6731"
[1] "Number of cases: 6876"
[1] "Number of cases: 5560"
[1] "Number of cases: 5987"
[1] "Number of cases: 5415"
[1] "Number of cases: 6304"
[1] "Number of cases: 6731"
[1] "Number of cases: 6876"
[1] "Number of cases: 5560"
[1] "Number of cases: 5987"
[1] "Number of cases: 5415"


png 
  2

### SBP

In [440]:
# hospital mortality
gamfitMed_eICU_vasso1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_24, family = binomial)
gamfitMed_eICU_vasso0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_24, family = binomial)

gamfitMed_eICU_vasso1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_48, family = binomial)
gamfitMed_eICU_vasso0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_48, family = binomial)

gamfitMed_eICU_vasso1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_72, family = binomial)
gamfitMed_eICU_vasso0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_72, family = binomial)

# icu mortality
gamfitMed_eICU_vasso1_24h_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_24, family = binomial)
gamfitMed_eICU_vasso0_24h_icu  <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_24, family = binomial)

gamfitMed_eICU_vasso1_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_48, family = binomial)
gamfitMed_eICU_vasso0_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_48, family = binomial)

gamfitMed_eICU_vasso1_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_72, family = binomial)
gamfitMed_eICU_vasso0_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_72, family = binomial)


# AKI
gamfitMed_eICU_vasso1_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_24, family = binomial)
gamfitMed_eICU_vasso0_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_24, family = binomial)

gamfitMed_eICU_vasso1_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_48, family = binomial)
gamfitMed_eICU_vasso0_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_48, family = binomial)

gamfitMed_eICU_vasso1_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_72, family = binomial)
gamfitMed_eICU_vasso0_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_72, family = binomial)


In [443]:
pdf(file = "figure/subgroup/Vasso_eICU_SBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(90, 140)
# hospital mortality
gam_plotMed(gamfitMed_eICU_vasso1_24h_hosp, main = "Vasopressor", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_vasso1_48h_hosp, main = "Vasopressor", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_vasso1_72h_hosp, main = "Vasopressor", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_vasso0_48h_hosp, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_vasso0_24h_hosp, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_vasso0_72h_hosp, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# icu mortality
gam_plotMed(gamfitMed_eICU_vasso1_24h_icu, main = "Vasopressor", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_vasso1_48h_icu, main = "Vasopressor", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_vasso1_72h_icu, main = "Vasopressor", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_vasso0_48h_icu, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_vasso0_24h_icu, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_vasso0_72h_icu, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# AKI
x_range = c(90, 140)
y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_eICU_vasso1_24h_aki, main = "Vasopressor", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_vasso1_48h_aki, main = "Vasopressor", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_vasso1_72h_aki, main = "Vasopressor", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_vasso0_48h_aki, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_vasso0_24h_aki, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_vasso0_72h_aki, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

dev.off()

[1] "Number of cases: 6557"
[1] "Number of cases: 6854"
[1] "Number of cases: 6962"
[1] "Number of cases: 7520"
[1] "Number of cases: 7817"
[1] "Number of cases: 7412"
[1] "Number of cases: 6557"
[1] "Number of cases: 6854"
[1] "Number of cases: 6962"
[1] "Number of cases: 7520"
[1] "Number of cases: 7817"
[1] "Number of cases: 7412"
[1] "Number of cases: 6557"
[1] "Number of cases: 6854"
[1] "Number of cases: 6962"
[1] "Number of cases: 7520"
[1] "Number of cases: 7817"
[1] "Number of cases: 7412"


png 
  2

### DBP

In [444]:
# hospital mortality
gamfitMed_eICU_vasso1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_24, family = binomial)
gamfitMed_eICU_vasso0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_24, family = binomial)

gamfitMed_eICU_vasso1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_48, family = binomial)
gamfitMed_eICU_vasso0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_48, family = binomial)

gamfitMed_eICU_vasso1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_72, family = binomial)
gamfitMed_eICU_vasso0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_72, family = binomial)

# icu mortality
gamfitMed_eICU_vasso1_24h_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_24, family = binomial)
gamfitMed_eICU_vasso0_24h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_24, family = binomial)

gamfitMed_eICU_vasso1_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_48, family = binomial)
gamfitMed_eICU_vasso0_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_48, family = binomial)

gamfitMed_eICU_vasso1_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_72, family = binomial)
gamfitMed_eICU_vasso0_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_72, family = binomial)


# AKI
gamfitMed_eICU_vasso1_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_24, family = binomial)
gamfitMed_eICU_vasso0_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_24, family = binomial)

gamfitMed_eICU_vasso1_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_48, family = binomial)
gamfitMed_eICU_vasso0_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_48, family = binomial)

gamfitMed_eICU_vasso1_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_72, family = binomial)
gamfitMed_eICU_vasso0_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_72, family = binomial)


In [445]:
pdf(file = "figure/subgroup/Vasso_eICU_DBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(35, 75)
# hospital mortality
gam_plotMed(gamfitMed_eICU_vasso1_24h_hosp, main = "Vasopressor", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_vasso1_48h_hosp, main = "Vasopressor", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_vasso1_72h_hosp, main = "Vasopressor", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_vasso0_48h_hosp, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_vasso0_24h_hosp, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_vasso0_72h_hosp, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# icu mortality
gam_plotMed(gamfitMed_eICU_vasso1_24h_icu, main = "Vasopressor", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_vasso1_48h_icu, main = "Vasopressor", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_vasso1_72h_icu, main = "Vasopressor", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_vasso0_48h_icu, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_vasso0_24h_icu, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_vasso0_72h_icu, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# AKI
x_range = c(35, 75)
y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_eICU_vasso1_24h_aki, main = "Vasopressor", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_vasso1_48h_aki, main = "Vasopressor", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_vasso1_72h_aki, main = "Vasopressor", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_vasso0_48h_aki, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_vasso0_24h_aki, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_vasso0_72h_aki, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

dev.off()

[1] "Number of cases: 6557"
[1] "Number of cases: 6854"
[1] "Number of cases: 6962"
[1] "Number of cases: 7520"
[1] "Number of cases: 7817"
[1] "Number of cases: 7412"
[1] "Number of cases: 6557"
[1] "Number of cases: 6854"
[1] "Number of cases: 6962"
[1] "Number of cases: 7520"
[1] "Number of cases: 7817"
[1] "Number of cases: 7412"
[1] "Number of cases: 6557"
[1] "Number of cases: 6854"
[1] "Number of cases: 6962"
[1] "Number of cases: 7520"
[1] "Number of cases: 7817"
[1] "Number of cases: 7412"


png 
  2

### MIMIC

In [446]:
df_vasso1_24 <-df_mimic_prop[df_mimic_prop$vasso_duration_24h >0,]
df_vasso0_24 <-df_mimic_prop[df_mimic_prop$vasso_duration_24h ==0,]
df_vasso1_48 <-df_mimic_prop[df_mimic_prop$vasso_duration_48h >0,]
df_vasso0_48 <-df_mimic_prop[df_mimic_prop$vasso_duration_48h ==0,]
df_vasso1_72 <-df_mimic_prop[df_mimic_prop$vasso_duration_72h >0,]
df_vasso0_72 <-df_mimic_prop[df_mimic_prop$vasso_duration_72h ==0,]

### map

In [437]:
# hospital mortality
gamfitMed_MIMIC_vasso1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_24, family = binomial)
gamfitMed_MIMIC_vasso0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_24, family = binomial)

gamfitMed_MIMIC_vasso1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_48, family = binomial)
gamfitMed_MIMIC_vasso0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_48, family = binomial)

gamfitMed_MIMIC_vasso1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_72, family = binomial)
gamfitMed_MIMIC_vasso0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_72, family = binomial)

# icu mortality
gamfitMed_MIMIC_vasso1_24h_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_24, family = binomial)
gamfitMed_MIMIC_vasso0_24h_icu  <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_24, family = binomial)

gamfitMed_MIMIC_vasso1_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_48, family = binomial)
gamfitMed_MIMIC_vasso0_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_48, family = binomial)

gamfitMed_MIMIC_vasso1_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_72, family = binomial)
gamfitMed_MIMIC_vasso0_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_72, family = binomial)


# AKI
gamfitMed_MIMIC_vasso1_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_24, family = binomial)
gamfitMed_MIMIC_vasso0_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_24, family = binomial)

gamfitMed_MIMIC_vasso1_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_48, family = binomial)
gamfitMed_MIMIC_vasso0_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_48, family = binomial)

gamfitMed_MIMIC_vasso1_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_72, family = binomial)
gamfitMed_MIMIC_vasso0_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_72, family = binomial)


In [438]:
pdf(file = "figure/subgroup/Vasso_MIMIC_MAP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(55, 95)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_vasso1_24h_hosp, main = "Vasopressor", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")
gam_plotMed(gamfitMed_MIMIC_vasso1_48h_hosp, main = "Vasopressor", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")
gam_plotMed(gamfitMed_MIMIC_vasso1_72h_hosp, main = "Vasopressor", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")
gam_plotMed(gamfitMed_MIMIC_vasso0_48h_hosp, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")
gam_plotMed(gamfitMed_MIMIC_vasso0_24h_hosp, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")
gam_plotMed(gamfitMed_MIMIC_vasso0_72h_hosp, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")
# icu mortality
gam_plotMed(gamfitMed_MIMIC_vasso1_24h_icu, main = "Vasopressor", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")
gam_plotMed(gamfitMed_MIMIC_vasso1_48h_icu, main = "Vasopressor", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")
gam_plotMed(gamfitMed_MIMIC_vasso1_72h_icu, main = "Vasopressor", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")
gam_plotMed(gamfitMed_MIMIC_vasso0_48h_icu, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")
gam_plotMed(gamfitMed_MIMIC_vasso0_24h_icu, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")
gam_plotMed(gamfitMed_MIMIC_vasso0_72h_icu, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")
# AKI
x_range = c(55, 95)
y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_MIMIC_vasso1_24h_aki, main = "Vasopressor", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")
gam_plotMed_noGradient(gamfitMed_MIMIC_vasso1_48h_aki, main = "Vasopressor", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")
gam_plotMed_noGradient(gamfitMed_MIMIC_vasso1_72h_aki, main = "Vasopressor", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")
gam_plotMed_noGradient(gamfitMed_MIMIC_vasso0_48h_aki, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")
gam_plotMed_noGradient(gamfitMed_MIMIC_vasso0_24h_aki, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")
gam_plotMed_noGradient(gamfitMed_MIMIC_vasso0_72h_aki, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

dev.off()

[1] "Number of cases: 6304"
[1] "Number of cases: 6731"
[1] "Number of cases: 6876"
[1] "Number of cases: 5560"
[1] "Number of cases: 5987"
[1] "Number of cases: 5415"
[1] "Number of cases: 6304"
[1] "Number of cases: 6731"
[1] "Number of cases: 6876"
[1] "Number of cases: 5560"
[1] "Number of cases: 5987"
[1] "Number of cases: 5415"
[1] "Number of cases: 6304"
[1] "Number of cases: 6731"
[1] "Number of cases: 6876"
[1] "Number of cases: 5560"
[1] "Number of cases: 5987"
[1] "Number of cases: 5415"


png 
  2

### SBP

In [447]:
# hospital mortality
gamfitMed_MIMIC_vasso1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_24, family = binomial)
gamfitMed_MIMIC_vasso0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_24, family = binomial)

gamfitMed_MIMIC_vasso1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_48, family = binomial)
gamfitMed_MIMIC_vasso0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_48, family = binomial)

gamfitMed_MIMIC_vasso1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_72, family = binomial)
gamfitMed_MIMIC_vasso0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_72, family = binomial)

# icu mortality
gamfitMed_MIMIC_vasso1_24h_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_24, family = binomial)
gamfitMed_MIMIC_vasso0_24h_icu  <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_24, family = binomial)

gamfitMed_MIMIC_vasso1_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_48, family = binomial)
gamfitMed_MIMIC_vasso0_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_48, family = binomial)

gamfitMed_MIMIC_vasso1_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_72, family = binomial)
gamfitMed_MIMIC_vasso0_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_72, family = binomial)


# AKI
gamfitMed_MIMIC_vasso1_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_24, family = binomial)
gamfitMed_MIMIC_vasso0_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_24, family = binomial)

gamfitMed_MIMIC_vasso1_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_48, family = binomial)
gamfitMed_MIMIC_vasso0_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_48, family = binomial)

gamfitMed_MIMIC_vasso1_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_72, family = binomial)
gamfitMed_MIMIC_vasso0_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_72, family = binomial)


In [448]:
pdf(file = "figure/subgroup/Vasso_MIMIC_SBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(90, 140)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_vasso1_24h_hosp, main = "Vasopressor", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")
gam_plotMed(gamfitMed_MIMIC_vasso1_48h_hosp, main = "Vasopressor", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")
gam_plotMed(gamfitMed_MIMIC_vasso1_72h_hosp, main = "Vasopressor", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")
gam_plotMed(gamfitMed_MIMIC_vasso0_48h_hosp, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")
gam_plotMed(gamfitMed_MIMIC_vasso0_24h_hosp, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")
gam_plotMed(gamfitMed_MIMIC_vasso0_72h_hosp, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")
# icu mortality
gam_plotMed(gamfitMed_MIMIC_vasso1_24h_icu, main = "Vasopressor", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")
gam_plotMed(gamfitMed_MIMIC_vasso1_48h_icu, main = "Vasopressor", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")
gam_plotMed(gamfitMed_MIMIC_vasso1_72h_icu, main = "Vasopressor", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")
gam_plotMed(gamfitMed_MIMIC_vasso0_48h_icu, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")
gam_plotMed(gamfitMed_MIMIC_vasso0_24h_icu, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")
gam_plotMed(gamfitMed_MIMIC_vasso0_72h_icu, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")
# AKI
x_range = c(90, 140)
y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_MIMIC_vasso1_24h_aki, main = "Vasopressor", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")
gam_plotMed_noGradient(gamfitMed_MIMIC_vasso1_48h_aki, main = "Vasopressor", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")
gam_plotMed_noGradient(gamfitMed_MIMIC_vasso1_72h_aki, main = "Vasopressor", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")
gam_plotMed_noGradient(gamfitMed_MIMIC_vasso0_48h_aki, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")
gam_plotMed_noGradient(gamfitMed_MIMIC_vasso0_24h_aki, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")
gam_plotMed_noGradient(gamfitMed_MIMIC_vasso0_72h_aki, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

dev.off()

[1] "Number of cases: 6304"
[1] "Number of cases: 6731"
[1] "Number of cases: 6876"
[1] "Number of cases: 5560"
[1] "Number of cases: 5987"
[1] "Number of cases: 5415"
[1] "Number of cases: 6304"
[1] "Number of cases: 6731"
[1] "Number of cases: 6876"
[1] "Number of cases: 5560"
[1] "Number of cases: 5987"
[1] "Number of cases: 5415"
[1] "Number of cases: 6304"
[1] "Number of cases: 6731"
[1] "Number of cases: 6876"
[1] "Number of cases: 5560"
[1] "Number of cases: 5987"
[1] "Number of cases: 5415"


png 
  2

### DBP

In [449]:
# hospital mortality
gamfitMed_MIMIC_vasso1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_24, family = binomial)
gamfitMed_MIMIC_vasso0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_24, family = binomial)

gamfitMed_MIMIC_vasso1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_48, family = binomial)
gamfitMed_MIMIC_vasso0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_48, family = binomial)

gamfitMed_MIMIC_vasso1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_72, family = binomial)
gamfitMed_MIMIC_vasso0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_72, family = binomial)

# icu mortality
gamfitMed_MIMIC_vasso1_24h_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_24, family = binomial)
gamfitMed_MIMIC_vasso0_24h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_24, family = binomial)

gamfitMed_MIMIC_vasso1_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_48, family = binomial)
gamfitMed_MIMIC_vasso0_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_48, family = binomial)

gamfitMed_MIMIC_vasso1_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_72, family = binomial)
gamfitMed_MIMIC_vasso0_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_72, family = binomial)


# AKI
gamfitMed_MIMIC_vasso1_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_24, family = binomial)
gamfitMed_MIMIC_vasso0_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_24, family = binomial)

gamfitMed_MIMIC_vasso1_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_48, family = binomial)
gamfitMed_MIMIC_vasso0_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_48, family = binomial)

gamfitMed_MIMIC_vasso1_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_72, family = binomial)
gamfitMed_MIMIC_vasso0_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_72, family = binomial)


In [450]:
pdf(file = "figure/subgroup/Vasso_MIMIC_DBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(35, 75)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_vasso1_24h_hosp, main = "Vasopressor", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")
gam_plotMed(gamfitMed_MIMIC_vasso1_48h_hosp, main = "Vasopressor", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")
gam_plotMed(gamfitMed_MIMIC_vasso1_72h_hosp, main = "Vasopressor", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")
gam_plotMed(gamfitMed_MIMIC_vasso0_48h_hosp, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")
gam_plotMed(gamfitMed_MIMIC_vasso0_24h_hosp, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")
gam_plotMed(gamfitMed_MIMIC_vasso0_72h_hosp, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")
# icu mortality
gam_plotMed(gamfitMed_MIMIC_vasso1_24h_icu, main = "Vasopressor", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")
gam_plotMed(gamfitMed_MIMIC_vasso1_48h_icu, main = "Vasopressor", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")
gam_plotMed(gamfitMed_MIMIC_vasso1_72h_icu, main = "Vasopressor", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")
gam_plotMed(gamfitMed_MIMIC_vasso0_48h_icu, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")
gam_plotMed(gamfitMed_MIMIC_vasso0_24h_icu, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")
gam_plotMed(gamfitMed_MIMIC_vasso0_72h_icu, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")
# AKI
x_range = c(35, 75)
y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_MIMIC_vasso1_24h_aki, main = "Vasopressor", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")
gam_plotMed_noGradient(gamfitMed_MIMIC_vasso1_48h_aki, main = "Vasopressor", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")
gam_plotMed_noGradient(gamfitMed_MIMIC_vasso1_72h_aki, main = "Vasopressor", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")
gam_plotMed_noGradient(gamfitMed_MIMIC_vasso0_48h_aki, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")
gam_plotMed_noGradient(gamfitMed_MIMIC_vasso0_24h_aki, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")
gam_plotMed_noGradient(gamfitMed_MIMIC_vasso0_72h_aki, main = "No Vasopressor", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

dev.off()

[1] "Number of cases: 6304"
[1] "Number of cases: 6731"
[1] "Number of cases: 6876"
[1] "Number of cases: 5560"
[1] "Number of cases: 5987"


Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”

[1] "Number of cases: 5415"
[1] "Number of cases: 6304"
[1] "Number of cases: 6731"
[1] "Number of cases: 6876"
[1] "Number of cases: 5560"
[1] "Number of cases: 5987"
[1] "Number of cases: 5415"
[1] "Number of cases: 6304"
[1] "Number of cases: 6731"
[1] "Number of cases: 6876"
[1] "Number of cases: 5560"
[1] "Number of cases: 5987"
[1] "Number of cases: 5415"


png 
  2

## vasso hour >=50% total hours vs. <50% 

In [454]:
df_vasso1_24 <-df_eicu_prop[df_eicu_prop$vasso_duration_24h>=12,]
df_vasso0_24 <-df_eicu_prop[df_eicu_prop$vasso_duration_24h<12,]
df_vasso1_48 <-df_eicu_prop[df_eicu_prop$vasso_duration_48h >=24,]
df_vasso0_48 <-df_eicu_prop[df_eicu_prop$vasso_duration_48h <24,]
df_vasso1_72 <-df_eicu_prop[df_eicu_prop$vasso_duration_72h >=36,]
df_vasso0_72 <-df_eicu_prop[df_eicu_prop$vasso_duration_72h <36,]

In [452]:
# hospital mortality
gamfitMed_eICU_vasso1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_24, family = binomial)
gamfitMed_eICU_vasso0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_24, family = binomial)

gamfitMed_eICU_vasso1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_48, family = binomial)
gamfitMed_eICU_vasso0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_48, family = binomial)

gamfitMed_eICU_vasso1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_72, family = binomial)
gamfitMed_eICU_vasso0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_72, family = binomial)

# icu mortality
gamfitMed_eICU_vasso1_24h_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_24, family = binomial)
gamfitMed_eICU_vasso0_24h_icu  <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_24, family = binomial)

gamfitMed_eICU_vasso1_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_48, family = binomial)
gamfitMed_eICU_vasso0_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_48, family = binomial)

gamfitMed_eICU_vasso1_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_72, family = binomial)
gamfitMed_eICU_vasso0_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_72, family = binomial)


# AKI
gamfitMed_eICU_vasso1_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_24, family = binomial)
gamfitMed_eICU_vasso0_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_24, family = binomial)

gamfitMed_eICU_vasso1_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_48, family = binomial)
gamfitMed_eICU_vasso0_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_48, family = binomial)

gamfitMed_eICU_vasso1_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_72, family = binomial)
gamfitMed_eICU_vasso0_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_72, family = binomial)


In [453]:
pdf(file = "figure/subgroup/Vasso50_eICU_MAP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(55, 95)
# hospital mortality
gam_plotMed(gamfitMed_eICU_vasso1_24h_hosp, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_vasso1_48h_hosp, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_vasso1_72h_hosp, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_vasso0_48h_hosp, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_vasso0_24h_hosp, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_vasso0_72h_hosp, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# icu mortality
gam_plotMed(gamfitMed_eICU_vasso1_24h_icu, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_vasso1_48h_icu, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_vasso1_72h_icu, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_vasso0_48h_icu, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_vasso0_24h_icu, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_vasso0_72h_icu, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# AKI
x_range = c(55, 95)
y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_eICU_vasso1_24h_aki, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_vasso1_48h_aki, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_vasso1_72h_aki, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_vasso0_48h_aki, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_vasso0_24h_aki, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_vasso0_72h_aki, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

dev.off()

[1] "Number of cases: 5614"
[1] "Number of cases: 5372"
[1] "Number of cases: 4829"
[1] "Number of cases: 9002"
[1] "Number of cases: 8760"
[1] "Number of cases: 9545"
[1] "Number of cases: 5614"
[1] "Number of cases: 5372"
[1] "Number of cases: 4829"
[1] "Number of cases: 9002"
[1] "Number of cases: 8760"
[1] "Number of cases: 9545"
[1] "Number of cases: 5614"
[1] "Number of cases: 5372"
[1] "Number of cases: 4829"
[1] "Number of cases: 9002"
[1] "Number of cases: 8760"
[1] "Number of cases: 9545"


png 
  2

### SBP

In [455]:
# hospital mortality
gamfitMed_eICU_vasso1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_24, family = binomial)
gamfitMed_eICU_vasso0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_24, family = binomial)

gamfitMed_eICU_vasso1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_48, family = binomial)
gamfitMed_eICU_vasso0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_48, family = binomial)

gamfitMed_eICU_vasso1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_72, family = binomial)
gamfitMed_eICU_vasso0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_72, family = binomial)

# icu mortality
gamfitMed_eICU_vasso1_24h_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_24, family = binomial)
gamfitMed_eICU_vasso0_24h_icu  <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_24, family = binomial)

gamfitMed_eICU_vasso1_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_48, family = binomial)
gamfitMed_eICU_vasso0_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_48, family = binomial)

gamfitMed_eICU_vasso1_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_72, family = binomial)
gamfitMed_eICU_vasso0_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_72, family = binomial)


# AKI
gamfitMed_eICU_vasso1_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_24, family = binomial)
gamfitMed_eICU_vasso0_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_24, family = binomial)

gamfitMed_eICU_vasso1_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_48, family = binomial)
gamfitMed_eICU_vasso0_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_48, family = binomial)

gamfitMed_eICU_vasso1_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_72, family = binomial)
gamfitMed_eICU_vasso0_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_72, family = binomial)


In [456]:
pdf(file = "figure/subgroup/Vasso50_eICU_SBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(90, 140)
# hospital mortality
gam_plotMed(gamfitMed_eICU_vasso1_24h_hosp, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_vasso1_48h_hosp, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_vasso1_72h_hosp, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_vasso0_48h_hosp, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_vasso0_24h_hosp, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_vasso0_72h_hosp, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# icu mortality
gam_plotMed(gamfitMed_eICU_vasso1_24h_icu, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_vasso1_48h_icu, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_vasso1_72h_icu, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_vasso0_48h_icu, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_vasso0_24h_icu, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_vasso0_72h_icu, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# AKI
x_range = c(90, 140)
y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_eICU_vasso1_24h_aki, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_vasso1_48h_aki, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_vasso1_72h_aki, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_vasso0_48h_aki, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_vasso0_24h_aki, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_vasso0_72h_aki, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

dev.off()

[1] "Number of cases: 5614"
[1] "Number of cases: 5372"
[1] "Number of cases: 4829"
[1] "Number of cases: 9002"
[1] "Number of cases: 8760"
[1] "Number of cases: 9545"
[1] "Number of cases: 5614"
[1] "Number of cases: 5372"
[1] "Number of cases: 4829"
[1] "Number of cases: 9002"
[1] "Number of cases: 8760"
[1] "Number of cases: 9545"
[1] "Number of cases: 5614"
[1] "Number of cases: 5372"
[1] "Number of cases: 4829"
[1] "Number of cases: 9002"
[1] "Number of cases: 8760"
[1] "Number of cases: 9545"


png 
  2

### DBP

In [457]:
# hospital mortality
gamfitMed_eICU_vasso1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_24, family = binomial)
gamfitMed_eICU_vasso0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_24, family = binomial)

gamfitMed_eICU_vasso1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_48, family = binomial)
gamfitMed_eICU_vasso0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_48, family = binomial)

gamfitMed_eICU_vasso1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_72, family = binomial)
gamfitMed_eICU_vasso0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_72, family = binomial)

# icu mortality
gamfitMed_eICU_vasso1_24h_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_24, family = binomial)
gamfitMed_eICU_vasso0_24h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_24, family = binomial)

gamfitMed_eICU_vasso1_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_48, family = binomial)
gamfitMed_eICU_vasso0_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_48, family = binomial)

gamfitMed_eICU_vasso1_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_72, family = binomial)
gamfitMed_eICU_vasso0_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_72, family = binomial)


# AKI
gamfitMed_eICU_vasso1_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_24, family = binomial)
gamfitMed_eICU_vasso0_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_24, family = binomial)

gamfitMed_eICU_vasso1_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_48, family = binomial)
gamfitMed_eICU_vasso0_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_48, family = binomial)

gamfitMed_eICU_vasso1_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_72, family = binomial)
gamfitMed_eICU_vasso0_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_72, family = binomial)


In [458]:
pdf(file = "figure/subgroup/Vasso50_eICU_DBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(35, 75)
# hospital mortality
gam_plotMed(gamfitMed_eICU_vasso1_24h_hosp, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_vasso1_48h_hosp, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_vasso1_72h_hosp, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_vasso0_48h_hosp, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_vasso0_24h_hosp, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_vasso0_72h_hosp, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# icu mortality
gam_plotMed(gamfitMed_eICU_vasso1_24h_icu, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_vasso1_48h_icu, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_vasso1_72h_icu, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_vasso0_48h_icu, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_vasso0_24h_icu, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_vasso0_72h_icu, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# AKI
x_range = c(35, 75)
y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_eICU_vasso1_24h_aki, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_vasso1_48h_aki, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_vasso1_72h_aki, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_vasso0_48h_aki, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_vasso0_24h_aki, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_vasso0_72h_aki, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

dev.off()

[1] "Number of cases: 5614"
[1] "Number of cases: 5372"
[1] "Number of cases: 4829"
[1] "Number of cases: 9002"
[1] "Number of cases: 8760"
[1] "Number of cases: 9545"
[1] "Number of cases: 5614"
[1] "Number of cases: 5372"
[1] "Number of cases: 4829"
[1] "Number of cases: 9002"
[1] "Number of cases: 8760"
[1] "Number of cases: 9545"
[1] "Number of cases: 5614"
[1] "Number of cases: 5372"
[1] "Number of cases: 4829"
[1] "Number of cases: 9002"
[1] "Number of cases: 8760"
[1] "Number of cases: 9545"


png 
  2

## MIMIC

In [459]:
df_vasso1_24 <-df_mimic[df_mimic$vasso_duration_24h >=12,]
df_vasso0_24 <-df_mimic[df_mimic$vasso_duration_24h <12,]
df_vasso1_48 <-df_mimic[df_mimic$vasso_duration_48h >=24,]
df_vasso0_48 <-df_mimic[df_mimic$vasso_duration_48h <24,]
df_vasso1_72 <-df_mimic[df_mimic$vasso_duration_72h >=36,]
df_vasso0_72 <-df_mimic[df_mimic$vasso_duration_72h <36,]

### MAP

In [460]:
# hospital mortality
gamfitMed_MIMIC_vasso1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_24, family = binomial)
gamfitMed_MIMIC_vasso0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_24, family = binomial)

gamfitMed_MIMIC_vasso1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_48, family = binomial)
gamfitMed_MIMIC_vasso0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_48, family = binomial)

gamfitMed_MIMIC_vasso1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_72, family = binomial)
gamfitMed_MIMIC_vasso0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_72, family = binomial)

# icu mortality
gamfitMed_MIMIC_vasso1_24h_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_24, family = binomial)
gamfitMed_MIMIC_vasso0_24h_icu  <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_24, family = binomial)

gamfitMed_MIMIC_vasso1_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_48, family = binomial)
gamfitMed_MIMIC_vasso0_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_48, family = binomial)

gamfitMed_MIMIC_vasso1_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_72, family = binomial)
gamfitMed_MIMIC_vasso0_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_72, family = binomial)


# AKI
gamfitMed_MIMIC_vasso1_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_24, family = binomial)
gamfitMed_MIMIC_vasso0_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_24, family = binomial)

gamfitMed_MIMIC_vasso1_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_48, family = binomial)
gamfitMed_MIMIC_vasso0_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_48, family = binomial)

gamfitMed_MIMIC_vasso1_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_72, family = binomial)
gamfitMed_MIMIC_vasso0_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_72, family = binomial)


In [461]:
pdf(file = "figure/subgroup/Vasso50_MIMIC_MAP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(55, 95)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_vasso1_24h_hosp, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_vasso1_48h_hosp, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_vasso1_72h_hosp, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_vasso0_48h_hosp, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_vasso0_24h_hosp, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_vasso0_72h_hosp, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# icu mortality
gam_plotMed(gamfitMed_MIMIC_vasso1_24h_icu, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_vasso1_48h_icu, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_vasso1_72h_icu, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_vasso0_48h_icu, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_vasso0_24h_icu, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_vasso0_72h_icu, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# AKI
x_range = c(55, 95)
y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_MIMIC_vasso1_24h_aki, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_vasso1_48h_aki, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_vasso1_72h_aki, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_vasso0_48h_aki, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_vasso0_24h_aki, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_vasso0_72h_aki, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

dev.off()

[1] "Number of cases: 3944"
[1] "Number of cases: 3229"
[1] "Number of cases: 2508"
[1] "Number of cases: 9062"
[1] "Number of cases: 8347"
[1] "Number of cases: 9783"
[1] "Number of cases: 3944"
[1] "Number of cases: 3229"
[1] "Number of cases: 2508"
[1] "Number of cases: 9062"
[1] "Number of cases: 8347"
[1] "Number of cases: 9783"
[1] "Number of cases: 3944"
[1] "Number of cases: 3229"
[1] "Number of cases: 2508"
[1] "Number of cases: 9062"
[1] "Number of cases: 8347"
[1] "Number of cases: 9783"


png 
  2

### SBP

In [462]:
# hospital mortality
gamfitMed_MIMIC_vasso1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_24, family = binomial)
gamfitMed_MIMIC_vasso0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_24, family = binomial)

gamfitMed_MIMIC_vasso1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_48, family = binomial)
gamfitMed_MIMIC_vasso0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_48, family = binomial)

gamfitMed_MIMIC_vasso1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_72, family = binomial)
gamfitMed_MIMIC_vasso0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_72, family = binomial)

# icu mortality
gamfitMed_MIMIC_vasso1_24h_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_24, family = binomial)
gamfitMed_MIMIC_vasso0_24h_icu  <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_24, family = binomial)

gamfitMed_MIMIC_vasso1_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_48, family = binomial)
gamfitMed_MIMIC_vasso0_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_48, family = binomial)

gamfitMed_MIMIC_vasso1_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_72, family = binomial)
gamfitMed_MIMIC_vasso0_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_72, family = binomial)


# AKI
gamfitMed_MIMIC_vasso1_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_24, family = binomial)
gamfitMed_MIMIC_vasso0_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_24, family = binomial)

gamfitMed_MIMIC_vasso1_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_48, family = binomial)
gamfitMed_MIMIC_vasso0_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_48, family = binomial)

gamfitMed_MIMIC_vasso1_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_72, family = binomial)
gamfitMed_MIMIC_vasso0_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_72, family = binomial)


In [463]:
pdf(file = "figure/subgroup/Vasso50_MIMIC_SBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(90, 140)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_vasso1_24h_hosp, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_vasso1_48h_hosp, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_vasso1_72h_hosp, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_vasso0_48h_hosp, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_vasso0_24h_hosp, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_vasso0_72h_hosp, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# icu mortality
gam_plotMed(gamfitMed_MIMIC_vasso1_24h_icu, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_vasso1_48h_icu, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_vasso1_72h_icu, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_vasso0_48h_icu, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_vasso0_24h_icu, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_vasso0_72h_icu, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_MIMIC_vasso1_24h_aki, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_vasso1_48h_aki, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_vasso1_72h_aki, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_vasso0_48h_aki, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_vasso0_24h_aki, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_vasso0_72h_aki, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

dev.off()

[1] "Number of cases: 3944"
[1] "Number of cases: 3229"
[1] "Number of cases: 2508"
[1] "Number of cases: 9062"
[1] "Number of cases: 8347"
[1] "Number of cases: 9783"
[1] "Number of cases: 3944"
[1] "Number of cases: 3229"
[1] "Number of cases: 2508"
[1] "Number of cases: 9062"
[1] "Number of cases: 8347"
[1] "Number of cases: 9783"
[1] "Number of cases: 3944"
[1] "Number of cases: 3229"
[1] "Number of cases: 2508"
[1] "Number of cases: 9062"
[1] "Number of cases: 8347"
[1] "Number of cases: 9783"


png 
  2

### DBP

In [464]:
# hospital mortality
gamfitMed_MIMIC_vasso1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_24, family = binomial)
gamfitMed_MIMIC_vasso0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_24, family = binomial)

gamfitMed_MIMIC_vasso1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_48, family = binomial)
gamfitMed_MIMIC_vasso0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_48, family = binomial)

gamfitMed_MIMIC_vasso1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_72, family = binomial)
gamfitMed_MIMIC_vasso0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_72, family = binomial)

# icu mortality
gamfitMed_MIMIC_vasso1_24h_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_24, family = binomial)
gamfitMed_MIMIC_vasso0_24h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_24, family = binomial)

gamfitMed_MIMIC_vasso1_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_48, family = binomial)
gamfitMed_MIMIC_vasso0_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_48, family = binomial)

gamfitMed_MIMIC_vasso1_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_72, family = binomial)
gamfitMed_MIMIC_vasso0_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_72, family = binomial)


# AKI
gamfitMed_MIMIC_vasso1_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_24, family = binomial)
gamfitMed_MIMIC_vasso0_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_24, family = binomial)

gamfitMed_MIMIC_vasso1_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_48, family = binomial)
gamfitMed_MIMIC_vasso0_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_48, family = binomial)

gamfitMed_MIMIC_vasso1_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso1_72, family = binomial)
gamfitMed_MIMIC_vasso0_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_vasso0_72, family = binomial)


In [470]:
gam_plotMed_DBP <- function(
                    gamfitMed,
                    main = "Median of measurements",
                    xRange = c(55, 95),
                    yRange = c(0, .20),
                    label,
                    xlabel
) {
print(paste("Number of cases:", summary(gamfitMed)$n))

xlab <- xlabel


if(colnames(gamfitMed$model)[1] == "hospital_expire_flag") {
    ylab <- "Probability of hospital mortality"
  }
    else if(colnames(gamfitMed$model)[1] == "AKI_7d") {
    ylab <- "Probability of acute kidney injury (AKI)"
  } 
    else {
    ylab <- "Probability of ICU mortality"
  }

xName <- colnames(gamfitMed$model)[grep("ibp", colnames(gamfitMed$model))]
# vasso_Name <- colnames(gamfitMed$model)[grep("vasso", colnames(gamfitMed$model))]

# Color for dotted line
colD <- "Black"
# colD <- "Blue"

    plot(1, type = 'n', xlim = xRange, ylim = yRange,
         ylab = ylab,
         xlab = xlab, main = main, yaxs = 'i', xaxs = 'i', yaxt = 'n', xaxt = 'n')
    
    att <- pretty(yRange)
if(!isTRUE(all.equal(att, round(att, digits = 2)))) {
  axis(2, at = att, lab = paste0(sprintf('%.1f', att*100), '%'), las = TRUE)
} else axis(2, at = att, lab = paste0(att*100, '%'), las = TRUE)
    
    att <- pretty(xRange)
    axis(1, at = att, lab = paste0(att), las = TRUE)
#     axis(1, at = att, lab = paste0(att, '%'), las = TRUE)

    
    eval(parse(text = paste(c('predictionsPlusCI <- predict(gamfitMed, newdata = data.frame(',
                              xName, ' = gamfitMed$model$', xName, ", gender = 'F', age = median(gamfitMed$model$age), 
                            bmi = median(gamfitMed$model$bmi),",
                      ifelse(
                        "high_vent_proportion" %in% colnames(gamfitMed$model),
                        "high_vent_proportion = median(gamfitMed$model$high_vent_proportion),",
                        ""
                      ),
                      ifelse(
                        "ethnicity" %in% colnames(gamfitMed$model),
                        "ethnicity = gamfitMed$model$ethnicity,",
                        ""
                      ),
                      ifelse(
                        "apsiii" %in% colnames(gamfitMed$model),
                        "apsiii = median(gamfitMed$model$apsiii),",
                        "sofatotal = median(gamfitMed$model$sofatotal),"
                        
                      ),
                      ifelse(
                        "vasso_duration_24h" %in% colnames(gamfitMed$model),
                        "vasso_duration_24h = median(gamfitMed$model$vasso_duration_24h),",
                        ""
                      ),
                      ifelse(
                        "vasso_duration_48h" %in% colnames(gamfitMed$model),
                        "vasso_duration_48h = median(gamfitMed$model$vasso_duration_48h),",
                        ""
                      ),
                      ifelse(
                        "vasso_duration_72h" %in% colnames(gamfitMed$model),
                        "vasso_duration_72h = median(gamfitMed$model$vasso_duration_72h),",
                        ""
                      ),
                      "hospital_id = 264), type = 'link', se.fit = T)"), collapse = "")))

  
  # We have to use the data on which GAM was fit for confidence region as the GAM does not provide standard errors for 'new' input
    eval(parse(text = paste0('xx <- gamfitMed$model$', xName)))
    ord.index <- order(xx)
    xx <- xx[ord.index]
    i<-2
    j<-1
    index <-1
    bp_val <-list()
    bp_grad<-list()
    current_list <-list()
    next_list<-list()
    while (i <=length(ord.index)){
        eval(parse(text = paste(c('x2 <- gamfitMed$model$', xName,"[ord.index[",i,"]]"), collapse = "")))
        eval(parse(text = paste(c('x1 <- gamfitMed$model$', xName,"[ord.index[",j,"]]"), collapse = "")))
#       x2 <- b$model$ibp_mean_24h[ord.index[i]]
#       x1 <-b$model$ibp_mean_24h[ord.index[j]]

        if((x2-x1)>=1.0){
            current_val <- logistic(predictionsPlusCI$fit[ord.index[j]])
            right_val <- logistic(predictionsPlusCI$fit[ord.index[i]])
            grad <-(right_val-current_val)*100/(x2-x1)
            bp_val[[index]] <- x1
            bp_grad[[index]]<-unlist(grad,use.names=FALSE)
            current_list[[index]]<-current_val
            next_list[[index]]<-right_val
            index <-index+1
            j <- i
            i <- i+1

        } else {
            i <-i+1
        }

        }
    df_gradient <- data.frame("bp_value" = unlist(bp_val, use.names=FALSE), 
                        "gradient" = unlist(bp_grad, use.names=FALSE)
                       , "pre_val_left" = unlist(current_list, use.names=FALSE)
                       , "pre_val_right" = unlist(next_list, use.names=FALSE))
    
    by_type <- colnames(gamfitMed$model)[grep("ibp", colnames(gamfitMed$model))]
    if(startsWith(xName, 'ibp_mean', ignore.case=TRUE)){
        bp_value_left <-60
        bp_value_right<-90
    } else if (startsWith(xName, 'ibp_systolic', ignore.case=TRUE)){
        bp_value_left <-100
        bp_value_right<-140
        
    } else {
        bp_value_left <-40
        bp_value_right<-70
    }
    
    
    grad_0.2_left<-min(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient< 0.2)
                                   & (df_gradient$bp_value< bp_value_right) & (df_gradient$bp_value> bp_value_left), "bp_value"])
    grad_0.2_right<-max(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient< 0.2)
                               & (df_gradient$bp_value< bp_value_right) & (df_gradient$bp_value> bp_value_left), "bp_value"])
    grad_0.1_left<-min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient< 0.1)
                                   & (df_gradient$bp_value< bp_value_right) & (df_gradient$bp_value> bp_value_left), "bp_value"])
    grad_0.1_right<-max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient< 0.1)
                               & (df_gradient$bp_value< bp_value_right) & (df_gradient$bp_value> bp_value_left), "bp_value"])
    grad_0.2_left <-ifelse(grad_0.2_left>=43, grad_0.2_left,40)
    grad_0.1_left <-ifelse(grad_0.1_left>=43, grad_0.1_left,41)
    grad_0.2_right<-ifelse(grad_0.2_right<=137, grad_0.2_right,139)
    grad_0.1_right<-ifelse(grad_0.1_right<=137, grad_0.1_right,138)

    if(gamfitMed$family$link == 'logit') {
        lcl <- logistic(predictionsPlusCI$fit[ord.index] - 1.96*predictionsPlusCI$se.fit[ord.index])
        ucl <- logistic(predictionsPlusCI$fit[ord.index] + 1.96*predictionsPlusCI$se.fit[ord.index])

        lines(x = xx, y = lcl, lty = 2, lwd = 2, col = colD)
        lines(x = xx, y = ucl, lty = 2, lwd = 2, col = colD)
        lines(xx, logistic(predictionsPlusCI$fit[ord.index]), lwd = 3)

        abline(v=57, col = "Red")
        abline(v=60, col = "Red")
        abline(v=58, col = "Blue")
        abline(v=59, col = "Blue")

    } else {
        lcl <- predictionsPlusCI$fit[ord.index] - 1.96*predictionsPlusCI$se.fit[ord.index]
        ucl <- predictionsPlusCI$fit[ord.index] + 1.96*predictionsPlusCI$se.fit[ord.index]

        lines(x = xx, y = lcl, lty = 2, lwd = 2)
        lines(x = xx, y = ucl, lty = 2, lwd = 2)
        lines(xx, predictionsPlusCI$fit[ord.index], lwd = 3)
        abline(v=grad_0.2_left, col = "Red")
        abline(v=grad_0.2_right, col = "Red")
        abline(v=grad_0.1_left, col = "Blue")
        abline(v=grad_0.1_right, col = "Blue")
    }

    if(!missing(label)) fig_label(label)
}


In [472]:
pdf(file = "figure/subgroup/Vasso50_MIMIC_DBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(35, 75)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_vasso1_24h_hosp, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_vasso1_48h_hosp, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_DBP(gamfitMed_MIMIC_vasso1_72h_hosp, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_vasso0_48h_hosp, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_vasso0_24h_hosp, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_vasso0_72h_hosp, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# icu mortality
gam_plotMed(gamfitMed_MIMIC_vasso1_24h_icu, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_vasso1_48h_icu, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_DBP(gamfitMed_MIMIC_vasso1_72h_icu, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_vasso0_48h_icu, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_vasso0_24h_icu, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_vasso0_72h_icu, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_MIMIC_vasso1_24h_aki, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_vasso1_48h_aki, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_vasso1_72h_aki, main = "Vasopressor >=50% Time", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_vasso0_48h_aki, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_vasso0_24h_aki, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_vasso0_72h_aki, main = "Vasopressor <50% Time", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

dev.off()

[1] "Number of cases: 3944"
[1] "Number of cases: 3229"
[1] "Number of cases: 2508"
[1] "Number of cases: 9062"


Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”

[1] "Number of cases: 8347"


Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”

[1] "Number of cases: 9783"
[1] "Number of cases: 3944"
[1] "Number of cases: 3229"
[1] "Number of cases: 2508"
[1] "Number of cases: 9062"
[1] "Number of cases: 8347"
[1] "Number of cases: 9783"
[1] "Number of cases: 3944"
[1] "Number of cases: 3229"
[1] "Number of cases: 2508"
[1] "Number of cases: 9062"
[1] "Number of cases: 8347"
[1] "Number of cases: 9783"


png 
  2

## Sepsis

In [474]:
df_sepsis1 <-df_mimic_prop[df_mimic_prop$sepsis==1,]
df_sepsis0 <-df_mimic_prop[df_mimic_prop$sepsis==0,]

## eICU

### MAP

In [483]:
# hospital mortality
gamfitMed_eICU_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_eICU_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_eICU_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


In [484]:
pdf(file = "figure/subgroup/Sepsis_eICU_MAP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(55, 95)
# hospital mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_hosp, main = "Sepsis = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_hosp, main = "Sepsis = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_hosp, main = "Sepsis = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_hosp, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_hosp, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_hosp, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# icu mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_icu, main = "Sepsis = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_icu, main = "Sepsis = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_icu, main = "Sepsis = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_icu, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_icu, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_icu, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_24h_aki, main = "Sepsis = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_48h_aki, main = "Sepsis = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_72h_aki, main = "Sepsis = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_48h_aki, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_24h_aki, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_72h_aki, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

dev.off()

[1] "Number of cases: 4448"
[1] "Number of cases: 4448"
[1] "Number of cases: 4448"
[1] "Number of cases: 7843"
[1] "Number of cases: 7843"
[1] "Number of cases: 7843"
[1] "Number of cases: 4448"
[1] "Number of cases: 4448"
[1] "Number of cases: 4448"
[1] "Number of cases: 7843"
[1] "Number of cases: 7843"
[1] "Number of cases: 7843"
[1] "Number of cases: 4448"
[1] "Number of cases: 4448"
[1] "Number of cases: 4448"
[1] "Number of cases: 7843"
[1] "Number of cases: 7843"
[1] "Number of cases: 7843"


png 
  2

### SBP

In [478]:
# hospital mortality
gamfitMed_eICU_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_eICU_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_eICU_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


In [482]:
pdf(file = "figure/subgroup/Sepsis_eICU_SBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(90, 140)
# hospital mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_hosp, main = "Sepsis = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_hosp, main = "Sepsis = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_hosp, main = "Sepsis = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_hosp, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_hosp, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_hosp, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# icu mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_icu, main = "Sepsis = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_icu, main = "Sepsis = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_icu, main = "Sepsis = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_icu, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_icu, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_icu, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_24h_aki, main = "Sepsis = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_48h_aki, main = "Sepsis = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_72h_aki, main = "Sepsis = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_48h_aki, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_24h_aki, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_72h_aki, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

dev.off()

[1] "Number of cases: 4448"
[1] "Number of cases: 4448"
[1] "Number of cases: 4448"
[1] "Number of cases: 7843"
[1] "Number of cases: 7843"
[1] "Number of cases: 7843"
[1] "Number of cases: 4448"
[1] "Number of cases: 4448"
[1] "Number of cases: 4448"
[1] "Number of cases: 7843"
[1] "Number of cases: 7843"
[1] "Number of cases: 7843"
[1] "Number of cases: 4448"
[1] "Number of cases: 4448"
[1] "Number of cases: 4448"
[1] "Number of cases: 7843"
[1] "Number of cases: 7843"
[1] "Number of cases: 7843"


png 
  2

### DBP

In [485]:
# hospital mortality
gamfitMed_eICU_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_eICU_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_eICU_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


In [494]:
pdf(file = "figure/subgroup/Sepsis_eICU_DBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(35, 75)
# hospital mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_hosp, main = "Sepsis = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_hosp, main = "Sepsis = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_hosp, main = "Sepsis = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_hosp, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_hosp, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_hosp, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# icu mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_icu, main = "Sepsis = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_icu, main = "Sepsis = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_icu, main = "Sepsis = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_icu, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_icu, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_icu, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_24h_aki, main = "Sepsis = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_48h_aki, main = "Sepsis = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_72h_aki, main = "Sepsis = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_48h_aki, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_24h_aki, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_72h_aki, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

dev.off()

Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”

png 
  2

## MIMIC

In [495]:
df_sepsis1 <-df_mimic[df_mimic$sepsis==1,]
df_sepsis0 <-df_mimic[df_mimic$sepsis==0,]

### MAP

In [496]:
# MIMIC MAP
# MIMIC MAP
# hospital mortality
gamfitMed_MIMIC_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_MIMIC_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_MIMIC_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/Sepsis_MIMIC_MAP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(55, 95)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_hosp, main = "Sepsis = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_hosp, main = "Sepsis = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_hosp, main = "Sepsis = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_hosp, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_hosp, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_hosp, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# icu mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_icu, main = "Sepsis = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_icu, main = "Sepsis = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_icu, main = "Sepsis = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_icu, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_icu, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_icu, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_24h_aki, main = "Sepsis = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_48h_aki, main = "Sepsis = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_72h_aki, main = "Sepsis = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_48h_aki, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_24h_aki, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_72h_aki, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

dev.off()

png 
  2

### SBP

In [497]:
# MIMIC SBP
# MIMIC SBP
# hospital mortality
gamfitMed_MIMIC_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_MIMIC_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_MIMIC_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/Sepsis_MIMIC_SBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(90, 140)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_hosp, main = "Sepsis = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_hosp, main = "Sepsis = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_hosp, main = "Sepsis = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_hosp, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_hosp, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_hosp, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# icu mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_icu, main = "Sepsis = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_icu, main = "Sepsis = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_icu, main = "Sepsis = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_icu, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_icu, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_icu, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_24h_aki, main = "Sepsis = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_48h_aki, main = "Sepsis = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_72h_aki, main = "Sepsis = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_48h_aki, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_24h_aki, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_72h_aki, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

dev.off()

png 
  2

### DBP

In [498]:
# MIMIC DBP
# hospital mortality
gamfitMed_MIMIC_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_MIMIC_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_MIMIC_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)



Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”

png 
  2

In [516]:
pdf(file = "figure/subgroup/Sepsis_MIMIC_DBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)

par(mfcol = c(3,6), cex = 1)
x_range = c(35, 75)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_hosp, main = "Sepsis = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_hosp, main = "Sepsis = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_hosp, main = "Sepsis = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_hosp, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_hosp, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_hosp, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# icu mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_icu, main = "Sepsis = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_icu, main = "Sepsis = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_icu, main = "Sepsis = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_icu, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_icu, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_icu, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_24h_aki, main = "Sepsis = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_48h_aki, main = "Sepsis = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_72h_aki, main = "Sepsis = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_48h_aki, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_24h_aki, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_72h_aki, main = "Sepsis = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# mtext("My 'Title' in a strange place", side = 1, line = -2, outer = TRUE)
# title("MIMIC DBP vs hospital mortality, ICU mortality and AKI", outer = TRUE, cex =1.5, font =3)
dev.off()

Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”

png 
  2

## age >=65 vs. age <65

In [521]:
df_sepsis1 <-df_eicu_prop[df_eicu_prop$age>=65,]
df_sepsis0 <-df_eicu_prop[df_eicu_prop$age<65,]

### eicu MAP

In [525]:
# eICU MAP
df_sepsis1 <-df_eicu_prop[df_eicu_prop$age>=65,]
df_sepsis0 <-df_eicu_prop[df_eicu_prop$age<65,]
# hospital mortality
gamfitMed_eICU_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_eICU_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_eICU_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/Age_eICU_MAP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(55, 95)
# hospital mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_hosp, main = "Age >=65y", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_hosp, main = "Age >=65y", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_hosp, main = "Age >=65y", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_hosp, main = "Age <65y", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_hosp, main = "Age <65y", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_hosp, main = "Age <65y", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# icu mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_icu, main = "Age >=65y", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_icu, main = "Age >=65y", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_icu, main = "Age >=65y", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_icu, main = "Age <65y", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_icu, main = "Age <65y", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_icu, main = "Age <65y", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_24h_aki, main = "Age >=65y", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_48h_aki, main = "Age >=65y", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_72h_aki, main = "Age >=65y", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_48h_aki, main = "Age <65y", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_24h_aki, main = "Age <65y", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_72h_aki, main = "Age <65y", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

dev.off()

png 
  2

### eicu SBP

In [526]:
# eICU SBP
df_sepsis1 <-df_eicu_prop[df_eicu_prop$age>=65,]
df_sepsis0 <-df_eicu_prop[df_eicu_prop$age<65,]
# hospital mortality
gamfitMed_eICU_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_eICU_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_eICU_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/Age_eICU_SBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(90, 140)
# hospital mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_hosp, main = "Age >=65y", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_hosp, main = "Age >=65y", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_hosp, main = "Age >=65y", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_hosp, main = "Age <65y", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_hosp, main = "Age <65y", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_hosp, main = "Age <65y", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# icu mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_icu, main = "Age >=65y", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_icu, main = "Age >=65y", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_icu, main = "Age >=65y", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_icu, main = "Age <65y", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_icu, main = "Age <65y", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_icu, main = "Age <65y", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_24h_aki, main = "Age >=65y", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_48h_aki, main = "Age >=65y", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_72h_aki, main = "Age >=65y", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_48h_aki, main = "Age <65y", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_24h_aki, main = "Age <65y", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_72h_aki, main = "Age <65y", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

dev.off()

png 
  2

### eicu DBP

In [527]:
# eICU DBP
df_sepsis1 <-df_eicu_prop[df_eicu_prop$age>=65,]
df_sepsis0 <-df_eicu_prop[df_eicu_prop$age<65,]
# hospital mortality
gamfitMed_eICU_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_eICU_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_eICU_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/Age_eICU_DBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(35, 75)
# hospital mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_hosp, main = "Age >=65y", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_hosp, main = "Age >=65y", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_hosp, main = "Age >=65y", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_hosp, main = "Age <65y", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_hosp, main = "Age <65y", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_hosp, main = "Age <65y", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# icu mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_icu, main = "Age >=65y", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_icu, main = "Age >=65y", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_icu, main = "Age >=65y", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_icu, main = "Age <65y", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_icu, main = "Age <65y", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_icu, main = "Age <65y", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_24h_aki, main = "Age >=65y", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_48h_aki, main = "Age >=65y", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_72h_aki, main = "Age >=65y", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_48h_aki, main = "Age <65y", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_24h_aki, main = "Age <65y", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_72h_aki, main = "Age <65y", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

dev.off()

png 
  2

### MIMIC MAP

In [528]:
df_sepsis1 <-df_mimic_prop[df_mimic_prop$age>=65,]
df_sepsis0 <-df_mimic_prop[df_mimic_prop$age<65,]

# hospital mortality
gamfitMed_MIMIC_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_MIMIC_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_MIMIC_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/Age_MIMIC_MAP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(55, 95)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_hosp, main = "Age >=65y", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_hosp, main = "Age >=65y", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_hosp, main = "Age >=65y", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_hosp, main = "Age <65y", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_hosp, main = "Age <65y", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_hosp, main = "Age <65y", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# icu mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_icu, main = "Age >=65y", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_icu, main = "Age >=65y", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_icu, main = "Age >=65y", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_icu, main = "Age <65y", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_icu, main = "Age <65y", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_icu, main = "Age <65y", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_24h_aki, main = "Age >=65y", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_48h_aki, main = "Age >=65y", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_72h_aki, main = "Age >=65y", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_48h_aki, main = "Age <65y", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_24h_aki, main = "Age <65y", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_72h_aki, main = "Age <65y", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

dev.off()

png 
  2

### MIMIC SBP

In [529]:
df_sepsis1 <-df_mimic_prop[df_mimic_prop$age>=65,]
df_sepsis0 <-df_mimic_prop[df_mimic_prop$age<65,]
# hospital mortality
gamfitMed_MIMIC_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_MIMIC_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_MIMIC_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/Age_MIMIC_SBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(90, 140)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_hosp, main = "Age >=65y", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_hosp, main = "Age >=65y", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_hosp, main = "Age >=65y", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_hosp, main = "Age <65y", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_hosp, main = "Age <65y", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_hosp, main = "Age <65y", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# icu mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_icu, main = "Age >=65y", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_icu, main = "Age >=65y", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_icu, main = "Age >=65y", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_icu, main = "Age <65y", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_icu, main = "Age <65y", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_icu, main = "Age <65y", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_24h_aki, main = "Age >=65y", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_48h_aki, main = "Age >=65y", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_72h_aki, main = "Age >=65y", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_48h_aki, main = "Age <65y", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_24h_aki, main = "Age <65y", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_72h_aki, main = "Age <65y", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

dev.off()

png 
  2

### MIMIC DBP

In [530]:
# MIMIC DBP
df_sepsis1 <-df_mimic_prop[df_mimic_prop$age>=65,]
df_sepsis0 <-df_mimic_prop[df_mimic_prop$age<65,]
# hospital mortality
gamfitMed_MIMIC_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_MIMIC_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_MIMIC_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/Age_MIMIC_DBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(35, 75)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_hosp, main = "Age >=65y", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_hosp, main = "Age >=65y", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_hosp, main = "Age >=65y", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_hosp, main = "Age <65y", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_hosp, main = "Age <65y", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_hosp, main = "Age <65y", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# icu mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_icu, main = "Age >=65y", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_icu, main = "Age >=65y", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_icu, main = "Age >=65y", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_icu, main = "Age <65y", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_icu, main = "Age <65y", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_icu, main = "Age <65y", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_24h_aki, main = "Age >=65y", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_48h_aki, main = "Age >=65y", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_72h_aki, main = "Age >=65y", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_48h_aki, main = "Age <65y", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_24h_aki, main = "Age <65y", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_72h_aki, main = "Age <65y", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

dev.off()

png 
  2

## Hypertension vs. non-hypertension

In [9]:
# eICU MAP
df_sepsis1 <-df_eicu_prop[df_eicu_prop$hypertension==1,]
df_sepsis0 <-df_eicu_prop[df_eicu_prop$hypertension==0,]

# hospital mortality
gamfitMed_eICU_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_eICU_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_eICU_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/Hypertension_eICU_MAP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(55, 95)
# hospital mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_hosp, main = "Hypertension = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_hosp, main = "Hypertension = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_hosp, main = "Hypertension = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_hosp, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_hosp, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_hosp, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# icu mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_icu, main = "Hypertension = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_icu, main = "Hypertension = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_icu, main = "Hypertension = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_icu, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_icu, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_icu, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_24h_aki, main = "Hypertension = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_48h_aki, main = "Hypertension = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_72h_aki, main = "Hypertension = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_48h_aki, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_24h_aki, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_72h_aki, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

dev.off()

png 
  2

In [10]:
# eICU SBP
df_sepsis1 <-df_eicu_prop[df_eicu_prop$hypertension==1,]
df_sepsis0 <-df_eicu_prop[df_eicu_prop$hypertension==0,]
# hospital mortality
gamfitMed_eICU_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_eICU_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_eICU_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/Hypertension_eICU_SBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(90, 140)
# hospital mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_hosp, main = "Hypertension = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_hosp, main = "Hypertension = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_hosp, main = "Hypertension = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_hosp, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_hosp, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_hosp, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# icu mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_icu, main = "Hypertension = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_icu, main = "Hypertension = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_icu, main = "Hypertension = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_icu, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_icu, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_icu, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_24h_aki, main = "Hypertension = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_48h_aki, main = "Hypertension = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_72h_aki, main = "Hypertension = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_48h_aki, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_24h_aki, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_72h_aki, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

dev.off()

png 
  2

In [11]:
# eICU DBP
df_sepsis1 <-df_eicu_prop[df_eicu_prop$hypertension==1,]
df_sepsis0 <-df_eicu_prop[df_eicu_prop$hypertension==0,]
# hospital mortality
gamfitMed_eICU_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_eICU_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_eICU_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/Hypertension_eICU_DBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(35, 75)
# hospital mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_hosp, main = "Hypertension = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_hosp, main = "Hypertension = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_hosp, main = "Hypertension = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_hosp, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_hosp, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_hosp, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# icu mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_icu, main = "Hypertension = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_icu, main = "Hypertension = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_icu, main = "Hypertension = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_icu, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_icu, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_icu, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_24h_aki, main = "Hypertension = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_48h_aki, main = "Hypertension = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_72h_aki, main = "Hypertension = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_48h_aki, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_24h_aki, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_72h_aki, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

dev.off()

png 
  2

In [162]:
# MIMIC MAP
df_sepsis1 <-df_mimic[df_mimic$hypertension==1,]
df_sepsis0 <-df_mimic[df_mimic$hypertension==0,]

# hospital mortality
gamfitMed_MIMIC_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_MIMIC_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_MIMIC_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/Hypertension_MIMIC_MAP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(55, 95)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_hosp, main = "Hypertension = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_hosp, main = "Hypertension = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_hosp, main = "Hypertension = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_hosp, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_hosp, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_hosp, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# icu mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_icu, main = "Hypertension = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_icu, main = "Hypertension = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_icu, main = "Hypertension = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_icu, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_icu, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_icu, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_24h_aki, main = "Hypertension = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_48h_aki, main = "Hypertension = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_72h_aki, main = "Hypertension = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_48h_aki, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_24h_aki, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_72h_aki, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

dev.off()


png 
  2

In [13]:
# MIMIC SBP
df_sepsis1 <-df_mimic[df_mimic$hypertension==1,]
df_sepsis0 <-df_mimic[df_mimic$hypertension==0,]
# MIMIC SBP
# hospital mortality
gamfitMed_MIMIC_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_MIMIC_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_MIMIC_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/Hypertension_MIMIC_SBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(90, 140)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_hosp, main = "Hypertension = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_hosp, main = "Hypertension = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_hosp, main = "Hypertension = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_hosp, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_hosp, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_hosp, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# icu mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_icu, main = "Hypertension = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_icu, main = "Hypertension = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_icu, main = "Hypertension = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_icu, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_icu, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_icu, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_24h_aki, main = "Hypertension = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_48h_aki, main = "Hypertension = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_72h_aki, main = "Hypertension = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_48h_aki, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_24h_aki, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_72h_aki, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

dev.off()

png 
  2

In [14]:
# MIMIC DBP
df_sepsis1 <-df_mimic[df_mimic$hypertension==1,]
df_sepsis0 <-df_mimic[df_mimic$hypertension==0,]

# hospital mortality
gamfitMed_MIMIC_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_MIMIC_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_MIMIC_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/Hypertension_MIMIC_DBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(35, 75)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_hosp, main = "Hypertension = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_hosp, main = "Hypertension = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_hosp, main = "Hypertension = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_hosp, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_hosp, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_hosp, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# icu mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_icu, main = "Hypertension = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_icu, main = "Hypertension = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_icu, main = "Hypertension = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_icu, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_icu, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_icu, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_24h_aki, main = "Hypertension = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_48h_aki, main = "Hypertension = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_72h_aki, main = "Hypertension = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_48h_aki, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_24h_aki, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_72h_aki, main = "Hypertension = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

dev.off()

png 
  2

## mechanical ventilation vs. no mechanical ventilation 

In [15]:
# eICU MAP
df_sepsis1 <-df_eicu_prop[df_eicu_prop$mechanical_ventilation==1,]
df_sepsis0 <-df_eicu_prop[df_eicu_prop$mechanical_ventilation==0,]

# hospital mortality
gamfitMed_eICU_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_eICU_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_eICU_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/MV_eICU_MAP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(55, 95)
# hospital mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_hosp, main = "MV = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_hosp, main = "MV = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_hosp, main = "MV = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_hosp, main = "MV = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_hosp, main = "MV = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_hosp, main = "MV = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# icu mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_icu, main = "MV = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_icu, main = "MV = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_icu, main = "MV = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_icu, main = "MV = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_icu, main = "MV = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_icu, main = "MV = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_24h_aki, main = "MV = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_48h_aki, main = "MV = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_72h_aki, main = "MV = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_48h_aki, main = "MV = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_24h_aki, main = "MV = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_72h_aki, main = "MV = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

dev.off()

png 
  2

In [16]:
# eICU SBP
df_sepsis1 <-df_eicu_prop[df_eicu_prop$mechanical_ventilation==1,]
df_sepsis0 <-df_eicu_prop[df_eicu_prop$mechanical_ventilation==0,]

# hospital mortality
gamfitMed_eICU_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_eICU_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_eICU_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/MV_eICU_SBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(90, 140)
# hospital mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_hosp, main = "MV = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_hosp, main = "MV = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_hosp, main = "MV = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_hosp, main = "MV = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_hosp, main = "MV = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_hosp, main = "MV = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# icu mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_icu, main = "MV = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_icu, main = "MV = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_icu, main = "MV = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_icu, main = "MV = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_icu, main = "MV = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_icu, main = "MV = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_24h_aki, main = "MV = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_48h_aki, main = "MV = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_72h_aki, main = "MV = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_48h_aki, main = "MV = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_24h_aki, main = "MV = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_72h_aki, main = "MV = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

dev.off()

png 
  2

In [17]:
# eICU DBP
df_sepsis1 <-df_eicu_prop[df_eicu_prop$mechanical_ventilation==1,]
df_sepsis0 <-df_eicu_prop[df_eicu_prop$mechanical_ventilation==0,]

# hospital mortality
gamfitMed_eICU_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_eICU_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_eICU_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/MV_eICU_DBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(35, 75)
# hospital mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_hosp, main = "MV = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_hosp, main = "MV = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_hosp, main = "MV = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_hosp, main = "MV = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_hosp, main = "MV = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_hosp, main = "MV = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# icu mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_icu, main = "MV = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_icu, main = "MV = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_icu, main = "MV = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_icu, main = "MV = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_icu, main = "MV = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_icu, main = "MV = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_24h_aki, main = "MV = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_48h_aki, main = "MV = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_72h_aki, main = "MV = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_48h_aki, main = "MV = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_24h_aki, main = "MV = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_72h_aki, main = "MV = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

dev.off()

png 
  2

In [163]:
# MIMIC MAP
df_sepsis1 <-df_mimic_prop[df_mimic_prop$mechanical_ventilation==1,]
df_sepsis0 <-df_mimic_prop[df_mimic_prop$mechanical_ventilation==0,]
# MIMIC MAP

# hospital mortality
gamfitMed_MIMIC_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_MIMIC_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_MIMIC_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/MV_MIMIC_MAP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(55, 95)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_hosp, main = "MV = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_hosp, main = "MV = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_hosp, main = "MV = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_hosp, main = "MV = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_hosp, main = "MV = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_hosp, main = "MV = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# icu mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_icu, main = "MV = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_icu, main = "MV = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_icu, main = "MV = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_icu, main = "MV = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_icu, main = "MV = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_icu, main = "MV = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_24h_aki, main = "MV = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_48h_aki, main = "MV = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_72h_aki, main = "MV = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_48h_aki, main = "MV = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_24h_aki, main = "MV = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_72h_aki, main = "MV = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

dev.off()

png 
  2

In [19]:
# MIMIC SBP
df_sepsis1 <-df_mimic_prop[df_mimic_prop$mechanical_ventilation==1,]
df_sepsis0 <-df_mimic_prop[df_mimic_prop$mechanical_ventilation==0,]

# MIMIC SBP
# hospital mortality
gamfitMed_MIMIC_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_MIMIC_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_MIMIC_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/MV_MIMIC_SBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(90, 140)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_hosp, main = "MV = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_hosp, main = "MV = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_hosp, main = "MV = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_hosp, main = "MV = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_hosp, main = "MV = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_hosp, main = "MV = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# icu mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_icu, main = "MV = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_icu, main = "MV = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_icu, main = "MV = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_icu, main = "MV = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_icu, main = "MV = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_icu, main = "MV = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_24h_aki, main = "MV = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_48h_aki, main = "MV = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_72h_aki, main = "MV = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_48h_aki, main = "MV = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_24h_aki, main = "MV = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_72h_aki, main = "MV = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

dev.off()

png 
  2

In [20]:
# MIMIC DBP
df_sepsis1 <-df_mimic_prop[df_mimic_prop$mechanical_ventilation==1,]
df_sepsis0 <-df_mimic_prop[df_mimic_prop$mechanical_ventilation==0,]

# hospital mortality
gamfitMed_MIMIC_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_MIMIC_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_MIMIC_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/MV_MIMIC_DBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(35, 75)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_hosp, main = "MV = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_hosp, main = "MV = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_hosp, main = "MV = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_hosp, main = "MV = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_hosp, main = "MV = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_hosp, main = "MV = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# icu mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_icu, main = "MV = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_icu, main = "MV = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_icu, main = "MV = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_icu, main = "MV = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_icu, main = "MV = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_icu, main = "MV = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_24h_aki, main = "MV = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_48h_aki, main = "MV = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_72h_aki, main = "MV = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_48h_aki, main = "MV = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_24h_aki, main = "MV = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_72h_aki, main = "MV = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

dev.off()

png 
  2

## BMI >= 30 vs. BMI < 30 

In [21]:
# eICU MAP
df_sepsis1 <-df_eicu_prop[df_eicu_prop$bmi>=30,]
df_sepsis0 <-df_eicu_prop[df_eicu_prop$bmi<30,]

# hospital mortality
gamfitMed_eICU_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_eICU_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_eICU_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/BMI_eICU_MAP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(55, 95)
# hospital mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_hosp, main = "BMI >=30", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_hosp, main = "BMI >=30", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_hosp, main = "BMI >=30", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_hosp, main = "BMI <30", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_hosp, main = "BMI <30", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_hosp, main = "BMI <30", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# icu mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_icu, main = "BMI >=30", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_icu, main = "BMI >=30", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_icu, main = "BMI >=30", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_icu, main = "BMI <30", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_icu, main = "BMI <30", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_icu, main = "BMI <30", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_24h_aki, main = "BMI >=30", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_48h_aki, main = "BMI >=30", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_72h_aki, main = "BMI >=30", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_48h_aki, main = "BMI <30", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_24h_aki, main = "BMI <30", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_72h_aki, main = "BMI <30", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

dev.off()

png 
  2

In [22]:
# eICU SBP
df_sepsis1 <-df_eicu_prop[df_eicu_prop$bmi>=30,]
df_sepsis0 <-df_eicu_prop[df_eicu_prop$bmi<30,]

# hospital mortality
gamfitMed_eICU_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_eICU_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_eICU_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/BMI_eICU_SBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(90, 140)
# hospital mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_hosp, main = "BMI >=30", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_hosp, main = "BMI >=30", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_hosp, main = "BMI >=30", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_hosp, main = "BMI <30", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_hosp, main = "BMI <30", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_hosp, main = "BMI <30", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# icu mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_icu, main = "BMI >=30", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_icu, main = "BMI >=30", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_icu, main = "BMI >=30", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_icu, main = "BMI <30", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_icu, main = "BMI <30", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_icu, main = "BMI <30", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_24h_aki, main = "BMI >=30", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_48h_aki, main = "BMI >=30", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_72h_aki, main = "BMI >=30", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_48h_aki, main = "BMI <30", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_24h_aki, main = "BMI <30", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_72h_aki, main = "BMI <30", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

dev.off()

png 
  2

In [23]:
# eICU DBP
df_sepsis1 <-df_eicu_prop[df_eicu_prop$bmi>=30,]
df_sepsis0 <-df_eicu_prop[df_eicu_prop$bmi<30,]

# hospital mortality
gamfitMed_eICU_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_eICU_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_eICU_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/BMI_eICU_DBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(35, 75)
# hospital mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_hosp, main = "BMI >=30", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_hosp, main = "BMI >=30", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_hosp, main = "BMI >=30", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_hosp, main = "BMI <30", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_hosp, main = "BMI <30", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_hosp, main = "BMI <30", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# icu mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_icu, main = "BMI >=30", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_icu, main = "BMI >=30", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_icu, main = "BMI >=30", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_icu, main = "BMI <30", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_icu, main = "BMI <30", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_icu, main = "BMI <30", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_24h_aki, main = "BMI >=30", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_48h_aki, main = "BMI >=30", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_72h_aki, main = "BMI >=30", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_48h_aki, main = "BMI <30", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_24h_aki, main = "BMI <30", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_72h_aki, main = "BMI <30", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

dev.off()

png 
  2

In [28]:
# MIMIC MAP
df_sepsis1 <-df_mimic_prop[df_mimic_prop$bmi>=30,]
df_sepsis0 <-df_mimic_prop[df_mimic_prop$bmi<30,]
# MIMIC MAP
# hospital mortality
gamfitMed_MIMIC_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_MIMIC_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_MIMIC_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/BMI_MIMIC_MAP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(55, 95)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_hosp, main = "BMI >=30", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_hosp, main = "BMI >=30", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_hosp, main = "BMI >=30", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_hosp, main = "BMI <30", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_hosp, main = "BMI <30", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_hosp, main = "BMI <30", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# icu mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_icu, main = "BMI >=30", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_icu, main = "BMI >=30", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_icu, main = "BMI >=30", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_icu, main = "BMI <30", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_icu, main = "BMI <30", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_icu, main = "BMI <30", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_24h_aki, main = "BMI >=30", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_48h_aki, main = "BMI >=30", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_72h_aki, main = "BMI >=30", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_48h_aki, main = "BMI <30", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_24h_aki, main = "BMI <30", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_72h_aki, main = "BMI <30", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

dev.off()

png 
  2

In [25]:
# MIMIC SBP
df_sepsis1 <-df_mimic_prop[df_mimic_prop$bmi>=30,]
df_sepsis0 <-df_mimic_prop[df_mimic_prop$bmi<30,]

# MIMIC SBP
# hospital mortality
gamfitMed_MIMIC_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_MIMIC_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_MIMIC_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/BMI_MIMIC_SBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(90, 140)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_hosp, main = "BMI >=30", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_hosp, main = "BMI >=30", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_hosp, main = "BMI >=30", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_hosp, main = "BMI <30", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_hosp, main = "BMI <30", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_hosp, main = "BMI <30", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# icu mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_icu, main = "BMI >=30", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_icu, main = "BMI >=30", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_icu, main = "BMI >=30", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_icu, main = "BMI <30", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_icu, main = "BMI <30", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_icu, main = "BMI <30", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_24h_aki, main = "BMI >=30", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_48h_aki, main = "BMI >=30", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_72h_aki, main = "BMI >=30", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_48h_aki, main = "BMI <30", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_24h_aki, main = "BMI <30", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_72h_aki, main = "BMI <30", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

dev.off()

png 
  2

In [26]:
# MIMIC DBP
df_sepsis1 <-df_mimic_prop[df_mimic_prop$bmi>=30,]
df_sepsis0 <-df_mimic_prop[df_mimic_prop$bmi<30,]

# hospital mortality
gamfitMed_MIMIC_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_MIMIC_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_MIMIC_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/BMI_MIMIC_DBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(35, 75)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_hosp, main = "BMI >=30", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_hosp, main = "BMI >=30", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_hosp, main = "BMI >=30", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_hosp, main = "BMI <30", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_hosp, main = "BMI <30", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_hosp, main = "BMI <30", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# icu mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_icu, main = "BMI >=30", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_icu, main = "BMI >=30", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_icu, main = "BMI >=30", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_icu, main = "BMI <30", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_icu, main = "BMI <30", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_icu, main = "BMI <30", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_24h_aki, main = "BMI >=30", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_48h_aki, main = "BMI >=30", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_72h_aki, main = "BMI >=30", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_48h_aki, main = "BMI <30", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_24h_aki, main = "BMI <30", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_72h_aki, main = "BMI <30", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

dev.off()

png 
  2

## ethnicity

In [29]:
df_white <-df_mimic[df_mimic$ethnicity=='WHITE',]
df_hispanic <-df_mimic[df_mimic$ethnicity=='HISPANIC',]
df_black<-df_mimic[df_mimic$ethnicity=='BLACK',]
df_asian <-df_mimic[df_mimic$ethnicity=='ASIAN',]
df_others<-df_mimic[df_mimic$ethnicity=='OTHERS',]


gamfitMed_MIMIC_white <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_MIMIC_hispanic <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_MIMIC_black <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_MIMIC_asian <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_MIMIC_others <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_MIMIC_overall <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+(age)+(bmi)+(sofatotal), data = df_mimic, family = binomial)

gamfitMed_MIMIC_white_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_MIMIC_hispanic_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_MIMIC_black_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_MIMIC_asian_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_MIMIC_others_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_MIMIC_overall_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+(age)+(bmi)+(sofatotal), data = df_mimic, family = binomial)

gamfitMed_MIMIC_white_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_MIMIC_hispanic_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_MIMIC_black_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_MIMIC_asian_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_MIMIC_others_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_MIMIC_overall_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+(age)+(bmi)+(sofatotal), data = df_mimic, family = binomial)



png 
  2

In [30]:
pdf(file = "figure/subgroup/Ethnicity_MIMIC_MAP24_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(55, 95)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_white, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_white_icu, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0, .50), label = "B1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_white_aki, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0.7,1.0),label = "C1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_hispanic, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0, .50),label = "A2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_hispanic_icu, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0, .50), label = "B2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_hispanic_aki, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")


gam_plotMed(gamfitMed_MIMIC_black, main = "eICU African American", xRange = x_range,
                    yRange = c(0, 0.5), label = "A3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_black_icu, main = "eICU African American", xRange = x_range,
                    yRange = c(0, 0.5), label = "B3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_black_aki, main = "eICU African American", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_asian, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0, .50),label = "A4", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_asian_icu, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0, .50), label = "B4", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_asian_aki, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C4", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")


gam_plotMed(gamfitMed_MIMIC_others, main = "eICU Others", xRange = x_range,
                    yRange = c(0,0.5), label = "A5", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_others_icu, main = "eICU Others", xRange = x_range,
                    yRange = c(0, 0.5), label = "B5", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_others_aki, main = "eICU Others", xRange = x_range,
                    yRange = c(0.7, 1.0),label = "C5", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_overall, main = "eICU Overall", xRange = x_range,
                    yRange = c(0,0.5),label = "A6", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_overall_icu, main = "eICU Overall", xRange = x_range,
                    yRange = c(0,0.5), label = "B6", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_overall_aki, main = "eICU Overall", xRange = x_range,
                    yRange = c(0.7,1.0),label = "C6", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

dev.off()


png 
  2

### ethnicity 48h MIMIC

In [34]:
df_white <-df_mimic[df_mimic$ethnicity=='WHITE',]
df_hispanic <-df_mimic[df_mimic$ethnicity=='HISPANIC',]
df_black<-df_mimic[df_mimic$ethnicity=='BLACK',]
df_asian <-df_mimic[df_mimic$ethnicity=='ASIAN',]
df_others<-df_mimic[df_mimic$ethnicity=='OTHERS',]


gamfitMed_MIMIC_white <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_MIMIC_hispanic <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_MIMIC_black <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_MIMIC_asian <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_MIMIC_others <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_MIMIC_overall <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+(age)+(bmi)+(sofatotal), data = df_mimic, family = binomial)

gamfitMed_MIMIC_white_icu <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_MIMIC_hispanic_icu <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_MIMIC_black_icu <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_MIMIC_asian_icu <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_MIMIC_others_icu <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_MIMIC_overall_icu <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+(age)+(bmi)+(sofatotal), data = df_mimic, family = binomial)

gamfitMed_MIMIC_white_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_MIMIC_hispanic_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_MIMIC_black_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_MIMIC_asian_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_MIMIC_others_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_MIMIC_overall_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+(age)+(bmi)+(sofatotal), data = df_mimic, family = binomial)

pdf(file = "figure/subgroup/Ethnicity_MIMIC_MAP48_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(55, 95)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_white, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_white_icu, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0, .50), label = "B1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_white_aki, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0.7,1.0),label = "C1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_hispanic, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0, .50),label = "A2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_hispanic_icu, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0, .50), label = "B2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_hispanic_aki, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")


gam_plotMed(gamfitMed_MIMIC_black, main = "eICU African American", xRange = x_range,
                    yRange = c(0, 0.5), label = "A3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_black_icu, main = "eICU African American", xRange = x_range,
                    yRange = c(0, 0.5), label = "B3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_black_aki, main = "eICU African American", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_asian, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0, .50),label = "A4", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_asian_icu, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0, .50), label = "B4", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_asian_aki, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C4", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")


gam_plotMed(gamfitMed_MIMIC_others, main = "eICU Others", xRange = x_range,
                    yRange = c(0,0.5), label = "A5", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_others_icu, main = "eICU Others", xRange = x_range,
                    yRange = c(0, 0.5), label = "B5", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_others_aki, main = "eICU Others", xRange = x_range,
                    yRange = c(0.7, 1.0),label = "C5", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_overall, main = "eICU Overall", xRange = x_range,
                    yRange = c(0,0.5),label = "A6", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_overall_icu, main = "eICU Overall", xRange = x_range,
                    yRange = c(0,0.5), label = "B6", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_overall_aki, main = "eICU Overall", xRange = x_range,
                    yRange = c(0.7,1.0),label = "C6", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

dev.off()



Warning message in min(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”

png 
  2

### Ethnicity 72h MIMIC

In [33]:
df_white <-df_mimic[df_mimic$ethnicity=='WHITE',]
df_hispanic <-df_mimic[df_mimic$ethnicity=='HISPANIC',]
df_black<-df_mimic[df_mimic$ethnicity=='BLACK',]
df_asian <-df_mimic[df_mimic$ethnicity=='ASIAN',]
df_others<-df_mimic[df_mimic$ethnicity=='OTHERS',]


gamfitMed_MIMIC_white <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_MIMIC_hispanic <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_MIMIC_black <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_MIMIC_asian <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_MIMIC_others <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_MIMIC_overall <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+(age)+(bmi)+(sofatotal), data = df_mimic, family = binomial)

gamfitMed_MIMIC_white_icu <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_MIMIC_hispanic_icu <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_MIMIC_black_icu <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_MIMIC_asian_icu <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_MIMIC_others_icu <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_MIMIC_overall_icu <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+(age)+(bmi)+(sofatotal), data = df_mimic, family = binomial)

gamfitMed_MIMIC_white_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_MIMIC_hispanic_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_MIMIC_black_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_MIMIC_asian_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_MIMIC_others_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_MIMIC_overall_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+(age)+(bmi)+(sofatotal), data = df_mimic, family = binomial)

pdf(file = "figure/subgroup/Ethnicity_MIMIC_MAP72_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(55, 95)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_white, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_white_icu, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0, .50), label = "B1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_white_aki, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0.7,1.0),label = "C1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_hispanic, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0, .50),label = "A2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_hispanic_icu, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0, .50), label = "B2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_hispanic_aki, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")


gam_plotMed(gamfitMed_MIMIC_black, main = "eICU African American", xRange = x_range,
                    yRange = c(0, 0.5), label = "A3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_black_icu, main = "eICU African American", xRange = x_range,
                    yRange = c(0, 0.5), label = "B3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_black_aki, main = "eICU African American", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_asian, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0, .50),label = "A4", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_asian_icu, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0, .50), label = "B4", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_asian_aki, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C4", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")


gam_plotMed(gamfitMed_MIMIC_others, main = "eICU Others", xRange = x_range,
                    yRange = c(0,0.5), label = "A5", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_others_icu, main = "eICU Others", xRange = x_range,
                    yRange = c(0, 0.5), label = "B5", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_others_aki, main = "eICU Others", xRange = x_range,
                    yRange = c(0.7, 1.0),label = "C5", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_overall, main = "eICU Overall", xRange = x_range,
                    yRange = c(0,0.5),label = "A6", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_overall_icu, main = "eICU Overall", xRange = x_range,
                    yRange = c(0,0.5), label = "B6", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_overall_aki, main = "eICU Overall", xRange = x_range,
                    yRange = c(0.7,1.0),label = "C6", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

dev.off()



Warning message in min(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”

png 
  2

### 24h SBP MIMIC

In [36]:
df_white <-df_mimic[df_mimic$ethnicity=='WHITE',]
df_hispanic <-df_mimic[df_mimic$ethnicity=='HISPANIC',]
df_black<-df_mimic[df_mimic$ethnicity=='BLACK',]
df_asian <-df_mimic[df_mimic$ethnicity=='ASIAN',]
df_others<-df_mimic[df_mimic$ethnicity=='OTHERS',]


gamfitMed_MIMIC_white <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_MIMIC_hispanic <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_MIMIC_black <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_MIMIC_asian <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_MIMIC_others <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_MIMIC_overall <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_mimic, family = binomial)

gamfitMed_MIMIC_white_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_MIMIC_hispanic_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_MIMIC_black_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_MIMIC_asian_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_MIMIC_others_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_MIMIC_overall_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_mimic, family = binomial)

gamfitMed_MIMIC_white_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_MIMIC_hispanic_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_MIMIC_black_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_MIMIC_asian_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_MIMIC_others_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_MIMIC_overall_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_mimic, family = binomial)

pdf(file = "figure/subgroup/Ethnicity_MIMIC_SBP24_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(90, 140)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_white, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_white_icu, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0, .50), label = "B1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_white_aki, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0.7,1.0),label = "C1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_hispanic, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0, .50),label = "A2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_hispanic_icu, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0, .50), label = "B2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_hispanic_aki, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")


gam_plotMed(gamfitMed_MIMIC_black, main = "eICU African American", xRange = x_range,
                    yRange = c(0, 0.5), label = "A3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_black_icu, main = "eICU African American", xRange = x_range,
                    yRange = c(0, 0.5), label = "B3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_black_aki, main = "eICU African American", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_asian, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0, .50),label = "A4", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_asian_icu, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0, .50), label = "B4", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_asian_aki, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C4", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")


gam_plotMed(gamfitMed_MIMIC_others, main = "eICU Others", xRange = x_range,
                    yRange = c(0,0.5), label = "A5", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_others_icu, main = "eICU Others", xRange = x_range,
                    yRange = c(0, 0.5), label = "B5", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_others_aki, main = "eICU Others", xRange = x_range,
                    yRange = c(0.7, 1.0),label = "C5", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_overall, main = "eICU Overall", xRange = x_range,
                    yRange = c(0,0.5),label = "A6", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_overall_icu, main = "eICU Overall", xRange = x_range,
                    yRange = c(0,0.5), label = "B6", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_overall_aki, main = "eICU Overall", xRange = x_range,
                    yRange = c(0.7,1.0),label = "C6", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

dev.off()



png 
  2

In [38]:
df_white <-df_mimic[df_mimic$ethnicity=='WHITE',]
df_hispanic <-df_mimic[df_mimic$ethnicity=='HISPANIC',]
df_black<-df_mimic[df_mimic$ethnicity=='BLACK',]
df_asian <-df_mimic[df_mimic$ethnicity=='ASIAN',]
df_others<-df_mimic[df_mimic$ethnicity=='OTHERS',]


gamfitMed_MIMIC_white <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_MIMIC_hispanic <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_MIMIC_black <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_MIMIC_asian <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_MIMIC_others <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_MIMIC_overall <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_mimic, family = binomial)

gamfitMed_MIMIC_white_icu <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_MIMIC_hispanic_icu <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_MIMIC_black_icu <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_MIMIC_asian_icu <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_MIMIC_others_icu <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_MIMIC_overall_icu <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_mimic, family = binomial)

gamfitMed_MIMIC_white_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_MIMIC_hispanic_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_MIMIC_black_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_MIMIC_asian_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_MIMIC_others_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_MIMIC_overall_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_mimic, family = binomial)

pdf(file = "figure/subgroup/Ethnicity_MIMIC_SBP48_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(90, 140)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_white, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_white_icu, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0, .50), label = "B1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_white_aki, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0.7,1.0),label = "C1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_hispanic, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0, .50),label = "A2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_hispanic_icu, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0, .50), label = "B2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_hispanic_aki, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")


gam_plotMed(gamfitMed_MIMIC_black, main = "eICU African American", xRange = x_range,
                    yRange = c(0, 0.5), label = "A3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_black_icu, main = "eICU African American", xRange = x_range,
                    yRange = c(0, 0.5), label = "B3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_black_aki, main = "eICU African American", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_asian, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0, .50),label = "A4", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_asian_icu, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0, .50), label = "B4", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_asian_aki, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C4", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")


gam_plotMed(gamfitMed_MIMIC_others, main = "eICU Others", xRange = x_range,
                    yRange = c(0,0.5), label = "A5", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_others_icu, main = "eICU Others", xRange = x_range,
                    yRange = c(0, 0.5), label = "B5", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_others_aki, main = "eICU Others", xRange = x_range,
                    yRange = c(0.7, 1.0),label = "C5", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_overall, main = "eICU Overall", xRange = x_range,
                    yRange = c(0,0.5),label = "A6", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_overall_icu, main = "eICU Overall", xRange = x_range,
                    yRange = c(0,0.5), label = "B6", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_overall_aki, main = "eICU Overall", xRange = x_range,
                    yRange = c(0.7,1.0),label = "C6", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

dev.off()



png 
  2

### SBP 72h MIMIC

In [37]:
df_white <-df_mimic[df_mimic$ethnicity=='WHITE',]
df_hispanic <-df_mimic[df_mimic$ethnicity=='HISPANIC',]
df_black<-df_mimic[df_mimic$ethnicity=='BLACK',]
df_asian <-df_mimic[df_mimic$ethnicity=='ASIAN',]
df_others<-df_mimic[df_mimic$ethnicity=='OTHERS',]


gamfitMed_MIMIC_white <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_MIMIC_hispanic <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_MIMIC_black <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_MIMIC_asian <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_MIMIC_others <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_MIMIC_overall <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_mimic, family = binomial)

gamfitMed_MIMIC_white_icu <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_MIMIC_hispanic_icu <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_MIMIC_black_icu <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_MIMIC_asian_icu <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_MIMIC_others_icu <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_MIMIC_overall_icu <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_mimic, family = binomial)

gamfitMed_MIMIC_white_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_MIMIC_hispanic_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_MIMIC_black_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_MIMIC_asian_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_MIMIC_others_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_MIMIC_overall_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_mimic, family = binomial)

pdf(file = "figure/subgroup/Ethnicity_MIMIC_SBP72_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(90, 140)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_white, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_white_icu, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0, .50), label = "B1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_white_aki, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0.7,1.0),label = "C1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_hispanic, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0, .50),label = "A2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_hispanic_icu, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0, .50), label = "B2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_hispanic_aki, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")


gam_plotMed(gamfitMed_MIMIC_black, main = "eICU African American", xRange = x_range,
                    yRange = c(0, 0.5), label = "A3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_black_icu, main = "eICU African American", xRange = x_range,
                    yRange = c(0, 0.5), label = "B3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_black_aki, main = "eICU African American", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_asian, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0, .50),label = "A4", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_asian_icu, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0, .50), label = "B4", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_asian_aki, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C4", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")


gam_plotMed(gamfitMed_MIMIC_others, main = "eICU Others", xRange = x_range,
                    yRange = c(0,0.5), label = "A5", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_others_icu, main = "eICU Others", xRange = x_range,
                    yRange = c(0, 0.5), label = "B5", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_others_aki, main = "eICU Others", xRange = x_range,
                    yRange = c(0.7, 1.0),label = "C5", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_overall, main = "eICU Overall", xRange = x_range,
                    yRange = c(0,0.5),label = "A6", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_overall_icu, main = "eICU Overall", xRange = x_range,
                    yRange = c(0,0.5), label = "B6", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_overall_aki, main = "eICU Overall", xRange = x_range,
                    yRange = c(0.7,1.0),label = "C6", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

dev.off()



png 
  2

### DBP 24h MIMIC

In [39]:
df_white <-df_mimic[df_mimic$ethnicity=='WHITE',]
df_hispanic <-df_mimic[df_mimic$ethnicity=='HISPANIC',]
df_black<-df_mimic[df_mimic$ethnicity=='BLACK',]
df_asian <-df_mimic[df_mimic$ethnicity=='ASIAN',]
df_others<-df_mimic[df_mimic$ethnicity=='OTHERS',]


gamfitMed_MIMIC_white <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_MIMIC_hispanic <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_MIMIC_black <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_MIMIC_asian <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_MIMIC_others <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_MIMIC_overall <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_mimic, family = binomial)

gamfitMed_MIMIC_white_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_MIMIC_hispanic_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_MIMIC_black_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_MIMIC_asian_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_MIMIC_others_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_MIMIC_overall_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_mimic, family = binomial)

gamfitMed_MIMIC_white_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_MIMIC_hispanic_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_MIMIC_black_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_MIMIC_asian_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_MIMIC_others_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_MIMIC_overall_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_mimic, family = binomial)

pdf(file = "figure/subgroup/Ethnicity_MIMIC_DBP24_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(35, 75)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_white, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_white_icu, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0, .50), label = "B1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_white_aki, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0.7,1.0),label = "C1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_hispanic, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0, .50),label = "A2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_hispanic_icu, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0, .50), label = "B2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_hispanic_aki, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")


gam_plotMed(gamfitMed_MIMIC_black, main = "eICU African American", xRange = x_range,
                    yRange = c(0, 0.5), label = "A3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_black_icu, main = "eICU African American", xRange = x_range,
                    yRange = c(0, 0.5), label = "B3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_black_aki, main = "eICU African American", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_asian, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0, .50),label = "A4", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_asian_icu, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0, .50), label = "B4", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_asian_aki, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C4", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")


gam_plotMed(gamfitMed_MIMIC_others, main = "eICU Others", xRange = x_range,
                    yRange = c(0,0.5), label = "A5", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_others_icu, main = "eICU Others", xRange = x_range,
                    yRange = c(0, 0.5), label = "B5", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_others_aki, main = "eICU Others", xRange = x_range,
                    yRange = c(0.7, 1.0),label = "C5", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_overall, main = "eICU Overall", xRange = x_range,
                    yRange = c(0,0.5),label = "A6", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_overall_icu, main = "eICU Overall", xRange = x_range,
                    yRange = c(0,0.5), label = "B6", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_overall_aki, main = "eICU Overall", xRange = x_range,
                    yRange = c(0.7,1.0),label = "C6", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

dev.off()



png 
  2

In [ ]:
### DBP 48h MIMIC

In [40]:
df_white <-df_mimic[df_mimic$ethnicity=='WHITE',]
df_hispanic <-df_mimic[df_mimic$ethnicity=='HISPANIC',]
df_black<-df_mimic[df_mimic$ethnicity=='BLACK',]
df_asian <-df_mimic[df_mimic$ethnicity=='ASIAN',]
df_others<-df_mimic[df_mimic$ethnicity=='OTHERS',]


gamfitMed_MIMIC_white <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_MIMIC_hispanic <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_MIMIC_black <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_MIMIC_asian <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_MIMIC_others <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_MIMIC_overall <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_mimic, family = binomial)

gamfitMed_MIMIC_white_icu <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_MIMIC_hispanic_icu <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_MIMIC_black_icu <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_MIMIC_asian_icu <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_MIMIC_others_icu <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_MIMIC_overall_icu <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_mimic, family = binomial)

gamfitMed_MIMIC_white_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_MIMIC_hispanic_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_MIMIC_black_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_MIMIC_asian_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_MIMIC_others_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_MIMIC_overall_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_mimic, family = binomial)

pdf(file = "figure/subgroup/Ethnicity_MIMIC_DBP48_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(35, 75)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_white, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_white_icu, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0, .50), label = "B1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_white_aki, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0.7,1.0),label = "C1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_hispanic, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0, .50),label = "A2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_hispanic_icu, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0, .50), label = "B2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_hispanic_aki, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")


gam_plotMed(gamfitMed_MIMIC_black, main = "eICU African American", xRange = x_range,
                    yRange = c(0, 0.5), label = "A3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_black_icu, main = "eICU African American", xRange = x_range,
                    yRange = c(0, 0.5), label = "B3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_black_aki, main = "eICU African American", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_asian, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0, .50),label = "A4", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_asian_icu, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0, .50), label = "B4", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_asian_aki, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C4", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")


gam_plotMed(gamfitMed_MIMIC_others, main = "eICU Others", xRange = x_range,
                    yRange = c(0,0.5), label = "A5", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_others_icu, main = "eICU Others", xRange = x_range,
                    yRange = c(0, 0.5), label = "B5", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_others_aki, main = "eICU Others", xRange = x_range,
                    yRange = c(0.7, 1.0),label = "C5", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_overall, main = "eICU Overall", xRange = x_range,
                    yRange = c(0,0.5),label = "A6", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_overall_icu, main = "eICU Overall", xRange = x_range,
                    yRange = c(0,0.5), label = "B6", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_overall_aki, main = "eICU Overall", xRange = x_range,
                    yRange = c(0.7,1.0),label = "C6", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

dev.off()



Warning message in min(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”

png 
  2

In [ ]:
### DBP 72h MIMIC

In [42]:
df_white <-df_mimic[df_mimic$ethnicity=='WHITE',]
df_hispanic <-df_mimic[df_mimic$ethnicity=='HISPANIC',]
df_black<-df_mimic[df_mimic$ethnicity=='BLACK',]
df_asian <-df_mimic[df_mimic$ethnicity=='ASIAN',]
df_others<-df_mimic[df_mimic$ethnicity=='OTHERS',]


gamfitMed_MIMIC_white <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_MIMIC_hispanic <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_MIMIC_black <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_MIMIC_asian <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_MIMIC_others <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_MIMIC_overall <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_mimic, family = binomial)

gamfitMed_MIMIC_white_icu <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_MIMIC_hispanic_icu <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_MIMIC_black_icu <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_MIMIC_asian_icu <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_MIMIC_others_icu <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_MIMIC_overall_icu <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_mimic, family = binomial)

gamfitMed_MIMIC_white_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_MIMIC_hispanic_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_MIMIC_black_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_MIMIC_asian_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_MIMIC_others_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_MIMIC_overall_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_mimic, family = binomial)

pdf(file = "figure/subgroup/Ethnicity_MIMIC_DBP72_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(35, 75)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_white, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_white_icu, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0, .50), label = "B1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_white_aki, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0.7,1.0),label = "C1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_hispanic, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0, .50),label = "A2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_hispanic_icu, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0, .50), label = "B2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_hispanic_aki, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")


gam_plotMed(gamfitMed_MIMIC_black, main = "eICU African American", xRange = x_range,
                    yRange = c(0, 0.5), label = "A3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_black_icu, main = "eICU African American", xRange = x_range,
                    yRange = c(0, 0.5), label = "B3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_black_aki, main = "eICU African American", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_asian, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0, .50),label = "A4", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_asian_icu, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0, .50), label = "B4", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_asian_aki, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C4", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")


gam_plotMed(gamfitMed_MIMIC_others, main = "eICU Others", xRange = x_range,
                    yRange = c(0,0.5), label = "A5", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_others_icu, main = "eICU Others", xRange = x_range,
                    yRange = c(0, 0.5), label = "B5", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_others_aki, main = "eICU Others", xRange = x_range,
                    yRange = c(0.7, 1.0),label = "C5", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_overall, main = "eICU Overall", xRange = x_range,
                    yRange = c(0,0.5),label = "A6", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_overall_icu, main = "eICU Overall", xRange = x_range,
                    yRange = c(0,0.5), label = "B6", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_overall_aki, main = "eICU Overall", xRange = x_range,
                    yRange = c(0.7,1.0),label = "C6", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

dev.off()



Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”

png 
  2

In [ ]:
### MAP 24h eICU

In [45]:
df_white <-df_eicu[df_eicu$ethnicity=='Caucasian',]
df_hispanic <-df_eicu[df_eicu$ethnicity=='Hispanic',]
df_black<-df_eicu[df_eicu$ethnicity=='African American',]
df_asian <-df_eicu[df_eicu$ethnicity=='Asian',]
df_others<-df_eicu[df_eicu$ethnicity=='Others' |df_eicu$ethnicity=='Native American', ]


gamfitMed_eICU_white <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_eICU_hispanic <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_eICU_black <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_eICU_asian <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_eICU_others <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_eICU_overall <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+(age)+(bmi)+(sofatotal), data = df_eicu, family = binomial)

gamfitMed_eICU_white_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_eICU_hispanic_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_eICU_black_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_eICU_asian_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_eICU_others_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_eICU_overall_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+(age)+(bmi)+(sofatotal), data = df_eicu, family = binomial)

gamfitMed_eICU_white_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_eICU_hispanic_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_eICU_black_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_eICU_asian_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_eICU_others_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_eICU_overall_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+(age)+(bmi)+(sofatotal), data = df_eicu, family = binomial)

pdf(file = "figure/subgroup/Ethnicity_eICU_MAP24_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(55, 95)
# hospital mortality
gam_plotMed(gamfitMed_eICU_white, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_white_icu, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0, .50), label = "B1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_white_aki, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0.7,1.0),label = "C1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_hispanic, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0, .50),label = "A2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_hispanic_icu, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0, .50), label = "B2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_hispanic_aki, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")


gam_plotMed(gamfitMed_eICU_black, main = "eICU African American", xRange = x_range,
                    yRange = c(0, 0.5), label = "A3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_black_icu, main = "eICU African American", xRange = x_range,
                    yRange = c(0, 0.5), label = "B3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_black_aki, main = "eICU African American", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_asian, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0, .50),label = "A4", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_asian_icu, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0, .50), label = "B4", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_asian_aki, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C4", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")


gam_plotMed(gamfitMed_eICU_others, main = "eICU Others", xRange = x_range,
                    yRange = c(0,0.5), label = "A5", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_others_icu, main = "eICU Others", xRange = x_range,
                    yRange = c(0, 0.5), label = "B5", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_others_aki, main = "eICU Others", xRange = x_range,
                    yRange = c(0.7, 1.0),label = "C5", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_overall, main = "eICU Overall", xRange = x_range,
                    yRange = c(0,0.5),label = "A6", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_overall_icu, main = "eICU Overall", xRange = x_range,
                    yRange = c(0,0.5), label = "B6", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_overall_aki, main = "eICU Overall", xRange = x_range,
                    yRange = c(0.7,1.0),label = "C6", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

dev.off()



Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”

png 
  2

In [ ]:
### MAP 48h eICU

In [46]:
df_white <-df_eicu[df_eicu$ethnicity=='Caucasian',]
df_hispanic <-df_eicu[df_eicu$ethnicity=='Hispanic',]
df_black<-df_eicu[df_eicu$ethnicity=='African American',]
df_asian <-df_eicu[df_eicu$ethnicity=='Asian',]
df_others<-df_eicu[df_eicu$ethnicity=='Others' |df_eicu$ethnicity=='Native American', ]


gamfitMed_eICU_white <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_eICU_hispanic <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_eICU_black <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_eICU_asian <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_eICU_others <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_eICU_overall <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+(age)+(bmi)+(sofatotal), data = df_eicu, family = binomial)

gamfitMed_eICU_white_icu <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_eICU_hispanic_icu <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_eICU_black_icu <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_eICU_asian_icu <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_eICU_others_icu <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_eICU_overall_icu <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+(age)+(bmi)+(sofatotal), data = df_eicu, family = binomial)

gamfitMed_eICU_white_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_eICU_hispanic_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_eICU_black_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_eICU_asian_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_eICU_others_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_eICU_overall_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+(age)+(bmi)+(sofatotal), data = df_eicu, family = binomial)

pdf(file = "figure/subgroup/Ethnicity_eICU_MAP48_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(55, 95)
# hospital mortality
gam_plotMed(gamfitMed_eICU_white, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_white_icu, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0, .50), label = "B1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_white_aki, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0.7,1.0),label = "C1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_hispanic, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0, .50),label = "A2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_hispanic_icu, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0, .50), label = "B2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_hispanic_aki, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")


gam_plotMed(gamfitMed_eICU_black, main = "eICU African American", xRange = x_range,
                    yRange = c(0, 0.5), label = "A3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_black_icu, main = "eICU African American", xRange = x_range,
                    yRange = c(0, 0.5), label = "B3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_black_aki, main = "eICU African American", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_asian, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0, .50),label = "A4", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_asian_icu, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0, .50), label = "B4", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_asian_aki, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C4", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")


gam_plotMed(gamfitMed_eICU_others, main = "eICU Others", xRange = x_range,
                    yRange = c(0,0.5), label = "A5", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_others_icu, main = "eICU Others", xRange = x_range,
                    yRange = c(0, 0.5), label = "B5", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_others_aki, main = "eICU Others", xRange = x_range,
                    yRange = c(0.7, 1.0),label = "C5", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_overall, main = "eICU Overall", xRange = x_range,
                    yRange = c(0,0.5),label = "A6", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_overall_icu, main = "eICU Overall", xRange = x_range,
                    yRange = c(0,0.5), label = "B6", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_overall_aki, main = "eICU Overall", xRange = x_range,
                    yRange = c(0.7,1.0),label = "C6", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

dev.off()



png 
  2

In [ ]:
### MAP 72h eICU

In [48]:
df_white <-df_eicu[df_eicu$ethnicity=='Caucasian',]
df_hispanic <-df_eicu[df_eicu$ethnicity=='Hispanic',]
df_black<-df_eicu[df_eicu$ethnicity=='African American',]
df_asian <-df_eicu[df_eicu$ethnicity=='Asian',]
df_others<-df_eicu[df_eicu$ethnicity=='Others' |df_eicu$ethnicity=='Native American', ]


gamfitMed_eICU_white <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_eICU_hispanic <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_eICU_black <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_eICU_asian <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_eICU_others <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_eICU_overall <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+(age)+(bmi)+(sofatotal), data = df_eicu, family = binomial)

gamfitMed_eICU_white_icu <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_eICU_hispanic_icu <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_eICU_black_icu <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_eICU_asian_icu <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_eICU_others_icu <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_eICU_overall_icu <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+(age)+(bmi)+(sofatotal), data = df_eicu, family = binomial)

gamfitMed_eICU_white_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_eICU_hispanic_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_eICU_black_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_eICU_asian_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_eICU_others_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_eICU_overall_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+(age)+(bmi)+(sofatotal), data = df_eicu, family = binomial)

pdf(file = "figure/subgroup/Ethnicity_eICU_MAP72_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(55, 95)
# hospital mortality
gam_plotMed(gamfitMed_eICU_white, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_white_icu, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0, .50), label = "B1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_white_aki, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0.7,1.0),label = "C1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_hispanic, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0, .50),label = "A2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_hispanic_icu, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0, .50), label = "B2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_hispanic_aki, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")


gam_plotMed(gamfitMed_eICU_black, main = "eICU African American", xRange = x_range,
                    yRange = c(0, 0.5), label = "A3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_black_icu, main = "eICU African American", xRange = x_range,
                    yRange = c(0, 0.5), label = "B3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_black_aki, main = "eICU African American", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_asian, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0, .50),label = "A4", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_asian_icu, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0, .50), label = "B4", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_asian_aki, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C4", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")


gam_plotMed(gamfitMed_eICU_others, main = "eICU Others", xRange = x_range,
                    yRange = c(0,0.5), label = "A5", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_others_icu, main = "eICU Others", xRange = x_range,
                    yRange = c(0, 0.5), label = "B5", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_others_aki, main = "eICU Others", xRange = x_range,
                    yRange = c(0.7, 1.0),label = "C5", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_overall, main = "eICU Overall", xRange = x_range,
                    yRange = c(0,0.5),label = "A6", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_overall_icu, main = "eICU Overall", xRange = x_range,
                    yRange = c(0,0.5), label = "B6", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_overall_aki, main = "eICU Overall", xRange = x_range,
                    yRange = c(0.7,1.0),label = "C6", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

dev.off()



png 
  2

In [ ]:
### SBP 24h eICU

In [50]:
df_white <-df_eicu[df_eicu$ethnicity=='Caucasian',]
df_hispanic <-df_eicu[df_eicu$ethnicity=='Hispanic',]
df_black<-df_eicu[df_eicu$ethnicity=='African American',]
df_asian <-df_eicu[df_eicu$ethnicity=='Asian',]
df_others<-df_eicu[df_eicu$ethnicity=='Others' |df_eicu$ethnicity=='Native American', ]


gamfitMed_eICU_white <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_eICU_hispanic <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_eICU_black <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_eICU_asian <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_eICU_others <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_eICU_overall <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_eicu, family = binomial)

gamfitMed_eICU_white_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_eICU_hispanic_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_eICU_black_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_eICU_asian_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_eICU_others_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_eICU_overall_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_eicu, family = binomial)

gamfitMed_eICU_white_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_eICU_hispanic_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_eICU_black_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_eICU_asian_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_eICU_others_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_eICU_overall_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_eicu, family = binomial)

pdf(file = "figure/subgroup/Ethnicity_eICU_SBP24_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(90, 140)
# hospital mortality
gam_plotMed(gamfitMed_eICU_white, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_white_icu, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0, .50), label = "B1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_white_aki, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0.7,1.0),label = "C1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_hispanic, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0, .50),label = "A2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_hispanic_icu, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0, .50), label = "B2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_hispanic_aki, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")


gam_plotMed(gamfitMed_eICU_black, main = "eICU African American", xRange = x_range,
                    yRange = c(0, 0.5), label = "A3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_black_icu, main = "eICU African American", xRange = x_range,
                    yRange = c(0, 0.5), label = "B3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_black_aki, main = "eICU African American", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_asian, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0, .50),label = "A4", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_asian_icu, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0, .50), label = "B4", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_asian_aki, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C4", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")


gam_plotMed(gamfitMed_eICU_others, main = "eICU Others", xRange = x_range,
                    yRange = c(0,0.5), label = "A5", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_others_icu, main = "eICU Others", xRange = x_range,
                    yRange = c(0, 0.5), label = "B5", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_others_aki, main = "eICU Others", xRange = x_range,
                    yRange = c(0.7, 1.0),label = "C5", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_overall, main = "eICU Overall", xRange = x_range,
                    yRange = c(0,0.5),label = "A6", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_overall_icu, main = "eICU Overall", xRange = x_range,
                    yRange = c(0,0.5), label = "B6", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_overall_aki, main = "eICU Overall", xRange = x_range,
                    yRange = c(0.7,1.0),label = "C6", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

dev.off()



png 
  2

In [ ]:
### SBP 48h eICU

In [51]:
df_white <-df_eicu[df_eicu$ethnicity=='Caucasian',]
df_hispanic <-df_eicu[df_eicu$ethnicity=='Hispanic',]
df_black<-df_eicu[df_eicu$ethnicity=='African American',]
df_asian <-df_eicu[df_eicu$ethnicity=='Asian',]
df_others<-df_eicu[df_eicu$ethnicity=='Others' |df_eicu$ethnicity=='Native American', ]


gamfitMed_eICU_white <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_eICU_hispanic <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_eICU_black <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_eICU_asian <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_eICU_others <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_eICU_overall <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_eicu, family = binomial)

gamfitMed_eICU_white_icu <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_eICU_hispanic_icu <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_eICU_black_icu <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_eICU_asian_icu <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_eICU_others_icu <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_eICU_overall_icu <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_eicu, family = binomial)

gamfitMed_eICU_white_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_eICU_hispanic_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_eICU_black_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_eICU_asian_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_eICU_others_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_eICU_overall_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_eicu, family = binomial)

pdf(file = "figure/subgroup/Ethnicity_eICU_SBP48_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(90, 140)
# hospital mortality
gam_plotMed(gamfitMed_eICU_white, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_white_icu, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0, .50), label = "B1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_white_aki, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0.7,1.0),label = "C1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_hispanic, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0, .50),label = "A2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_hispanic_icu, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0, .50), label = "B2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_hispanic_aki, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")


gam_plotMed(gamfitMed_eICU_black, main = "eICU African American", xRange = x_range,
                    yRange = c(0, 0.5), label = "A3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_black_icu, main = "eICU African American", xRange = x_range,
                    yRange = c(0, 0.5), label = "B3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_black_aki, main = "eICU African American", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_asian, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0, .50),label = "A4", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_asian_icu, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0, .50), label = "B4", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_asian_aki, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C4", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")


gam_plotMed(gamfitMed_eICU_others, main = "eICU Others", xRange = x_range,
                    yRange = c(0,0.5), label = "A5", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_others_icu, main = "eICU Others", xRange = x_range,
                    yRange = c(0, 0.5), label = "B5", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_others_aki, main = "eICU Others", xRange = x_range,
                    yRange = c(0.7, 1.0),label = "C5", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_overall, main = "eICU Overall", xRange = x_range,
                    yRange = c(0,0.5),label = "A6", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_overall_icu, main = "eICU Overall", xRange = x_range,
                    yRange = c(0,0.5), label = "B6", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_overall_aki, main = "eICU Overall", xRange = x_range,
                    yRange = c(0.7,1.0),label = "C6", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

dev.off()



Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”

png 
  2

In [ ]:
## SBP 72h eICU

In [52]:
df_white <-df_eicu[df_eicu$ethnicity=='Caucasian',]
df_hispanic <-df_eicu[df_eicu$ethnicity=='Hispanic',]
df_black<-df_eicu[df_eicu$ethnicity=='African American',]
df_asian <-df_eicu[df_eicu$ethnicity=='Asian',]
df_others<-df_eicu[df_eicu$ethnicity=='Others' |df_eicu$ethnicity=='Native American', ]


gamfitMed_eICU_white <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_eICU_hispanic <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_eICU_black <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_eICU_asian <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_eICU_others <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_eICU_overall <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_eicu, family = binomial)

gamfitMed_eICU_white_icu <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_eICU_hispanic_icu <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_eICU_black_icu <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_eICU_asian_icu <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_eICU_others_icu <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_eICU_overall_icu <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_eicu, family = binomial)

gamfitMed_eICU_white_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_eICU_hispanic_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_eICU_black_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_eICU_asian_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_eICU_others_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_eICU_overall_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_eicu, family = binomial)

pdf(file = "figure/subgroup/Ethnicity_eICU_SBP72_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(90, 140)
# hospital mortality
gam_plotMed(gamfitMed_eICU_white, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_white_icu, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0, .50), label = "B1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_white_aki, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0.7,1.0),label = "C1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_hispanic, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0, .50),label = "A2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_hispanic_icu, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0, .50), label = "B2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_hispanic_aki, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")


gam_plotMed(gamfitMed_eICU_black, main = "eICU African American", xRange = x_range,
                    yRange = c(0, 0.5), label = "A3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_black_icu, main = "eICU African American", xRange = x_range,
                    yRange = c(0, 0.5), label = "B3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_black_aki, main = "eICU African American", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_asian, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0, .50),label = "A4", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_asian_icu, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0, .50), label = "B4", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_asian_aki, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C4", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")


gam_plotMed(gamfitMed_eICU_others, main = "eICU Others", xRange = x_range,
                    yRange = c(0,0.5), label = "A5", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_others_icu, main = "eICU Others", xRange = x_range,
                    yRange = c(0, 0.5), label = "B5", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_others_aki, main = "eICU Others", xRange = x_range,
                    yRange = c(0.7, 1.0),label = "C5", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_overall, main = "eICU Overall", xRange = x_range,
                    yRange = c(0,0.5),label = "A6", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_overall_icu, main = "eICU Overall", xRange = x_range,
                    yRange = c(0,0.5), label = "B6", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_overall_aki, main = "eICU Overall", xRange = x_range,
                    yRange = c(0.7,1.0),label = "C6", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

dev.off()



Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”

png 
  2

In [ ]:
### DBP 24h eICU

In [53]:
df_white <-df_eicu[df_eicu$ethnicity=='Caucasian',]
df_hispanic <-df_eicu[df_eicu$ethnicity=='Hispanic',]
df_black<-df_eicu[df_eicu$ethnicity=='African American',]
df_asian <-df_eicu[df_eicu$ethnicity=='Asian',]
df_others<-df_eicu[df_eicu$ethnicity=='Others' |df_eicu$ethnicity=='Native American', ]


gamfitMed_eICU_white <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_eICU_hispanic <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_eICU_black <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_eICU_asian <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_eICU_others <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_eICU_overall <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_eicu, family = binomial)

gamfitMed_eICU_white_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_eICU_hispanic_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_eICU_black_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_eICU_asian_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_eICU_others_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_eICU_overall_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_eicu, family = binomial)

gamfitMed_eICU_white_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_eICU_hispanic_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_eICU_black_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_eICU_asian_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_eICU_others_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_eICU_overall_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+(age)+(bmi)+(sofatotal), data = df_eicu, family = binomial)

pdf(file = "figure/subgroup/Ethnicity_eICU_DBP24_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(35, 75)
# hospital mortality
gam_plotMed(gamfitMed_eICU_white, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_white_icu, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0, .50), label = "B1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_white_aki, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0.7,1.0),label = "C1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_hispanic, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0, .50),label = "A2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_hispanic_icu, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0, .50), label = "B2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_hispanic_aki, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")


gam_plotMed(gamfitMed_eICU_black, main = "eICU African American", xRange = x_range,
                    yRange = c(0, 0.5), label = "A3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_black_icu, main = "eICU African American", xRange = x_range,
                    yRange = c(0, 0.5), label = "B3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_black_aki, main = "eICU African American", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_asian, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0, .50),label = "A4", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_asian_icu, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0, .50), label = "B4", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_asian_aki, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C4", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")


gam_plotMed(gamfitMed_eICU_others, main = "eICU Others", xRange = x_range,
                    yRange = c(0,0.5), label = "A5", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_others_icu, main = "eICU Others", xRange = x_range,
                    yRange = c(0, 0.5), label = "B5", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_others_aki, main = "eICU Others", xRange = x_range,
                    yRange = c(0.7, 1.0),label = "C5", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_overall, main = "eICU Overall", xRange = x_range,
                    yRange = c(0,0.5),label = "A6", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_overall_icu, main = "eICU Overall", xRange = x_range,
                    yRange = c(0,0.5), label = "B6", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_overall_aki, main = "eICU Overall", xRange = x_range,
                    yRange = c(0.7,1.0),label = "C6", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

dev.off()



Warning message in min(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning me

png 
  2

In [ ]:
### DBP 48h eICU

In [57]:
df_white <-df_eicu[df_eicu$ethnicity=='Caucasian',]
df_hispanic <-df_eicu[df_eicu$ethnicity=='Hispanic',]
df_black<-df_eicu[df_eicu$ethnicity=='African American',]
df_asian <-df_eicu[df_eicu$ethnicity=='Asian',]
df_others<-df_eicu[df_eicu$ethnicity=='Others' |df_eicu$ethnicity=='Native American', ]


gamfitMed_eICU_white <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_eICU_hispanic <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_eICU_black <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_eICU_asian <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_eICU_others <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_eICU_overall <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_eicu, family = binomial)

gamfitMed_eICU_white_icu <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_eICU_hispanic_icu <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_eICU_black_icu <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_eICU_asian_icu <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_eICU_others_icu <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_eICU_overall_icu <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_eicu, family = binomial)

gamfitMed_eICU_white_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_eICU_hispanic_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_eICU_black_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_eICU_asian_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_eICU_others_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_eICU_overall_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+(age)+(bmi)+(sofatotal), data = df_eicu, family = binomial)

pdf(file = "figure/subgroup/Ethnicity_eICU_DBP48_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(35, 75)
# hospital mortality
gam_plotMed(gamfitMed_eICU_white, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_white_icu, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0, .50), label = "B1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_white_aki, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0.7,1.0),label = "C1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_hispanic, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0, .50),label = "A2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_hispanic_icu, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0, .50), label = "B2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_hispanic_aki, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")


gam_plotMed(gamfitMed_eICU_black, main = "eICU African American", xRange = x_range,
                    yRange = c(0, 0.5), label = "A3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_black_icu, main = "eICU African American", xRange = x_range,
                    yRange = c(0, 0.5), label = "B3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_black_aki, main = "eICU African American", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_asian, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0, .50),label = "A4", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_asian_icu, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0, .50), label = "B4", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_asian_aki, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C4", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")


gam_plotMed_noGradient(gamfitMed_eICU_others, main = "eICU Others", xRange = x_range,
                    yRange = c(0,0.5), label = "A5", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_others_icu, main = "eICU Others", xRange = x_range,
                    yRange = c(0, 0.5), label = "B5", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_others_aki, main = "eICU Others", xRange = x_range,
                    yRange = c(0.7, 1.0),label = "C5", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_overall, main = "eICU Overall", xRange = x_range,
                    yRange = c(0,0.5),label = "A6", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_overall_icu, main = "eICU Overall", xRange = x_range,
                    yRange = c(0,0.5), label = "B6", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_overall_aki, main = "eICU Overall", xRange = x_range,
                    yRange = c(0.7,1.0),label = "C6", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

dev.off()



png 
  2

In [ ]:
### DBP 72h eICU

In [56]:
df_white <-df_eicu[df_eicu$ethnicity=='Caucasian',]
df_hispanic <-df_eicu[df_eicu$ethnicity=='Hispanic',]
df_black<-df_eicu[df_eicu$ethnicity=='African American',]
df_asian <-df_eicu[df_eicu$ethnicity=='Asian',]
df_others<-df_eicu[df_eicu$ethnicity=='Others' |df_eicu$ethnicity=='Native American', ]


gamfitMed_eICU_white <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_eICU_hispanic <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_eICU_black <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_eICU_asian <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_eICU_others <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_eICU_overall <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_eicu, family = binomial)

gamfitMed_eICU_white_icu <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_eICU_hispanic_icu <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_eICU_black_icu <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_eICU_asian_icu <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_eICU_others_icu <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_eICU_overall_icu <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_eicu, family = binomial)

gamfitMed_eICU_white_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_white, family = binomial)
gamfitMed_eICU_hispanic_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_hispanic, family = binomial)
gamfitMed_eICU_black_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_black, family = binomial)
gamfitMed_eICU_asian_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_asian, family = binomial)
gamfitMed_eICU_others_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_others, family = binomial)
gamfitMed_eICU_overall_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+(age)+(bmi)+(sofatotal), data = df_eicu, family = binomial)

pdf(file = "figure/subgroup/Ethnicity_eICU_DBP72_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(35, 75)
# hospital mortality
gam_plotMed(gamfitMed_eICU_white, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_white_icu, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0, .50), label = "B1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_white_aki, main = "eICU Caucasian", xRange = x_range,
                    yRange = c(0.7,1.0),label = "C1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_hispanic, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0, .50),label = "A2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_hispanic_icu, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0, .50), label = "B2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_hispanic_aki, main = "eICU Hispanic", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")


gam_plotMed(gamfitMed_eICU_black, main = "eICU African American", xRange = x_range,
                    yRange = c(0, 0.5), label = "A3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_black_icu, main = "eICU African American", xRange = x_range,
                    yRange = c(0, 0.5), label = "B3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_black_aki, main = "eICU African American", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_asian, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0, .50),label = "A4", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_asian_icu, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0, .50), label = "B4", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_asian_aki, main = "eICU ASIAN", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C4", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")


gam_plotMed_noGradient(gamfitMed_eICU_others, main = "eICU Others", xRange = x_range,
                    yRange = c(0,0.5), label = "A5", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_others_icu, main = "eICU Others", xRange = x_range,
                    yRange = c(0, 0.5), label = "B5", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_others_aki, main = "eICU Others", xRange = x_range,
                    yRange = c(0.7, 1.0),label = "C5", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_overall, main = "eICU Overall", xRange = x_range,
                    yRange = c(0,0.5),label = "A6", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_overall_icu, main = "eICU Overall", xRange = x_range,
                    yRange = c(0,0.5), label = "B6", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_overall_aki, main = "eICU Overall", xRange = x_range,
                    yRange = c(0.7,1.0),label = "C6", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

dev.off()



Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”

png 
  2

## ICU types

In [66]:
unique(df_mimic$unittype)
# unique(df_eicu$unittype)

[1] SICU  TSICU MICU  CSRU  CCU  
Levels: CCU CSRU MICU SICU TSICU

In [58]:
df_sicu <-df_mimic[df_mimic$unittype=='SICU',]
df_tsicu <-df_mimic[df_mimic$unittype=='TSICU',]
df_micu<-df_mimic[df_mimic$unittype=='MICU',]
df_csru <-df_mimic[df_mimic$unittype=='CSRU',]
df_ccu<-df_mimic[df_mimic$unittype=='CCU',]

gamfitMed_MIMIC_SICU <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sicu, family = binomial)
gamfitMed_MIMIC_TSICU <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_tsicu, family = binomial)
gamfitMed_MIMIC_MICU <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_micu, family = binomial)
gamfitMed_MIMIC_CSRU <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_csru, family = binomial)
gamfitMed_MIMIC_CCU <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_ccu, family = binomial)
gamfitMed_MIMIC_overall <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

gamfitMed_MIMIC_SICU_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sicu, family = binomial)
gamfitMed_MIMIC_TSICU_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_tsicu, family = binomial)
gamfitMed_MIMIC_MICU_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_micu, family = binomial)
gamfitMed_MIMIC_CSRU_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_csru, family = binomial)
gamfitMed_MIMIC_CCU_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_ccu, family = binomial)
gamfitMed_MIMIC_overall_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

gamfitMed_MIMIC_SICU_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sicu, family = binomial)
gamfitMed_MIMIC_TSICU_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_tsicu, family = binomial)
gamfitMed_MIMIC_MICU_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_micu, family = binomial)
gamfitMed_MIMIC_CSRU_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_csru, family = binomial)
gamfitMed_MIMIC_CCU_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_ccu, family = binomial)
gamfitMed_MIMIC_overall_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

pdf(file = "figure/subgroup/ICUType_MIMIC_MAP24_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(55, 95)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_SICU, main = "MIMIC SICU", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_SICU_icu, main = "MIMIC SICU", xRange = x_range,
                    yRange = c(0, .50), label = "B1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_SICU_aki, main = "MIMIC SICU", xRange = x_range,
                    yRange = c(0,1),label = "C1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_TSICU, main = "MIMIC TSICU", xRange = x_range,
                    yRange = c(0, .50),label = "A2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_TSICU_icu, main = "MIMIC TSICU", xRange = x_range,
                    yRange = c(0, .50), label = "B2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_TSICU_aki, main = "MIMIC TSICU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_MICU, main = "MIMIC MICU", xRange = x_range,
                    yRange = c(0, 0.5), label = "A3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_MICU_icu, main = "MIMIC MICU", xRange = x_range,
                    yRange = c(0, 0.5), label = "B3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_MICU_aki, main = "MIMIC MICU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_CSRU, main = "MIMIC CSRU", xRange = x_range,
                    yRange = c(0, .50),label = "A4", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_CSRU_icu, main = "MIMIC CSRU", xRange = x_range,
                    yRange = c(0, .50), label = "B4", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_CSRU_aki, main = "MIMIC CSRU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C4", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_CCU, main = "MIMIC CCU", xRange = x_range,
                    yRange = c(0,0.4), label = "A5", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_CCU_icu, main = "MIMIC CCU", xRange = x_range,
                    yRange = c(0, 0.5), label = "B5", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_CCU_aki, main = "MIMIC CCU", xRange = x_range,
                    yRange = c(0, 1.0),label = "C5", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_overall, main = "MIMIC Overall", xRange = x_range,
                    yRange = c(0,0.4),label = "A6", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_overall_icu, main = "MIMIC Overall", xRange = x_range,
                    yRange = c(0,0.4), label = "B6", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_overall_aki, main = "MIMIC Overall", xRange = x_range,
                    yRange = c(0,1),label = "C6", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

dev.off()

png 
  2

In [60]:

df_sicu <-df_mimic[df_mimic$unittype=='SICU',]
df_tsicu <-df_mimic[df_mimic$unittype=='TSICU',]
df_micu<-df_mimic[df_mimic$unittype=='MICU',]
df_csru <-df_mimic[df_mimic$unittype=='CSRU',]
df_ccu<-df_mimic[df_mimic$unittype=='CCU',]

gamfitMed_MIMIC_SICU <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sicu, family = binomial)
gamfitMed_MIMIC_TSICU <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_tsicu, family = binomial)
gamfitMed_MIMIC_MICU <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_micu, family = binomial)
gamfitMed_MIMIC_CSRU <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_csru, family = binomial)
gamfitMed_MIMIC_CCU <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_ccu, family = binomial)
gamfitMed_MIMIC_overall <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

gamfitMed_MIMIC_SICU_icu <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sicu, family = binomial)
gamfitMed_MIMIC_TSICU_icu <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_tsicu, family = binomial)
gamfitMed_MIMIC_MICU_icu <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_micu, family = binomial)
gamfitMed_MIMIC_CSRU_icu <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_csru, family = binomial)
gamfitMed_MIMIC_CCU_icu <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_ccu, family = binomial)
gamfitMed_MIMIC_overall_icu <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

gamfitMed_MIMIC_SICU_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sicu, family = binomial)
gamfitMed_MIMIC_TSICU_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_tsicu, family = binomial)
gamfitMed_MIMIC_MICU_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_micu, family = binomial)
gamfitMed_MIMIC_CSRU_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_csru, family = binomial)
gamfitMed_MIMIC_CCU_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_ccu, family = binomial)
gamfitMed_MIMIC_overall_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

pdf(file = "figure/subgroup/ICUType_MIMIC_MAP48_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(55, 95)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_SICU, main = "MIMIC SICU", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_SICU_icu, main = "MIMIC SICU", xRange = x_range,
                    yRange = c(0, .50), label = "B1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_SICU_aki, main = "MIMIC SICU", xRange = x_range,
                    yRange = c(0,1),label = "C1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_TSICU, main = "MIMIC TSICU", xRange = x_range,
                    yRange = c(0, .50),label = "A2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_TSICU_icu, main = "MIMIC TSICU", xRange = x_range,
                    yRange = c(0, .50), label = "B2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_TSICU_aki, main = "MIMIC TSICU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_MICU, main = "MIMIC MICU", xRange = x_range,
                    yRange = c(0, 0.5), label = "A3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_MICU_icu, main = "MIMIC MICU", xRange = x_range,
                    yRange = c(0, 0.5), label = "B3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_MICU_aki, main = "MIMIC MICU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_CSRU, main = "MIMIC CSRU", xRange = x_range,
                    yRange = c(0, .50),label = "A4", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_CSRU_icu, main = "MIMIC CSRU", xRange = x_range,
                    yRange = c(0, .50), label = "B4", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_CSRU_aki, main = "MIMIC CSRU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C4", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_CCU, main = "MIMIC CCU", xRange = x_range,
                    yRange = c(0,0.4), label = "A5", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_CCU_icu, main = "MIMIC CCU", xRange = x_range,
                    yRange = c(0, 0.5), label = "B5", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_CCU_aki, main = "MIMIC CCU", xRange = x_range,
                    yRange = c(0, 1.0),label = "C5", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_overall, main = "MIMIC Overall", xRange = x_range,
                    yRange = c(0,0.4),label = "A6", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_overall_icu, main = "MIMIC Overall", xRange = x_range,
                    yRange = c(0,0.4), label = "B6", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_overall_aki, main = "MIMIC Overall", xRange = x_range,
                    yRange = c(0,1),label = "C6", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

dev.off()

png 
  2

In [61]:

df_sicu <-df_mimic[df_mimic$unittype=='SICU',]
df_tsicu <-df_mimic[df_mimic$unittype=='TSICU',]
df_micu<-df_mimic[df_mimic$unittype=='MICU',]
df_csru <-df_mimic[df_mimic$unittype=='CSRU',]
df_ccu<-df_mimic[df_mimic$unittype=='CCU',]

gamfitMed_MIMIC_SICU <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sicu, family = binomial)
gamfitMed_MIMIC_TSICU <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_tsicu, family = binomial)
gamfitMed_MIMIC_MICU <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_micu, family = binomial)
gamfitMed_MIMIC_CSRU <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_csru, family = binomial)
gamfitMed_MIMIC_CCU <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_ccu, family = binomial)
gamfitMed_MIMIC_overall <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

gamfitMed_MIMIC_SICU_icu <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sicu, family = binomial)
gamfitMed_MIMIC_TSICU_icu <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_tsicu, family = binomial)
gamfitMed_MIMIC_MICU_icu <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_micu, family = binomial)
gamfitMed_MIMIC_CSRU_icu <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_csru, family = binomial)
gamfitMed_MIMIC_CCU_icu <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_ccu, family = binomial)
gamfitMed_MIMIC_overall_icu <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

gamfitMed_MIMIC_SICU_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sicu, family = binomial)
gamfitMed_MIMIC_TSICU_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_tsicu, family = binomial)
gamfitMed_MIMIC_MICU_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_micu, family = binomial)
gamfitMed_MIMIC_CSRU_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_csru, family = binomial)
gamfitMed_MIMIC_CCU_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_ccu, family = binomial)
gamfitMed_MIMIC_overall_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

pdf(file = "figure/subgroup/ICUType_MIMIC_MAP72_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(55, 95)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_SICU, main = "MIMIC SICU", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_SICU_icu, main = "MIMIC SICU", xRange = x_range,
                    yRange = c(0, .50), label = "B1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_SICU_aki, main = "MIMIC SICU", xRange = x_range,
                    yRange = c(0,1),label = "C1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_TSICU, main = "MIMIC TSICU", xRange = x_range,
                    yRange = c(0, .50),label = "A2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_TSICU_icu, main = "MIMIC TSICU", xRange = x_range,
                    yRange = c(0, .50), label = "B2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_TSICU_aki, main = "MIMIC TSICU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_MICU, main = "MIMIC MICU", xRange = x_range,
                    yRange = c(0, 0.5), label = "A3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_MICU_icu, main = "MIMIC MICU", xRange = x_range,
                    yRange = c(0, 0.5), label = "B3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_MICU_aki, main = "MIMIC MICU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_CSRU, main = "MIMIC CSRU", xRange = x_range,
                    yRange = c(0, .50),label = "A4", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_CSRU_icu, main = "MIMIC CSRU", xRange = x_range,
                    yRange = c(0, .50), label = "B4", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_CSRU_aki, main = "MIMIC CSRU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C4", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_CCU, main = "MIMIC CCU", xRange = x_range,
                    yRange = c(0,0.4), label = "A5", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_CCU_icu, main = "MIMIC CCU", xRange = x_range,
                    yRange = c(0, 0.5), label = "B5", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_CCU_aki, main = "MIMIC CCU", xRange = x_range,
                    yRange = c(0, 1.0),label = "C5", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_overall, main = "MIMIC Overall", xRange = x_range,
                    yRange = c(0,0.4),label = "A6", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_overall_icu, main = "MIMIC Overall", xRange = x_range,
                    yRange = c(0,0.4), label = "B6", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_overall_aki, main = "MIMIC Overall", xRange = x_range,
                    yRange = c(0,1),label = "C6", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

dev.off()

png 
  2

### MIMIC SBP 24, 48, 72h

In [62]:

df_sicu <-df_mimic[df_mimic$unittype=='SICU',]
df_tsicu <-df_mimic[df_mimic$unittype=='TSICU',]
df_micu<-df_mimic[df_mimic$unittype=='MICU',]
df_csru <-df_mimic[df_mimic$unittype=='CSRU',]
df_ccu<-df_mimic[df_mimic$unittype=='CCU',]

gamfitMed_MIMIC_SICU <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sicu, family = binomial)
gamfitMed_MIMIC_TSICU <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_tsicu, family = binomial)
gamfitMed_MIMIC_MICU <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_micu, family = binomial)
gamfitMed_MIMIC_CSRU <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_csru, family = binomial)
gamfitMed_MIMIC_CCU <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_ccu, family = binomial)
gamfitMed_MIMIC_overall <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

gamfitMed_MIMIC_SICU_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sicu, family = binomial)
gamfitMed_MIMIC_TSICU_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_tsicu, family = binomial)
gamfitMed_MIMIC_MICU_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_micu, family = binomial)
gamfitMed_MIMIC_CSRU_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_csru, family = binomial)
gamfitMed_MIMIC_CCU_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_ccu, family = binomial)
gamfitMed_MIMIC_overall_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

gamfitMed_MIMIC_SICU_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sicu, family = binomial)
gamfitMed_MIMIC_TSICU_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_tsicu, family = binomial)
gamfitMed_MIMIC_MICU_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_micu, family = binomial)
gamfitMed_MIMIC_CSRU_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_csru, family = binomial)
gamfitMed_MIMIC_CCU_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_ccu, family = binomial)
gamfitMed_MIMIC_overall_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

pdf(file = "figure/subgroup/ICUType_MIMIC_SBP24_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(90, 140)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_SICU, main = "MIMIC SICU", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_SICU_icu, main = "MIMIC SICU", xRange = x_range,
                    yRange = c(0, .50), label = "B1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_SICU_aki, main = "MIMIC SICU", xRange = x_range,
                    yRange = c(0,1),label = "C1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_TSICU, main = "MIMIC TSICU", xRange = x_range,
                    yRange = c(0, .50),label = "A2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_TSICU_icu, main = "MIMIC TSICU", xRange = x_range,
                    yRange = c(0, .50), label = "B2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_TSICU_aki, main = "MIMIC TSICU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_MICU, main = "MIMIC MICU", xRange = x_range,
                    yRange = c(0, 0.5), label = "A3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_MICU_icu, main = "MIMIC MICU", xRange = x_range,
                    yRange = c(0, 0.5), label = "B3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_MICU_aki, main = "MIMIC MICU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_CSRU, main = "MIMIC CSRU", xRange = x_range,
                    yRange = c(0, .50),label = "A4", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_CSRU_icu, main = "MIMIC CSRU", xRange = x_range,
                    yRange = c(0, .50), label = "B4", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_CSRU_aki, main = "MIMIC CSRU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C4", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_CCU, main = "MIMIC CCU", xRange = x_range,
                    yRange = c(0,0.4), label = "A5", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_CCU_icu, main = "MIMIC CCU", xRange = x_range,
                    yRange = c(0, 0.5), label = "B5", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_CCU_aki, main = "MIMIC CCU", xRange = x_range,
                    yRange = c(0, 1.0),label = "C5", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_overall, main = "MIMIC Overall", xRange = x_range,
                    yRange = c(0,0.4),label = "A6", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_overall_icu, main = "MIMIC Overall", xRange = x_range,
                    yRange = c(0,0.4), label = "B6", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_overall_aki, main = "MIMIC Overall", xRange = x_range,
                    yRange = c(0,1),label = "C6", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

dev.off()

png 
  2

In [65]:

df_sicu <-df_mimic[df_mimic$unittype=='SICU',]
df_tsicu <-df_mimic[df_mimic$unittype=='TSICU',]
df_micu<-df_mimic[df_mimic$unittype=='MICU',]
df_csru <-df_mimic[df_mimic$unittype=='CSRU',]
df_ccu<-df_mimic[df_mimic$unittype=='CCU',]

gamfitMed_MIMIC_SICU <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sicu, family = binomial)
gamfitMed_MIMIC_TSICU <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_tsicu, family = binomial)
gamfitMed_MIMIC_MICU <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_micu, family = binomial)
gamfitMed_MIMIC_CSRU <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_csru, family = binomial)
gamfitMed_MIMIC_CCU <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_ccu, family = binomial)
gamfitMed_MIMIC_overall <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

gamfitMed_MIMIC_SICU_icu <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sicu, family = binomial)
gamfitMed_MIMIC_TSICU_icu <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_tsicu, family = binomial)
gamfitMed_MIMIC_MICU_icu <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_micu, family = binomial)
gamfitMed_MIMIC_CSRU_icu <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_csru, family = binomial)
gamfitMed_MIMIC_CCU_icu <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_ccu, family = binomial)
gamfitMed_MIMIC_overall_icu <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

gamfitMed_MIMIC_SICU_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sicu, family = binomial)
gamfitMed_MIMIC_TSICU_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_tsicu, family = binomial)
gamfitMed_MIMIC_MICU_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_micu, family = binomial)
gamfitMed_MIMIC_CSRU_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_csru, family = binomial)
gamfitMed_MIMIC_CCU_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_ccu, family = binomial)
gamfitMed_MIMIC_overall_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

pdf(file = "figure/subgroup/ICUType_MIMIC_SBP48_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(90, 140)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_SICU, main = "MIMIC SICU", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_SICU_icu, main = "MIMIC SICU", xRange = x_range,
                    yRange = c(0, .50), label = "B1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_SICU_aki, main = "MIMIC SICU", xRange = x_range,
                    yRange = c(0,1),label = "C1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_TSICU, main = "MIMIC TSICU", xRange = x_range,
                    yRange = c(0, .50),label = "A2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_TSICU_icu, main = "MIMIC TSICU", xRange = x_range,
                    yRange = c(0, .50), label = "B2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_TSICU_aki, main = "MIMIC TSICU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_MICU, main = "MIMIC MICU", xRange = x_range,
                    yRange = c(0, 0.5), label = "A3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_MICU_icu, main = "MIMIC MICU", xRange = x_range,
                    yRange = c(0, 0.5), label = "B3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_MICU_aki, main = "MIMIC MICU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_CSRU, main = "MIMIC CSRU", xRange = x_range,
                    yRange = c(0, .50),label = "A4", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_CSRU_icu, main = "MIMIC CSRU", xRange = x_range,
                    yRange = c(0, .50), label = "B4", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_CSRU_aki, main = "MIMIC CSRU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C4", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_CCU, main = "MIMIC CCU", xRange = x_range,
                    yRange = c(0,0.4), label = "A5", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_CCU_icu, main = "MIMIC CCU", xRange = x_range,
                    yRange = c(0, 0.5), label = "B5", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_CCU_aki, main = "MIMIC CCU", xRange = x_range,
                    yRange = c(0, 1.0),label = "C5", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_overall, main = "MIMIC Overall", xRange = x_range,
                    yRange = c(0,0.4),label = "A6", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_overall_icu, main = "MIMIC Overall", xRange = x_range,
                    yRange = c(0,0.4), label = "B6", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_overall_aki, main = "MIMIC Overall", xRange = x_range,
                    yRange = c(0,1),label = "C6", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

dev.off()

png 
  2

In [66]:

df_sicu <-df_mimic[df_mimic$unittype=='SICU',]
df_tsicu <-df_mimic[df_mimic$unittype=='TSICU',]
df_micu<-df_mimic[df_mimic$unittype=='MICU',]
df_csru <-df_mimic[df_mimic$unittype=='CSRU',]
df_ccu<-df_mimic[df_mimic$unittype=='CCU',]

gamfitMed_MIMIC_SICU <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sicu, family = binomial)
gamfitMed_MIMIC_TSICU <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_tsicu, family = binomial)
gamfitMed_MIMIC_MICU <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_micu, family = binomial)
gamfitMed_MIMIC_CSRU <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_csru, family = binomial)
gamfitMed_MIMIC_CCU <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_ccu, family = binomial)
gamfitMed_MIMIC_overall <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

gamfitMed_MIMIC_SICU_icu <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sicu, family = binomial)
gamfitMed_MIMIC_TSICU_icu <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_tsicu, family = binomial)
gamfitMed_MIMIC_MICU_icu <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_micu, family = binomial)
gamfitMed_MIMIC_CSRU_icu <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_csru, family = binomial)
gamfitMed_MIMIC_CCU_icu <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_ccu, family = binomial)
gamfitMed_MIMIC_overall_icu <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

gamfitMed_MIMIC_SICU_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sicu, family = binomial)
gamfitMed_MIMIC_TSICU_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_tsicu, family = binomial)
gamfitMed_MIMIC_MICU_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_micu, family = binomial)
gamfitMed_MIMIC_CSRU_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_csru, family = binomial)
gamfitMed_MIMIC_CCU_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_ccu, family = binomial)
gamfitMed_MIMIC_overall_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

pdf(file = "figure/subgroup/ICUType_MIMIC_SBP72_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(90, 140)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_SICU, main = "MIMIC SICU", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_SICU_icu, main = "MIMIC SICU", xRange = x_range,
                    yRange = c(0, .50), label = "B1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_SICU_aki, main = "MIMIC SICU", xRange = x_range,
                    yRange = c(0,1),label = "C1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_TSICU, main = "MIMIC TSICU", xRange = x_range,
                    yRange = c(0, .50),label = "A2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_TSICU_icu, main = "MIMIC TSICU", xRange = x_range,
                    yRange = c(0, .50), label = "B2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_TSICU_aki, main = "MIMIC TSICU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_MICU, main = "MIMIC MICU", xRange = x_range,
                    yRange = c(0, 0.5), label = "A3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_MICU_icu, main = "MIMIC MICU", xRange = x_range,
                    yRange = c(0, 0.5), label = "B3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_MICU_aki, main = "MIMIC MICU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_CSRU, main = "MIMIC CSRU", xRange = x_range,
                    yRange = c(0, .50),label = "A4", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_CSRU_icu, main = "MIMIC CSRU", xRange = x_range,
                    yRange = c(0, .50), label = "B4", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_CSRU_aki, main = "MIMIC CSRU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C4", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_CCU, main = "MIMIC CCU", xRange = x_range,
                    yRange = c(0,0.4), label = "A5", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_CCU_icu, main = "MIMIC CCU", xRange = x_range,
                    yRange = c(0, 0.5), label = "B5", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_CCU_aki, main = "MIMIC CCU", xRange = x_range,
                    yRange = c(0, 1.0),label = "C5", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_overall, main = "MIMIC Overall", xRange = x_range,
                    yRange = c(0,0.4),label = "A6", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_overall_icu, main = "MIMIC Overall", xRange = x_range,
                    yRange = c(0,0.4), label = "B6", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_overall_aki, main = "MIMIC Overall", xRange = x_range,
                    yRange = c(0,1),label = "C6", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

dev.off()

png 
  2

### MIMIC DBP 24, 48, 72h

In [67]:

df_sicu <-df_mimic[df_mimic$unittype=='SICU',]
df_tsicu <-df_mimic[df_mimic$unittype=='TSICU',]
df_micu<-df_mimic[df_mimic$unittype=='MICU',]
df_csru <-df_mimic[df_mimic$unittype=='CSRU',]
df_ccu<-df_mimic[df_mimic$unittype=='CCU',]

gamfitMed_MIMIC_SICU <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sicu, family = binomial)
gamfitMed_MIMIC_TSICU <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_tsicu, family = binomial)
gamfitMed_MIMIC_MICU <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_micu, family = binomial)
gamfitMed_MIMIC_CSRU <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_csru, family = binomial)
gamfitMed_MIMIC_CCU <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_ccu, family = binomial)
gamfitMed_MIMIC_overall <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

gamfitMed_MIMIC_SICU_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sicu, family = binomial)
gamfitMed_MIMIC_TSICU_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_tsicu, family = binomial)
gamfitMed_MIMIC_MICU_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_micu, family = binomial)
gamfitMed_MIMIC_CSRU_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_csru, family = binomial)
gamfitMed_MIMIC_CCU_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_ccu, family = binomial)
gamfitMed_MIMIC_overall_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

gamfitMed_MIMIC_SICU_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sicu, family = binomial)
gamfitMed_MIMIC_TSICU_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_tsicu, family = binomial)
gamfitMed_MIMIC_MICU_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_micu, family = binomial)
gamfitMed_MIMIC_CSRU_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_csru, family = binomial)
gamfitMed_MIMIC_CCU_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_ccu, family = binomial)
gamfitMed_MIMIC_overall_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

pdf(file = "figure/subgroup/ICUType_MIMIC_DBP24_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(35, 75)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_SICU, main = "MIMIC SICU", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_SICU_icu, main = "MIMIC SICU", xRange = x_range,
                    yRange = c(0, .50), label = "B1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_SICU_aki, main = "MIMIC SICU", xRange = x_range,
                    yRange = c(0,1),label = "C1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_TSICU, main = "MIMIC TSICU", xRange = x_range,
                    yRange = c(0, .50),label = "A2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_TSICU_icu, main = "MIMIC TSICU", xRange = x_range,
                    yRange = c(0, .50), label = "B2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_TSICU_aki, main = "MIMIC TSICU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_MICU, main = "MIMIC MICU", xRange = x_range,
                    yRange = c(0, 0.5), label = "A3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_MICU_icu, main = "MIMIC MICU", xRange = x_range,
                    yRange = c(0, 0.5), label = "B3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_MICU_aki, main = "MIMIC MICU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_CSRU, main = "MIMIC CSRU", xRange = x_range,
                    yRange = c(0, .50),label = "A4", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_CSRU_icu, main = "MIMIC CSRU", xRange = x_range,
                    yRange = c(0, .50), label = "B4", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_CSRU_aki, main = "MIMIC CSRU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C4", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_CCU, main = "MIMIC CCU", xRange = x_range,
                    yRange = c(0,0.4), label = "A5", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_CCU_icu, main = "MIMIC CCU", xRange = x_range,
                    yRange = c(0, 0.5), label = "B5", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_CCU_aki, main = "MIMIC CCU", xRange = x_range,
                    yRange = c(0, 1.0),label = "C5", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_overall, main = "MIMIC Overall", xRange = x_range,
                    yRange = c(0,0.4),label = "A6", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_overall_icu, main = "MIMIC Overall", xRange = x_range,
                    yRange = c(0,0.4), label = "B6", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_overall_aki, main = "MIMIC Overall", xRange = x_range,
                    yRange = c(0,1),label = "C6", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

dev.off()

Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”

png 
  2

In [68]:

df_sicu <-df_mimic[df_mimic$unittype=='SICU',]
df_tsicu <-df_mimic[df_mimic$unittype=='TSICU',]
df_micu<-df_mimic[df_mimic$unittype=='MICU',]
df_csru <-df_mimic[df_mimic$unittype=='CSRU',]
df_ccu<-df_mimic[df_mimic$unittype=='CCU',]

gamfitMed_MIMIC_SICU <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sicu, family = binomial)
gamfitMed_MIMIC_TSICU <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_tsicu, family = binomial)
gamfitMed_MIMIC_MICU <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_micu, family = binomial)
gamfitMed_MIMIC_CSRU <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_csru, family = binomial)
gamfitMed_MIMIC_CCU <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_ccu, family = binomial)
gamfitMed_MIMIC_overall <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

gamfitMed_MIMIC_SICU_icu <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sicu, family = binomial)
gamfitMed_MIMIC_TSICU_icu <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_tsicu, family = binomial)
gamfitMed_MIMIC_MICU_icu <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_micu, family = binomial)
gamfitMed_MIMIC_CSRU_icu <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_csru, family = binomial)
gamfitMed_MIMIC_CCU_icu <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_ccu, family = binomial)
gamfitMed_MIMIC_overall_icu <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

gamfitMed_MIMIC_SICU_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sicu, family = binomial)
gamfitMed_MIMIC_TSICU_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_tsicu, family = binomial)
gamfitMed_MIMIC_MICU_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_micu, family = binomial)
gamfitMed_MIMIC_CSRU_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_csru, family = binomial)
gamfitMed_MIMIC_CCU_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_ccu, family = binomial)
gamfitMed_MIMIC_overall_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

pdf(file = "figure/subgroup/ICUType_MIMIC_DBP48_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(35, 75)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_SICU, main = "MIMIC SICU", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_SICU_icu, main = "MIMIC SICU", xRange = x_range,
                    yRange = c(0, .50), label = "B1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_SICU_aki, main = "MIMIC SICU", xRange = x_range,
                    yRange = c(0,1),label = "C1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_TSICU, main = "MIMIC TSICU", xRange = x_range,
                    yRange = c(0, .50),label = "A2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_TSICU_icu, main = "MIMIC TSICU", xRange = x_range,
                    yRange = c(0, .50), label = "B2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_TSICU_aki, main = "MIMIC TSICU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_MICU, main = "MIMIC MICU", xRange = x_range,
                    yRange = c(0, 0.5), label = "A3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_MICU_icu, main = "MIMIC MICU", xRange = x_range,
                    yRange = c(0, 0.5), label = "B3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_MICU_aki, main = "MIMIC MICU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_CSRU, main = "MIMIC CSRU", xRange = x_range,
                    yRange = c(0, .50),label = "A4", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_CSRU_icu, main = "MIMIC CSRU", xRange = x_range,
                    yRange = c(0, .50), label = "B4", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_CSRU_aki, main = "MIMIC CSRU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C4", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_CCU, main = "MIMIC CCU", xRange = x_range,
                    yRange = c(0,0.4), label = "A5", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_CCU_icu, main = "MIMIC CCU", xRange = x_range,
                    yRange = c(0, 0.5), label = "B5", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_CCU_aki, main = "MIMIC CCU", xRange = x_range,
                    yRange = c(0, 1.0),label = "C5", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_overall, main = "MIMIC Overall", xRange = x_range,
                    yRange = c(0,0.4),label = "A6", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_overall_icu, main = "MIMIC Overall", xRange = x_range,
                    yRange = c(0,0.4), label = "B6", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_overall_aki, main = "MIMIC Overall", xRange = x_range,
                    yRange = c(0,1),label = "C6", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

dev.off()

png 
  2

In [69]:

df_sicu <-df_mimic[df_mimic$unittype=='SICU',]
df_tsicu <-df_mimic[df_mimic$unittype=='TSICU',]
df_micu<-df_mimic[df_mimic$unittype=='MICU',]
df_csru <-df_mimic[df_mimic$unittype=='CSRU',]
df_ccu<-df_mimic[df_mimic$unittype=='CCU',]

gamfitMed_MIMIC_SICU <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sicu, family = binomial)
gamfitMed_MIMIC_TSICU <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_tsicu, family = binomial)
gamfitMed_MIMIC_MICU <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_micu, family = binomial)
gamfitMed_MIMIC_CSRU <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_csru, family = binomial)
gamfitMed_MIMIC_CCU <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_ccu, family = binomial)
gamfitMed_MIMIC_overall <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

gamfitMed_MIMIC_SICU_icu <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sicu, family = binomial)
gamfitMed_MIMIC_TSICU_icu <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_tsicu, family = binomial)
gamfitMed_MIMIC_MICU_icu <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_micu, family = binomial)
gamfitMed_MIMIC_CSRU_icu <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_csru, family = binomial)
gamfitMed_MIMIC_CCU_icu <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_ccu, family = binomial)
gamfitMed_MIMIC_overall_icu <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

gamfitMed_MIMIC_SICU_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sicu, family = binomial)
gamfitMed_MIMIC_TSICU_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_tsicu, family = binomial)
gamfitMed_MIMIC_MICU_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_micu, family = binomial)
gamfitMed_MIMIC_CSRU_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_csru, family = binomial)
gamfitMed_MIMIC_CCU_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_ccu, family = binomial)
gamfitMed_MIMIC_overall_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

pdf(file = "figure/subgroup/ICUType_MIMIC_DBP72_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(35, 75)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_SICU, main = "MIMIC SICU", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_SICU_icu, main = "MIMIC SICU", xRange = x_range,
                    yRange = c(0, .50), label = "B1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_SICU_aki, main = "MIMIC SICU", xRange = x_range,
                    yRange = c(0,1),label = "C1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_TSICU, main = "MIMIC TSICU", xRange = x_range,
                    yRange = c(0, .50),label = "A2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_TSICU_icu, main = "MIMIC TSICU", xRange = x_range,
                    yRange = c(0, .50), label = "B2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_TSICU_aki, main = "MIMIC TSICU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_MICU, main = "MIMIC MICU", xRange = x_range,
                    yRange = c(0, 0.5), label = "A3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_MICU_icu, main = "MIMIC MICU", xRange = x_range,
                    yRange = c(0, 0.5), label = "B3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_MICU_aki, main = "MIMIC MICU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_CSRU, main = "MIMIC CSRU", xRange = x_range,
                    yRange = c(0, .50),label = "A4", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_CSRU_icu, main = "MIMIC CSRU", xRange = x_range,
                    yRange = c(0, .50), label = "B4", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_CSRU_aki, main = "MIMIC CSRU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C4", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_CCU, main = "MIMIC CCU", xRange = x_range,
                    yRange = c(0,0.4), label = "A5", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_CCU_icu, main = "MIMIC CCU", xRange = x_range,
                    yRange = c(0, 0.5), label = "B5", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_CCU_aki, main = "MIMIC CCU", xRange = x_range,
                    yRange = c(0, 1.0),label = "C5", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_overall, main = "MIMIC Overall", xRange = x_range,
                    yRange = c(0,0.4),label = "A6", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_overall_icu, main = "MIMIC Overall", xRange = x_range,
                    yRange = c(0,0.4), label = "B6", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_overall_aki, main = "MIMIC Overall", xRange = x_range,
                    yRange = c(0,1),label = "C6", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

dev.off()

png 
  2

### eICU MAP 24, 48, 72h

In [164]:
df_SICU <-df_eicu[df_eicu$unittype=='SICU',]
df_CTICU <-df_eicu[df_eicu$unittype=='CTICU',]
df_MICU<-df_eicu[df_eicu$unittype=='MICU',]
df_MSICU<-df_eicu[df_eicu$unittype=='Med-Surg ICU',]
df_CCU_CTICU<-df_eicu[df_eicu$unittype=='CCU-CTICU',]
df_NICU <-df_eicu[df_eicu$unittype=='Neuro ICU',]
df_CCU<-df_eicu[df_eicu$unittype=='Cardiac ICU',]
df_CSICU<-df_eicu[df_eicu$unittype=='CSICU',]

gamfitMed_eICU_SICU <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_SICU, family = binomial)
gamfitMed_eICU_CTICU <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CTICU, family = binomial)
gamfitMed_eICU_MICU <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MICU, family = binomial)
gamfitMed_eICU_MSICU <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MSICU, family = binomial)
gamfitMed_eICU_CCU_CTICU <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU_CTICU, family = binomial)
gamfitMed_eICU_NICU <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_NICU, family = binomial)
gamfitMed_eICU_CCU <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU, family = binomial)
gamfitMed_eICU_CSICU <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CSICU, family = binomial)


gamfitMed_eICU_SICU_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_SICU, family = binomial)
gamfitMed_eICU_CTICU_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CTICU, family = binomial)
gamfitMed_eICU_MICU_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MICU, family = binomial)
gamfitMed_eICU_MSICU_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MSICU, family = binomial)
gamfitMed_eICU_CCU_CTICU_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU_CTICU, family = binomial)
gamfitMed_eICU_NICU_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_NICU, family = binomial)
gamfitMed_eICU_CCU_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU, family = binomial)
gamfitMed_eICU_CSICU_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CSICU, family = binomial)

gamfitMed_eICU_SICU_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_SICU, family = binomial)
gamfitMed_eICU_CTICU_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CTICU, family = binomial)
gamfitMed_eICU_MICU_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MICU, family = binomial)
gamfitMed_eICU_MSICU_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MSICU, family = binomial)
gamfitMed_eICU_CCU_CTICU_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU_CTICU, family = binomial)
gamfitMed_eICU_NICU_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_NICU, family = binomial)
gamfitMed_eICU_CCU_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU, family = binomial)
gamfitMed_eICU_CSICU_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CSICU, family = binomial)


pdf(file = "figure/subgroup/ICUType_eICU_MAP24_Hosp_ICU_AKI.pdf", width = plot_width*4,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,8), cex = 1)
x_range = c(55, 95)
# hospital mortality
gam_plotMed(gamfitMed_eICU_SICU, main = "eICU SICU", xRange = x_range,
                    yRange = c(0, 0.4), label = "A1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_SICU_icu, main = "eICU SICU", xRange = x_range,
                    yRange = c(0, 0.4), label = "B1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_SICU_aki, main = "eICU SICU", xRange = x_range,
                    yRange = c(0,1),label = "C1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_CTICU, main = "eICU CTICU", xRange = x_range,
                    yRange = c(0, 0.4),label = "A2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_CTICU_icu, main = "eICU CTICU", xRange = x_range,
                    yRange = c(0, 0.4), label = "B2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_CTICU_aki, main = "eICU CTICU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_MICU, main = "eICU MICU", xRange = x_range,
                    yRange = c(0, 0.4), label = "A3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_MICU_icu, main = "eICU MICU", xRange = x_range,
                    yRange = c(0, 0.4), label = "B3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_MICU_aki, main = "eICU MICU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_MSICU, main = "eICU Med-Surg ICU", xRange = x_range,
                    yRange = c(0, .50),label = "A4", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_MSICU_icu, main = "eICU Med-Surg ICU", xRange = x_range,
                    yRange = c(0, 0.4), label = "B4", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_MSICU_aki, main = "eICU Med-Surg ICU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C4", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_CCU_CTICU, main = "eICU CCU-CTICU", xRange = x_range,
                    yRange = c(0, 0.4), label = "A5", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_CCU_CTICU_icu, main = "eICU CCU-CTICU", xRange = x_range,
                    yRange = c(0, 0.4), label = "B5", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_CCU_CTICU_aki, main = "eICU CCU-CTICU", xRange = x_range,
                    yRange = c(0, 1.0),label = "C5", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_NICU, main = "eICU NICU", xRange = x_range,
                    yRange = c(0,0.4),label = "A6", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_NICU_icu, main = "eICU NICU", xRange = x_range,
                    yRange = c(0,0.4), label = "B6", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_NICU_aki, main = "eICU NICU", xRange = x_range,
                    yRange = c(0,1),label = "C6", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")
gam_plotMed(gamfitMed_eICU_CCU, main = "eICU CCU", xRange = x_range,
                    yRange = c(0,1),label = "A7", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_CCU_icu, main = "eICU CCU", xRange = x_range,
                    yRange = c(0,1), label = "B7", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_CCU_aki, main = "eICU CCU", xRange = x_range,
                    yRange = c(0,1),label = "C7", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_CSICU, main = "eICU CSICU", xRange = x_range,
                    yRange = c(0,0.4),label = "A8", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_CSICU_icu, main = "eICU CSICU", xRange = x_range,
                    yRange = c(0,0.4), label = "B8", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_CSICU_aki, main = "eICU CSICU", xRange = x_range,
                    yRange = c(0,1),label = "C8", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

dev.off()

png 
  2

In [165]:
df_SICU <-df_eicu[df_eicu$unittype=='SICU',]
df_CTICU <-df_eicu[df_eicu$unittype=='CTICU',]
df_MICU<-df_eicu[df_eicu$unittype=='MICU',]
df_MSICU<-df_eicu[df_eicu$unittype=='Med-Surg ICU',]
df_CCU_CTICU<-df_eicu[df_eicu$unittype=='CCU-CTICU',]
df_NICU <-df_eicu[df_eicu$unittype=='Neuro ICU',]
df_CCU<-df_eicu[df_eicu$unittype=='Cardiac ICU',]
df_CSICU<-df_eicu[df_eicu$unittype=='CSICU',]

gamfitMed_eICU_SICU <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_SICU, family = binomial)
gamfitMed_eICU_TSICU <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CTICU, family = binomial)
gamfitMed_eICU_MICU <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MICU, family = binomial)
gamfitMed_eICU_CSRU <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MSICU, family = binomial)
gamfitMed_eICU_CCU_CTICU <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU_CTICU, family = binomial)
gamfitMed_eICU_NICU <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_NICU, family = binomial)
gamfitMed_eICU_CCU <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU, family = binomial)
gamfitMed_eICU_CSICU <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CSICU, family = binomial)


gamfitMed_eICU_SICU_icu <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_SICU, family = binomial)
gamfitMed_eICU_TSICU_icu <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CTICU, family = binomial)
gamfitMed_eICU_MICU_icu <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MICU, family = binomial)
gamfitMed_eICU_CSRU_icu <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MSICU, family = binomial)
gamfitMed_eICU_CCU_CTICU_icu <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU_CTICU, family = binomial)
gamfitMed_eICU_NICU_icu <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_NICU, family = binomial)
gamfitMed_eICU_CCU_icu <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU, family = binomial)
gamfitMed_eICU_CSICU_icu <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CSICU, family = binomial)

gamfitMed_eICU_SICU_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_SICU, family = binomial)
gamfitMed_eICU_TSICU_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CTICU, family = binomial)
gamfitMed_eICU_MICU_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MICU, family = binomial)
gamfitMed_eICU_CSRU_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MSICU, family = binomial)
gamfitMed_eICU_CCU_CTICU_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU_CTICU, family = binomial)
gamfitMed_eICU_NICU_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_NICU, family = binomial)
gamfitMed_eICU_CCU_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU, family = binomial)
gamfitMed_eICU_CSICU_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CSICU, family = binomial)


pdf(file = "figure/subgroup/ICUType_eICU_MAP48_Hosp_ICU_AKI.pdf", width = plot_width*4,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,8), cex = 1)
x_range = c(55, 95)
# hospital mortality
gam_plotMed(gamfitMed_eICU_SICU, main = "eICU SICU", xRange = x_range,
                    yRange = c(0, 0.4), label = "A1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_SICU_icu, main = "eICU SICU", xRange = x_range,
                    yRange = c(0, 0.4), label = "B1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_SICU_aki, main = "eICU SICU", xRange = x_range,
                    yRange = c(0,1),label = "C1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_TSICU, main = "eICU CTICU", xRange = x_range,
                    yRange = c(0, 0.4),label = "A2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_TSICU_icu, main = "eICU CTICU", xRange = x_range,
                    yRange = c(0, 0.4), label = "B2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_TSICU_aki, main = "eICU CTICU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_MICU, main = "eICU MICU", xRange = x_range,
                    yRange = c(0, 0.4), label = "A3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_MICU_icu, main = "eICU MICU", xRange = x_range,
                    yRange = c(0, 0.4), label = "B3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_MICU_aki, main = "eICU MICU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_CSRU, main = "eICU Med-Surg ICU", xRange = x_range,
                    yRange = c(0, 0.4),label = "A4", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_CSRU_icu, main = "eICU Med-Surg ICU", xRange = x_range,
                    yRange = c(0, 0.4), label = "B4", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_CSRU_aki, main = "eICU Med-Surg ICU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C4", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_CCU_CTICU, main = "eICU CCU-CTICU", xRange = x_range,
                    yRange = c(0, 0.4), label = "A5", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_CCU_CTICU_icu, main = "eICU CCU-CTICU", xRange = x_range,
                    yRange = c(0, 0.4), label = "B5", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_CCU_CTICU_aki, main = "eICU CCU-CTICU", xRange = x_range,
                    yRange = c(0, 1.0),label = "C5", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_NICU, main = "eICU NICU", xRange = x_range,
                    yRange = c(0,0.4),label = "A6", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_NICU_icu, main = "eICU NICU", xRange = x_range,
                    yRange = c(0,0.4), label = "B6", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_NICU_aki, main = "eICU NICU", xRange = x_range,
                    yRange = c(0,1),label = "C6", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")
gam_plotMed(gamfitMed_eICU_CCU, main = "eICU CCU", xRange = x_range,
                    yRange = c(0, 1),label = "A7", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_CCU_icu, main = "eICU CCU", xRange = x_range,
                    yRange = c(0, 1), label = "B7", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_CCU_aki, main = "eICU CCU", xRange = x_range,
                    yRange = c(0,1),label = "C7", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_CSICU, main = "eICU CSICU", xRange = x_range,
                    yRange = c(0, 0.4),label = "A8", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_CSICU_icu, main = "eICU CSICU", xRange = x_range,
                    yRange = c(0, 0.4), label = "B8", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_CSICU_aki, main = "eICU CSICU", xRange = x_range,
                    yRange = c(0,1),label = "C8", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

dev.off()

Warning message in min(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”

png 
  2

In [166]:
df_SICU <-df_eicu[df_eicu$unittype=='SICU',]
df_CTICU <-df_eicu[df_eicu$unittype=='CTICU',]
df_MICU<-df_eicu[df_eicu$unittype=='MICU',]
df_MSICU<-df_eicu[df_eicu$unittype=='Med-Surg ICU',]
df_CCU_CTICU<-df_eicu[df_eicu$unittype=='CCU-CTICU',]
df_NICU <-df_eicu[df_eicu$unittype=='Neuro ICU',]
df_CCU<-df_eicu[df_eicu$unittype=='Cardiac ICU',]
df_CSICU<-df_eicu[df_eicu$unittype=='CSICU',]

gamfitMed_eICU_SICU <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_SICU, family = binomial)
gamfitMed_eICU_TSICU <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CTICU, family = binomial)
gamfitMed_eICU_MICU <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MICU, family = binomial)
gamfitMed_eICU_CSRU <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MSICU, family = binomial)
gamfitMed_eICU_CCU_CTICU <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU_CTICU, family = binomial)
gamfitMed_eICU_NICU <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_NICU, family = binomial)
gamfitMed_eICU_CCU <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU, family = binomial)
gamfitMed_eICU_CSICU <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CSICU, family = binomial)


gamfitMed_eICU_SICU_icu <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_SICU, family = binomial)
gamfitMed_eICU_TSICU_icu <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CTICU, family = binomial)
gamfitMed_eICU_MICU_icu <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MICU, family = binomial)
gamfitMed_eICU_CSRU_icu <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MSICU, family = binomial)
gamfitMed_eICU_CCU_CTICU_icu <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU_CTICU, family = binomial)
gamfitMed_eICU_NICU_icu <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_NICU, family = binomial)
gamfitMed_eICU_CCU_icu <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU, family = binomial)
gamfitMed_eICU_CSICU_icu <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CSICU, family = binomial)

gamfitMed_eICU_SICU_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_SICU, family = binomial)
gamfitMed_eICU_TSICU_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CTICU, family = binomial)
gamfitMed_eICU_MICU_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MICU, family = binomial)
gamfitMed_eICU_CSRU_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MSICU, family = binomial)
gamfitMed_eICU_CCU_CTICU_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU_CTICU, family = binomial)
gamfitMed_eICU_NICU_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_NICU, family = binomial)
gamfitMed_eICU_CCU_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU, family = binomial)
gamfitMed_eICU_CSICU_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CSICU, family = binomial)


pdf(file = "figure/subgroup/ICUType_eICU_MAP72_Hosp_ICU_AKI.pdf", width = plot_width*4,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,8), cex = 1)
x_range = c(55, 95)
# hospital mortality
gam_plotMed(gamfitMed_eICU_SICU, main = "eICU SICU", xRange = x_range,
                    yRange = c(0, 0.4), label = "A1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_SICU_icu, main = "eICU SICU", xRange = x_range,
                    yRange = c(0, .40), label = "B1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_SICU_aki, main = "eICU SICU", xRange = x_range,
                    yRange = c(0,1),label = "C1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_TSICU, main = "eICU CTICU", xRange = x_range,
                    yRange = c(0, .40),label = "A2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_TSICU_icu, main = "eICU CTICU", xRange = x_range,
                    yRange = c(0, .40), label = "B2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_TSICU_aki, main = "eICU CTICU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_MICU, main = "eICU MICU", xRange = x_range,
                    yRange = c(0, 0.4), label = "A3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_MICU_icu, main = "eICU MICU", xRange = x_range,
                    yRange = c(0, 0.4), label = "B3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_MICU_aki, main = "eICU MICU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_CSRU, main = "eICU Med-Surg ICU", xRange = x_range,
                    yRange = c(0, .40),label = "A4", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_CSRU_icu, main = "eICU Med-Surg ICU", xRange = x_range,
                    yRange = c(0, .40), label = "B4", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_CSRU_aki, main = "eICU Med-Surg ICU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C4", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_CCU_CTICU, main = "eICU CCU-CTICU", xRange = x_range,
                    yRange = c(0,0.4), label = "A5", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_CCU_CTICU_icu, main = "eICU CCU-CTICU", xRange = x_range,
                    yRange = c(0, 0.4), label = "B5", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_CCU_CTICU_aki, main = "eICU CCU-CTICU", xRange = x_range,
                    yRange = c(0, 1.0),label = "C5", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_NICU, main = "eICU NICU", xRange = x_range,
                    yRange = c(0,0.4),label = "A6", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_NICU_icu, main = "eICU NICU", xRange = x_range,
                    yRange = c(0,0.4), label = "B6", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_NICU_aki, main = "eICU NICU", xRange = x_range,
                    yRange = c(0,1),label = "C6", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")
gam_plotMed(gamfitMed_eICU_CCU, main = "eICU CCU", xRange = x_range,
                    yRange = c(0,1),label = "A7", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_CCU_icu, main = "eICU CCU", xRange = x_range,
                    yRange = c(0,1), label = "B7", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_CCU_aki, main = "eICU CCU", xRange = x_range,
                    yRange = c(0,1),label = "C7", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_CSICU, main = "eICU CSICU", xRange = x_range,
                    yRange = c(0,0.4),label = "A8", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_CSICU_icu, main = "eICU CSICU", xRange = x_range,
                    yRange = c(0,0.4), label = "B8", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_CSICU_aki, main = "eICU CSICU", xRange = x_range,
                    yRange = c(0,1),label = "C8", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

dev.off()

Warning message in min(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning me

png 
  2

### eICU SBP 24h, 48h, 72h

In [167]:
df_SICU <-df_eicu[df_eicu$unittype=='SICU',]
df_CTICU <-df_eicu[df_eicu$unittype=='CTICU',]
df_MICU<-df_eicu[df_eicu$unittype=='MICU',]
df_MSICU<-df_eicu[df_eicu$unittype=='Med-Surg ICU',]
df_CCU_CTICU<-df_eicu[df_eicu$unittype=='CCU-CTICU',]
df_NICU <-df_eicu[df_eicu$unittype=='Neuro ICU',]
df_CCU<-df_eicu[df_eicu$unittype=='Cardiac ICU',]
df_CSICU<-df_eicu[df_eicu$unittype=='CSICU',]

gamfitMed_eICU_SICU <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_SICU, family = binomial)
gamfitMed_eICU_TSICU <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CTICU, family = binomial)
gamfitMed_eICU_MICU <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MICU, family = binomial)
gamfitMed_eICU_CSRU <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MSICU, family = binomial)
gamfitMed_eICU_CCU_CTICU <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU_CTICU, family = binomial)
gamfitMed_eICU_NICU <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_NICU, family = binomial)
gamfitMed_eICU_CCU <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU, family = binomial)
gamfitMed_eICU_CSICU <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CSICU, family = binomial)


gamfitMed_eICU_SICU_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_SICU, family = binomial)
gamfitMed_eICU_TSICU_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CTICU, family = binomial)
gamfitMed_eICU_MICU_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MICU, family = binomial)
gamfitMed_eICU_CSRU_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MSICU, family = binomial)
gamfitMed_eICU_CCU_CTICU_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU_CTICU, family = binomial)
gamfitMed_eICU_NICU_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_NICU, family = binomial)
gamfitMed_eICU_CCU_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU, family = binomial)
gamfitMed_eICU_CSICU_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CSICU, family = binomial)

gamfitMed_eICU_SICU_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_SICU, family = binomial)
gamfitMed_eICU_TSICU_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CTICU, family = binomial)
gamfitMed_eICU_MICU_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MICU, family = binomial)
gamfitMed_eICU_CSRU_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MSICU, family = binomial)
gamfitMed_eICU_CCU_CTICU_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU_CTICU, family = binomial)
gamfitMed_eICU_NICU_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_NICU, family = binomial)
gamfitMed_eICU_CCU_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU, family = binomial)
gamfitMed_eICU_CSICU_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CSICU, family = binomial)


pdf(file = "figure/subgroup/ICUType_eICU_SBP24_Hosp_ICU_AKI.pdf", width = plot_width*4,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,8), cex = 1)
x_range = c(90, 140)
# hospital mortality
gam_plotMed(gamfitMed_eICU_SICU, main = "eICU SICU", xRange = x_range,
                    yRange = c(0, 0.4), label = "A1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_SICU_icu, main = "eICU SICU", xRange = x_range,
                    yRange = c(0, .40), label = "B1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_SICU_aki, main = "eICU SICU", xRange = x_range,
                    yRange = c(0,1),label = "C1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_TSICU, main = "eICU CTICU", xRange = x_range,
                    yRange = c(0, .40),label = "A2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_TSICU_icu, main = "eICU CTICU", xRange = x_range,
                    yRange = c(0, .40), label = "B2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_TSICU_aki, main = "eICU CTICU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_MICU, main = "eICU MICU", xRange = x_range,
                    yRange = c(0, 0.4), label = "A3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_MICU_icu, main = "eICU MICU", xRange = x_range,
                    yRange = c(0, 0.4), label = "B3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_MICU_aki, main = "eICU MICU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_CSRU, main = "eICU Med-Surg ICU", xRange = x_range,
                    yRange = c(0, .40),label = "A4", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_CSRU_icu, main = "eICU Med-Surg ICU", xRange = x_range,
                    yRange = c(0, .40), label = "B4", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_CSRU_aki, main = "eICU Med-Surg ICU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C4", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_CCU_CTICU, main = "eICU CCU-CTICU", xRange = x_range,
                    yRange = c(0,0.4), label = "A5", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_CCU_CTICU_icu, main = "eICU CCU-CTICU", xRange = x_range,
                    yRange = c(0, 0.4), label = "B5", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_CCU_CTICU_aki, main = "eICU CCU-CTICU", xRange = x_range,
                    yRange = c(0, 1.0),label = "C5", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_NICU, main = "eICU NICU", xRange = x_range,
                    yRange = c(0,0.4),label = "A6", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_NICU_icu, main = "eICU NICU", xRange = x_range,
                    yRange = c(0,0.4), label = "B6", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_NICU_aki, main = "eICU NICU", xRange = x_range,
                    yRange = c(0,1),label = "C6", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")
gam_plotMed(gamfitMed_eICU_CCU, main = "eICU CCU", xRange = x_range,
                    yRange = c(0,1),label = "A7", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_CCU_icu, main = "eICU CCU", xRange = x_range,
                    yRange = c(0,1), label = "B7", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_CCU_aki, main = "eICU CCU", xRange = x_range,
                    yRange = c(0,1),label = "C7", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_CSICU, main = "eICU CSICU", xRange = x_range,
                    yRange = c(0,0.4),label = "A8", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_CSICU_icu, main = "eICU CSICU", xRange = x_range,
                    yRange = c(0,0.4), label = "B8", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_CSICU_aki, main = "eICU CSICU", xRange = x_range,
                    yRange = c(0,1),label = "C8", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

dev.off()

png 
  2

In [168]:
df_SICU <-df_eicu[df_eicu$unittype=='SICU',]
df_CTICU <-df_eicu[df_eicu$unittype=='CTICU',]
df_MICU<-df_eicu[df_eicu$unittype=='MICU',]
df_MSICU<-df_eicu[df_eicu$unittype=='Med-Surg ICU',]
df_CCU_CTICU<-df_eicu[df_eicu$unittype=='CCU-CTICU',]
df_NICU <-df_eicu[df_eicu$unittype=='Neuro ICU',]
df_CCU<-df_eicu[df_eicu$unittype=='Cardiac ICU',]
df_CSICU<-df_eicu[df_eicu$unittype=='CSICU',]

gamfitMed_eICU_SICU <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_SICU, family = binomial)
gamfitMed_eICU_TSICU <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CTICU, family = binomial)
gamfitMed_eICU_MICU <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MICU, family = binomial)
gamfitMed_eICU_CSRU <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MSICU, family = binomial)
gamfitMed_eICU_CCU_CTICU <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU_CTICU, family = binomial)
gamfitMed_eICU_NICU <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_NICU, family = binomial)
gamfitMed_eICU_CCU <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU, family = binomial)
gamfitMed_eICU_CSICU <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CSICU, family = binomial)


gamfitMed_eICU_SICU_icu <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_SICU, family = binomial)
gamfitMed_eICU_TSICU_icu <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CTICU, family = binomial)
gamfitMed_eICU_MICU_icu <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MICU, family = binomial)
gamfitMed_eICU_CSRU_icu <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MSICU, family = binomial)
gamfitMed_eICU_CCU_CTICU_icu <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU_CTICU, family = binomial)
gamfitMed_eICU_NICU_icu <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_NICU, family = binomial)
gamfitMed_eICU_CCU_icu <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU, family = binomial)
gamfitMed_eICU_CSICU_icu <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CSICU, family = binomial)

gamfitMed_eICU_SICU_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_SICU, family = binomial)
gamfitMed_eICU_TSICU_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CTICU, family = binomial)
gamfitMed_eICU_MICU_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MICU, family = binomial)
gamfitMed_eICU_CSRU_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MSICU, family = binomial)
gamfitMed_eICU_CCU_CTICU_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU_CTICU, family = binomial)
gamfitMed_eICU_NICU_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_NICU, family = binomial)
gamfitMed_eICU_CCU_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU, family = binomial)
gamfitMed_eICU_CSICU_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CSICU, family = binomial)


pdf(file = "figure/subgroup/ICUType_eICU_SBP48_Hosp_ICU_AKI.pdf", width = plot_width*4,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,8), cex = 1)
x_range = c(90, 140)
# hospital mortality
gam_plotMed(gamfitMed_eICU_SICU, main = "eICU SICU", xRange = x_range,
                    yRange = c(0, 0.4), label = "A1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_SICU_icu, main = "eICU SICU", xRange = x_range,
                    yRange = c(0, .40), label = "B1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_SICU_aki, main = "eICU SICU", xRange = x_range,
                    yRange = c(0,1),label = "C1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_TSICU, main = "eICU CTICU", xRange = x_range,
                    yRange = c(0, .40),label = "A2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_TSICU_icu, main = "eICU CTICU", xRange = x_range,
                    yRange = c(0, .40), label = "B2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_TSICU_aki, main = "eICU CTICU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_MICU, main = "eICU MICU", xRange = x_range,
                    yRange = c(0, 0.4), label = "A3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_MICU_icu, main = "eICU MICU", xRange = x_range,
                    yRange = c(0, 0.4), label = "B3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_MICU_aki, main = "eICU MICU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_CSRU, main = "eICU Med-Surg ICU", xRange = x_range,
                    yRange = c(0, .40),label = "A4", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_CSRU_icu, main = "eICU Med-Surg ICU", xRange = x_range,
                    yRange = c(0, .40), label = "B4", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_CSRU_aki, main = "eICU Med-Surg ICU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C4", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_CCU_CTICU, main = "eICU CCU-CTICU", xRange = x_range,
                    yRange = c(0,0.4), label = "A5", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_CCU_CTICU_icu, main = "eICU CCU-CTICU", xRange = x_range,
                    yRange = c(0, 0.4), label = "B5", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_CCU_CTICU_aki, main = "eICU CCU-CTICU", xRange = x_range,
                    yRange = c(0, 1.0),label = "C5", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_NICU, main = "eICU NICU", xRange = x_range,
                    yRange = c(0,0.4),label = "A6", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_NICU_icu, main = "eICU NICU", xRange = x_range,
                    yRange = c(0,0.4), label = "B6", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_NICU_aki, main = "eICU NICU", xRange = x_range,
                    yRange = c(0,1),label = "C6", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")
gam_plotMed(gamfitMed_eICU_CCU, main = "eICU CCU", xRange = x_range,
                    yRange = c(0,1),label = "A7", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_CCU_icu, main = "eICU CCU", xRange = x_range,
                    yRange = c(0,1), label = "B7", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_CCU_aki, main = "eICU CCU", xRange = x_range,
                    yRange = c(0,1),label = "C7", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_CSICU, main = "eICU CSICU", xRange = x_range,
                    yRange = c(0,0.4),label = "A8", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_CSICU_icu, main = "eICU CSICU", xRange = x_range,
                    yRange = c(0,0.4), label = "B8", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_CSICU_aki, main = "eICU CSICU", xRange = x_range,
                    yRange = c(0,1),label = "C8", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

dev.off()

png 
  2

In [169]:
df_SICU <-df_eicu[df_eicu$unittype=='SICU',]
df_CTICU <-df_eicu[df_eicu$unittype=='CTICU',]
df_MICU<-df_eicu[df_eicu$unittype=='MICU',]
df_MSICU<-df_eicu[df_eicu$unittype=='Med-Surg ICU',]
df_CCU_CTICU<-df_eicu[df_eicu$unittype=='CCU-CTICU',]
df_NICU <-df_eicu[df_eicu$unittype=='Neuro ICU',]
df_CCU<-df_eicu[df_eicu$unittype=='Cardiac ICU',]
df_CSICU<-df_eicu[df_eicu$unittype=='CSICU',]

gamfitMed_eICU_SICU <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_SICU, family = binomial)
gamfitMed_eICU_TSICU <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CTICU, family = binomial)
gamfitMed_eICU_MICU <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MICU, family = binomial)
gamfitMed_eICU_CSRU <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MSICU, family = binomial)
gamfitMed_eICU_CCU_CTICU <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU_CTICU, family = binomial)
gamfitMed_eICU_NICU <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_NICU, family = binomial)
gamfitMed_eICU_CCU <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU, family = binomial)
gamfitMed_eICU_CSICU <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CSICU, family = binomial)


gamfitMed_eICU_SICU_icu <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_SICU, family = binomial)
gamfitMed_eICU_TSICU_icu <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CTICU, family = binomial)
gamfitMed_eICU_MICU_icu <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MICU, family = binomial)
gamfitMed_eICU_CSRU_icu <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MSICU, family = binomial)
gamfitMed_eICU_CCU_CTICU_icu <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU_CTICU, family = binomial)
gamfitMed_eICU_NICU_icu <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_NICU, family = binomial)
gamfitMed_eICU_CCU_icu <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU, family = binomial)
gamfitMed_eICU_CSICU_icu <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CSICU, family = binomial)

gamfitMed_eICU_SICU_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_SICU, family = binomial)
gamfitMed_eICU_TSICU_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CTICU, family = binomial)
gamfitMed_eICU_MICU_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MICU, family = binomial)
gamfitMed_eICU_CSRU_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MSICU, family = binomial)
gamfitMed_eICU_CCU_CTICU_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU_CTICU, family = binomial)
gamfitMed_eICU_NICU_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_NICU, family = binomial)
gamfitMed_eICU_CCU_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU, family = binomial)
gamfitMed_eICU_CSICU_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CSICU, family = binomial)


pdf(file = "figure/subgroup/ICUType_eICU_SBP72_Hosp_ICU_AKI.pdf", width = plot_width*4,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,8), cex = 1)
x_range = c(90, 140)
# hospital mortality
gam_plotMed(gamfitMed_eICU_SICU, main = "eICU SICU", xRange = x_range,
                    yRange = c(0, 0.4), label = "A1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_SICU_icu, main = "eICU SICU", xRange = x_range,
                    yRange = c(0, .40), label = "B1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_SICU_aki, main = "eICU SICU", xRange = x_range,
                    yRange = c(0,1),label = "C1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_TSICU, main = "eICU CTICU", xRange = x_range,
                    yRange = c(0, .40),label = "A2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_TSICU_icu, main = "eICU CTICU", xRange = x_range,
                    yRange = c(0, .40), label = "B2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_TSICU_aki, main = "eICU CTICU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_MICU, main = "eICU MICU", xRange = x_range,
                    yRange = c(0, 0.4), label = "A3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_MICU_icu, main = "eICU MICU", xRange = x_range,
                    yRange = c(0, 0.4), label = "B3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_MICU_aki, main = "eICU MICU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_CSRU, main = "eICU Med-Surg ICU", xRange = x_range,
                    yRange = c(0, .40),label = "A4", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_CSRU_icu, main = "eICU  Med-Surg ICU", xRange = x_range,
                    yRange = c(0, .40), label = "B4", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_CSRU_aki, main = "eICU  Med-Surg ICU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C4", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_CCU_CTICU, main = "eICU CCU-CTICU", xRange = x_range,
                    yRange = c(0,0.4), label = "A5", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_CCU_CTICU_icu, main = "eICU CCU-CTICU", xRange = x_range,
                    yRange = c(0, 0.4), label = "B5", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_CCU_CTICU_aki, main = "eICU CCU-CTICU", xRange = x_range,
                    yRange = c(0, 1.0),label = "C5", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_NICU, main = "eICU NICU", xRange = x_range,
                    yRange = c(0,0.4),label = "A6", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_NICU_icu, main = "eICU NICU", xRange = x_range,
                    yRange = c(0,0.4), label = "B6", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_NICU_aki, main = "eICU NICU", xRange = x_range,
                    yRange = c(0,1),label = "C6", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")
gam_plotMed(gamfitMed_eICU_CCU, main = "eICU CCU", xRange = x_range,
                    yRange = c(0,1),label = "A7", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_CCU_icu, main = "eICU CCU", xRange = x_range,
                    yRange = c(0,1), label = "B7", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_CCU_aki, main = "eICU CCU", xRange = x_range,
                    yRange = c(0,1),label = "C7", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_CSICU, main = "eICU CSICU", xRange = x_range,
                    yRange = c(0,0.4),label = "A8", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_CSICU_icu, main = "eICU CSICU", xRange = x_range,
                    yRange = c(0,0.4), label = "B8", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_CSICU_aki, main = "eICU CSICU", xRange = x_range,
                    yRange = c(0,1),label = "C8", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

dev.off()

png 
  2

### eICU DBP 24h, 48h, 72h

In [170]:
df_SICU <-df_eicu[df_eicu$unittype=='SICU',]
df_CTICU <-df_eicu[df_eicu$unittype=='CTICU',]
df_MICU<-df_eicu[df_eicu$unittype=='MICU',]
df_MSICU<-df_eicu[df_eicu$unittype=='Med-Surg ICU',]
df_CCU_CTICU<-df_eicu[df_eicu$unittype=='CCU-CTICU',]
df_NICU <-df_eicu[df_eicu$unittype=='Neuro ICU',]
df_CCU<-df_eicu[df_eicu$unittype=='Cardiac ICU',]
df_CSICU<-df_eicu[df_eicu$unittype=='CSICU',]

gamfitMed_eICU_SICU <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_SICU, family = binomial)
gamfitMed_eICU_TSICU <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CTICU, family = binomial)
gamfitMed_eICU_MICU <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MICU, family = binomial)
gamfitMed_eICU_CSRU <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MSICU, family = binomial)
gamfitMed_eICU_CCU_CTICU <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU_CTICU, family = binomial)
gamfitMed_eICU_NICU <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_NICU, family = binomial)
gamfitMed_eICU_CCU <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU, family = binomial)
gamfitMed_eICU_CSICU <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CSICU, family = binomial)


gamfitMed_eICU_SICU_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_SICU, family = binomial)
gamfitMed_eICU_TSICU_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CTICU, family = binomial)
gamfitMed_eICU_MICU_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MICU, family = binomial)
gamfitMed_eICU_CSRU_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MSICU, family = binomial)
gamfitMed_eICU_CCU_CTICU_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU_CTICU, family = binomial)
gamfitMed_eICU_NICU_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_NICU, family = binomial)
gamfitMed_eICU_CCU_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU, family = binomial)
gamfitMed_eICU_CSICU_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CSICU, family = binomial)

gamfitMed_eICU_SICU_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_SICU, family = binomial)
gamfitMed_eICU_TSICU_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CTICU, family = binomial)
gamfitMed_eICU_MICU_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MICU, family = binomial)
gamfitMed_eICU_CSRU_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MSICU, family = binomial)
gamfitMed_eICU_CCU_CTICU_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU_CTICU, family = binomial)
gamfitMed_eICU_NICU_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_NICU, family = binomial)
gamfitMed_eICU_CCU_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU, family = binomial)
gamfitMed_eICU_CSICU_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CSICU, family = binomial)


pdf(file = "figure/subgroup/ICUType_eICU_DBP24_Hosp_ICU_AKI.pdf", width = plot_width*4,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,8), cex = 1)
x_range = c(35, 75)
# hospital mortality
gam_plotMed(gamfitMed_eICU_SICU, main = "eICU SICU", xRange = x_range,
                    yRange = c(0, 0.4), label = "A1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_SICU_icu, main = "eICU SICU", xRange = x_range,
                    yRange = c(0, .40), label = "B1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_SICU_aki, main = "eICU SICU", xRange = x_range,
                    yRange = c(0,1),label = "C1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_TSICU, main = "eICU CTICU", xRange = x_range,
                    yRange = c(0, .40),label = "A2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_TSICU_icu, main = "eICU CTICU", xRange = x_range,
                    yRange = c(0, .40), label = "B2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_TSICU_aki, main = "eICU CTICU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_MICU, main = "eICU MICU", xRange = x_range,
                    yRange = c(0, 0.4), label = "A3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_MICU_icu, main = "eICU MICU", xRange = x_range,
                    yRange = c(0, 0.4), label = "B3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_MICU_aki, main = "eICU MICU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_CSRU, main = "eICU Med-Surg ICU", xRange = x_range,
                    yRange = c(0, .40),label = "A4", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_CSRU_icu, main = "eICU Med-Surg ICU", xRange = x_range,
                    yRange = c(0, .40), label = "B4", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_CSRU_aki, main = "eICU Med-Surg ICU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C4", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_CCU_CTICU, main = "eICU CCU-CTICU", xRange = x_range,
                    yRange = c(0,0.4), label = "A5", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_CCU_CTICU_icu, main = "eICU CCU-CTICU", xRange = x_range,
                    yRange = c(0, 0.4), label = "B5", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_CCU_CTICU_aki, main = "eICU CCU-CTICU", xRange = x_range,
                    yRange = c(0, 1.0),label = "C5", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_NICU, main = "eICU NICU", xRange = x_range,
                    yRange = c(0,0.4),label = "A6", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_NICU_icu, main = "eICU NICU", xRange = x_range,
                    yRange = c(0,0.4), label = "B6", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_NICU_aki, main = "eICU NICU", xRange = x_range,
                    yRange = c(0,1),label = "C6", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")
gam_plotMed(gamfitMed_eICU_CCU, main = "eICU CCU", xRange = x_range,
                    yRange = c(0,1),label = "A7", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_CCU_icu, main = "eICU CCU", xRange = x_range,
                    yRange = c(0,1), label = "B7", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_CCU_aki, main = "eICU CCU", xRange = x_range,
                    yRange = c(0,1),label = "C7", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_CSICU, main = "eICU CSICU", xRange = x_range,
                    yRange = c(0,0.4),label = "A8", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_CSICU_icu, main = "eICU CSICU", xRange = x_range,
                    yRange = c(0,0.4), label = "B8", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_CSICU_aki, main = "eICU CSICU", xRange = x_range,
                    yRange = c(0,1),label = "C8", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

dev.off()

Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”

png 
  2

In [171]:
df_SICU <-df_eicu[df_eicu$unittype=='SICU',]
df_CTICU <-df_eicu[df_eicu$unittype=='CTICU',]
df_MICU<-df_eicu[df_eicu$unittype=='MICU',]
df_MSICU<-df_eicu[df_eicu$unittype=='Med-Surg ICU',]
df_CCU_CTICU<-df_eicu[df_eicu$unittype=='CCU-CTICU',]
df_NICU <-df_eicu[df_eicu$unittype=='Neuro ICU',]
df_CCU<-df_eicu[df_eicu$unittype=='Cardiac ICU',]
df_CSICU<-df_eicu[df_eicu$unittype=='CSICU',]

gamfitMed_eICU_SICU <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_SICU, family = binomial)
gamfitMed_eICU_TSICU <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CTICU, family = binomial)
gamfitMed_eICU_MICU <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MICU, family = binomial)
gamfitMed_eICU_CSRU <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MSICU, family = binomial)
gamfitMed_eICU_CCU_CTICU <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU_CTICU, family = binomial)
gamfitMed_eICU_NICU <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_NICU, family = binomial)
gamfitMed_eICU_CCU <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU, family = binomial)
gamfitMed_eICU_CSICU <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CSICU, family = binomial)


gamfitMed_eICU_SICU_icu <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_SICU, family = binomial)
gamfitMed_eICU_TSICU_icu <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CTICU, family = binomial)
gamfitMed_eICU_MICU_icu <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MICU, family = binomial)
gamfitMed_eICU_CSRU_icu <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MSICU, family = binomial)
gamfitMed_eICU_CCU_CTICU_icu <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU_CTICU, family = binomial)
gamfitMed_eICU_NICU_icu <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_NICU, family = binomial)
gamfitMed_eICU_CCU_icu <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU, family = binomial)
gamfitMed_eICU_CSICU_icu <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CSICU, family = binomial)

gamfitMed_eICU_SICU_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_SICU, family = binomial)
gamfitMed_eICU_TSICU_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CTICU, family = binomial)
gamfitMed_eICU_MICU_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MICU, family = binomial)
gamfitMed_eICU_CSRU_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MSICU, family = binomial)
gamfitMed_eICU_CCU_CTICU_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU_CTICU, family = binomial)
gamfitMed_eICU_NICU_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_NICU, family = binomial)
gamfitMed_eICU_CCU_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU, family = binomial)
gamfitMed_eICU_CSICU_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CSICU, family = binomial)


pdf(file = "figure/subgroup/ICUType_eICU_DBP48_Hosp_ICU_AKI.pdf", width = plot_width*4,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,8), cex = 1)
x_range = c(35, 75)
# hospital mortality
gam_plotMed(gamfitMed_eICU_SICU, main = "eICU SICU", xRange = x_range,
                    yRange = c(0, 0.4), label = "A1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_SICU_icu, main = "eICU SICU", xRange = x_range,
                    yRange = c(0, .40), label = "B1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_SICU_aki, main = "eICU SICU", xRange = x_range,
                    yRange = c(0,1),label = "C1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_TSICU, main = "eICU CTICU", xRange = x_range,
                    yRange = c(0, .40),label = "A2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_TSICU_icu, main = "eICU CTICU", xRange = x_range,
                    yRange = c(0, .40), label = "B2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_TSICU_aki, main = "eICU CTICU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_MICU, main = "eICU MICU", xRange = x_range,
                    yRange = c(0, 0.4), label = "A3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_MICU_icu, main = "eICU MICU", xRange = x_range,
                    yRange = c(0, 0.4), label = "B3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_MICU_aki, main = "eICU MICU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_CSRU, main = "eICU Med-Surg ICU", xRange = x_range,
                    yRange = c(0, .40),label = "A4", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_CSRU_icu, main = "eICU Med-Surg ICU", xRange = x_range,
                    yRange = c(0, .40), label = "B4", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_CSRU_aki, main = "eICU Med-Surg ICU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C4", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_CCU_CTICU, main = "eICU CCU-CTICU", xRange = x_range,
                    yRange = c(0,0.4), label = "A5", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_CCU_CTICU_icu, main = "eICU CCU-CTICU", xRange = x_range,
                    yRange = c(0, 0.4), label = "B5", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_CCU_CTICU_aki, main = "eICU CCU-CTICU", xRange = x_range,
                    yRange = c(0, 1.0),label = "C5", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_NICU, main = "eICU NICU", xRange = x_range,
                    yRange = c(0,0.4),label = "A6", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_NICU_icu, main = "eICU NICU", xRange = x_range,
                    yRange = c(0,0.4), label = "B6", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_NICU_aki, main = "eICU NICU", xRange = x_range,
                    yRange = c(0,1),label = "C6", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")
gam_plotMed(gamfitMed_eICU_CCU, main = "eICU CCU", xRange = x_range,
                    yRange = c(0,1),label = "A7", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_CCU_icu, main = "eICU CCU", xRange = x_range,
                    yRange = c(0,1), label = "B7", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_CCU_aki, main = "eICU CCU", xRange = x_range,
                    yRange = c(0,1),label = "C7", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_CSICU, main = "eICU CSICU", xRange = x_range,
                    yRange = c(0,0.4),label = "A8", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_CSICU_icu, main = "eICU CSICU", xRange = x_range,
                    yRange = c(0,0.4), label = "B8", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_CSICU_aki, main = "eICU CSICU", xRange = x_range,
                    yRange = c(0,1),label = "C8", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

dev.off()

Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”

png 
  2

In [172]:
df_SICU <-df_eicu[df_eicu$unittype=='SICU',]
df_CTICU <-df_eicu[df_eicu$unittype=='CTICU',]
df_MICU<-df_eicu[df_eicu$unittype=='MICU',]
df_MSICU<-df_eicu[df_eicu$unittype=='Med-Surg ICU',]
df_CCU_CTICU<-df_eicu[df_eicu$unittype=='CCU-CTICU',]
df_NICU <-df_eicu[df_eicu$unittype=='Neuro ICU',]
df_CCU<-df_eicu[df_eicu$unittype=='Cardiac ICU',]
df_CSICU<-df_eicu[df_eicu$unittype=='CSICU',]

gamfitMed_eICU_SICU <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_SICU, family = binomial)
gamfitMed_eICU_TSICU <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CTICU, family = binomial)
gamfitMed_eICU_MICU <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MICU, family = binomial)
gamfitMed_eICU_CSRU <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MSICU, family = binomial)
gamfitMed_eICU_CCU_CTICU <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU_CTICU, family = binomial)
gamfitMed_eICU_NICU <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_NICU, family = binomial)
gamfitMed_eICU_CCU <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU, family = binomial)
gamfitMed_eICU_CSICU <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CSICU, family = binomial)


gamfitMed_eICU_SICU_icu <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_SICU, family = binomial)
gamfitMed_eICU_TSICU_icu <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CTICU, family = binomial)
gamfitMed_eICU_MICU_icu <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MICU, family = binomial)
gamfitMed_eICU_CSRU_icu <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MSICU, family = binomial)
gamfitMed_eICU_CCU_CTICU_icu <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU_CTICU, family = binomial)
gamfitMed_eICU_NICU_icu <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_NICU, family = binomial)
gamfitMed_eICU_CCU_icu <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU, family = binomial)
gamfitMed_eICU_CSICU_icu <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CSICU, family = binomial)

gamfitMed_eICU_SICU_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_SICU, family = binomial)
gamfitMed_eICU_TSICU_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CTICU, family = binomial)
gamfitMed_eICU_MICU_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MICU, family = binomial)
gamfitMed_eICU_CSRU_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_MSICU, family = binomial)
gamfitMed_eICU_CCU_CTICU_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU_CTICU, family = binomial)
gamfitMed_eICU_NICU_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_NICU, family = binomial)
gamfitMed_eICU_CCU_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CCU, family = binomial)
gamfitMed_eICU_CSICU_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_CSICU, family = binomial)


pdf(file = "figure/subgroup/ICUType_eICU_DBP72_Hosp_ICU_AKI.pdf", width = plot_width*4,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,8), cex = 1)
x_range = c(35, 75)
# hospital mortality
gam_plotMed(gamfitMed_eICU_SICU, main = "eICU SICU", xRange = x_range,
                    yRange = c(0, 0.4), label = "A1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_SICU_icu, main = "eICU SICU", xRange = x_range,
                    yRange = c(0, .40), label = "B1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_SICU_aki, main = "eICU SICU", xRange = x_range,
                    yRange = c(0,1),label = "C1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_TSICU, main = "eICU CTICU", xRange = x_range,
                    yRange = c(0, .40),label = "A2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_TSICU_icu, main = "eICU CTICU", xRange = x_range,
                    yRange = c(0, .40), label = "B2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_TSICU_aki, main = "eICU CTICU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_MICU, main = "eICU MICU", xRange = x_range,
                    yRange = c(0, 0.4), label = "A3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_MICU_icu, main = "eICU MICU", xRange = x_range,
                    yRange = c(0, 0.4), label = "B3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_MICU_aki, main = "eICU MICU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_CSRU, main = "eICU Med-Surg ICU", xRange = x_range,
                    yRange = c(0, .40),label = "A4", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_CSRU_icu, main = "eICU Med-Surg ICU", xRange = x_range,
                    yRange = c(0, .40), label = "B4", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_CSRU_aki, main = "eICU Med-Surg ICU", xRange = x_range,
                    yRange = c(0.0, 1.0),label = "C4", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_CCU_CTICU, main = "eICU CCU-CTICU", xRange = x_range,
                    yRange = c(0,0.4), label = "A5", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_CCU_CTICU_icu, main = "eICU CCU-CTICU", xRange = x_range,
                    yRange = c(0, 0.4), label = "B5", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_CCU_CTICU_aki, main = "eICU CCU-CTICU", xRange = x_range,
                    yRange = c(0, 1.0),label = "C5", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_NICU, main = "eICU NICU", xRange = x_range,
                    yRange = c(0,0.4),label = "A6", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_NICU_icu, main = "eICU NICU", xRange = x_range,
                    yRange = c(0,0.4), label = "B6", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_NICU_aki, main = "eICU NICU", xRange = x_range,
                    yRange = c(0,1),label = "C6", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")
gam_plotMed(gamfitMed_eICU_CCU, main = "eICU CCU", xRange = x_range,
                    yRange = c(0,1),label = "A7", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_CCU_icu, main = "eICU CCU", xRange = x_range,
                    yRange = c(0,1), label = "B7", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_CCU_aki, main = "eICU CCU", xRange = x_range,
                    yRange = c(0,1),label = "C7", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_CSICU, main = "eICU CSICU", xRange = x_range,
                    yRange = c(0,0.4),label = "A8", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_CSICU_icu, main = "eICU CSICU", xRange = x_range,
                    yRange = c(0,0.4), label = "B8", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_CSICU_aki, main = "eICU CSICU", xRange = x_range,
                    yRange = c(0,1),label = "C8", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

dev.off()

Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”

png 
  2

## with vs. without medical history of AF, DM, CHF, IHD, CKD or stroke.

In [77]:
print(colnames(df_mimic))

 [1] "icustay_id"             "age"                    "gender"                
 [4] "ethnicity"              "unittype"               "icu_mortality"         
 [7] "hospital_expire_flag"   "sofatotal"              "vasso_duration_24h"    
[10] "vasso_duration_48h"     "vasso_duration_72h"     "hypertension"          
[13] "chf"                    "diabetes"               "afib"                  
[16] "ckd"                    "cld"                    "copd"                  
[19] "stroke"                 "ihd"                    "cancer"                
[22] "sepsis"                 "mechanical_ventilation" "ibp_systolic_24h"      
[25] "ibp_systolic_48h"       "ibp_systolic_72h"       "ibp_diastolic_24h"     
[28] "ibp_diastolic_48h"      "ibp_diastolic_72h"      "ibp_mean_24h"          
[31] "ibp_mean_48h"           "ibp_mean_72h"           "bmi"                   
[34] "AKI_7d"                 "mortality_7d"           "vasso_use"             


### Atrial fibrillation

In [101]:
# MIMIC MAP
df_sepsis1 <-df_mimic[df_mimic$afib==1,]
df_sepsis0 <-df_mimic[df_mimic$afib==0,]


# MIMIC MAP
# hospital mortality
gamfitMed_MIMIC_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_MIMIC_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_MIMIC_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/Afib_MIMIC_MAP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(55, 95)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_hosp, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_hosp, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_hosp, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_hosp, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_hosp, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_hosp, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# icu mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_icu, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_icu, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_icu, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_icu, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_icu, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_icu, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_24h_aki, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_48h_aki, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_72h_aki, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_48h_aki, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_24h_aki, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_72h_aki, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

dev.off()

png 
  2

In [102]:
# MIMIC SBP
df_sepsis1 <-df_mimic[df_mimic$afib==1,]
df_sepsis0 <-df_mimic[df_mimic$afib==0,]

# MIMIC SBP
# hospital mortality
gamfitMed_MIMIC_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_MIMIC_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_MIMIC_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/Afib_MIMIC_SBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(90, 140)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_hosp, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_hosp, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_hosp, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_hosp, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_hosp, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_hosp, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# icu mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_icu, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_icu, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_icu, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_icu, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_icu, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_icu, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_24h_aki, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_48h_aki, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_72h_aki, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_48h_aki, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_24h_aki, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_72h_aki, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

dev.off()

png 
  2

In [103]:
# MIMIC DBP
df_sepsis1 <-df_mimic[df_mimic$afib==1,]
df_sepsis0 <-df_mimic[df_mimic$afib==0,]

# hospital mortality
gamfitMed_MIMIC_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_MIMIC_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_MIMIC_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/Afib_MIMIC_DBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(35, 75)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_hosp, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_hosp, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_hosp, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_hosp, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_hosp, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_hosp, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# icu mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_icu, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_icu, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_icu, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_icu, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_icu, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_icu, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_24h_aki, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_48h_aki, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_72h_aki, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_48h_aki, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_24h_aki, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_72h_aki, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

dev.off()

png 
  2

In [104]:
df_sepsis1 <-df_eicu[df_eicu$afib==1,]
df_sepsis0 <-df_eicu[df_eicu$afib==0,]

# eICU MAP
# hospital mortality
gamfitMed_eICU_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_eICU_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_eICU_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/Afib_eICU_MAP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(55, 95)
# hospital mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_hosp, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_hosp, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_hosp, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_hosp, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_hosp, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_hosp, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# icu mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_icu, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_icu, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_icu, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_icu, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_icu, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_icu, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_24h_aki, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_48h_aki, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_72h_aki, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_48h_aki, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_24h_aki, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_72h_aki, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

dev.off()

png 
  2

In [105]:
df_sepsis1 <-df_eicu[df_eicu$afib==1,]
df_sepsis0 <-df_eicu[df_eicu$afib==0,]

# eICU SBP
# hospital mortality
gamfitMed_eICU_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_eICU_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_eICU_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/Afib_eICU_SBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(90, 140)
# hospital mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_hosp, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_hosp, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_hosp, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_hosp, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_hosp, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_hosp, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# icu mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_icu, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_icu, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_icu, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_icu, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_icu, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_icu, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_24h_aki, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_48h_aki, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_72h_aki, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_48h_aki, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_24h_aki, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_72h_aki, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

dev.off()

png 
  2

In [106]:
df_sepsis1 <-df_eicu[df_eicu$afib==1,]
df_sepsis0 <-df_eicu[df_eicu$afib==0,]

# eICU DBP
# hospital mortality
gamfitMed_eICU_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_eICU_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_eICU_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/Afib_eICU_DBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(35, 75)
# hospital mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_hosp, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_hosp, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_hosp, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_hosp, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_hosp, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_hosp, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# icu mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_icu, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_icu, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_icu, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_icu, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_icu, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_icu, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_24h_aki, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_48h_aki, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_72h_aki, main = "Atrial fibrillation = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_48h_aki, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_24h_aki, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_72h_aki, main = "Atrial fibrillation = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

dev.off()

Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”

png 
  2

### Congestive heart failure 

In [109]:
df_sepsis1 <-df_mimic[df_mimic$chf==1,]
df_sepsis0 <-df_mimic[df_mimic$chf==0,]
# MIMIC MAP
# MIMIC MAP
# hospital mortality
gamfitMed_MIMIC_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_MIMIC_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_MIMIC_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/CHF_MIMIC_MAP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(55, 95)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_hosp, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_hosp, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_hosp, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_hosp, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_hosp, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_hosp, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# icu mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_icu, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_icu, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_icu, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_icu, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_icu, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_icu, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_24h_aki, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_48h_aki, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_72h_aki, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_48h_aki, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_24h_aki, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_72h_aki, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

dev.off()

png 
  2

In [110]:
df_sepsis1 <-df_mimic[df_mimic$chf==1,]
df_sepsis0 <-df_mimic[df_mimic$chf==0,]
# MIMIC SBP
# MIMIC SBP
# hospital mortality
gamfitMed_MIMIC_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_MIMIC_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_MIMIC_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/CHF_MIMIC_SBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(90, 140)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_hosp, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_hosp, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_hosp, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_hosp, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_hosp, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_hosp, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# icu mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_icu, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_icu, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_icu, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_icu, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_icu, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_icu, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_24h_aki, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_48h_aki, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_72h_aki, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_48h_aki, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_24h_aki, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_72h_aki, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

dev.off()

png 
  2

In [111]:
df_sepsis1 <-df_mimic[df_mimic$chf==1,]
df_sepsis0 <-df_mimic[df_mimic$chf==0,]
# MIMIC DBP
# hospital mortality
gamfitMed_MIMIC_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_MIMIC_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_MIMIC_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/CHF_MIMIC_DBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(35, 75)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_hosp, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_hosp, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_hosp, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_hosp, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_hosp, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_hosp, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# icu mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_icu, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_icu, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_icu, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_icu, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_icu, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_icu, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_24h_aki, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_48h_aki, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_72h_aki, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_48h_aki, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_24h_aki, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_72h_aki, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

dev.off()

png 
  2

In [112]:
df_sepsis1 <-df_eicu[df_eicu$chf==1,]
df_sepsis0 <-df_eicu[df_eicu$chf==0,]
# eICU MAP
# hospital mortality
gamfitMed_eICU_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_eICU_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_eICU_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/CHF_eICU_MAP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(55, 95)
# hospital mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_hosp, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_hosp, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_hosp, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_hosp, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_hosp, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_hosp, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# icu mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_icu, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_icu, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_icu, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_icu, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_icu, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_icu, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_24h_aki, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_48h_aki, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_72h_aki, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_48h_aki, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_24h_aki, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_72h_aki, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

dev.off()

Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning me

png 
  2

In [113]:
df_sepsis1 <-df_eicu[df_eicu$chf==1,]
df_sepsis0 <-df_eicu[df_eicu$chf==0,]
# eICU SBP
# hospital mortality
gamfitMed_eICU_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_eICU_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_eICU_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/CHF_eICU_SBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(90, 140)
# hospital mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_hosp, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_hosp, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_hosp, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_hosp, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_hosp, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_hosp, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# icu mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_icu, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_icu, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_icu, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_icu, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_icu, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_icu, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_24h_aki, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_48h_aki, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_72h_aki, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_48h_aki, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_24h_aki, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_72h_aki, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

dev.off()

Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”

png 
  2

In [114]:
df_sepsis1 <-df_eicu[df_eicu$chf==1,]
df_sepsis0 <-df_eicu[df_eicu$chf==0,]
# eICU DBP
# hospital mortality
gamfitMed_eICU_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_eICU_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_eICU_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/CHF_eICU_DBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(35, 75)
# hospital mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_hosp, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_hosp, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_hosp, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_hosp, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_hosp, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_hosp, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# icu mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_icu, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_icu, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_icu, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_icu, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_icu, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_icu, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_24h_aki, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_48h_aki, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_72h_aki, main = "Congestive heart failure = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_48h_aki, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_24h_aki, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_72h_aki, main = "Congestive heart failure = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

dev.off()

Warning message in min(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning me

png 
  2

### diabetes

In [115]:
df_sepsis1 <-df_mimic[df_mimic$diabetes==1,]
df_sepsis0 <-df_mimic[df_mimic$diabetes==0,]
# MIMIC MAP
# MIMIC MAP
# hospital mortality
gamfitMed_MIMIC_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_MIMIC_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_MIMIC_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/Diabetes_MIMIC_MAP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(55, 95)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_hosp, main = "Diabetes = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_hosp, main = "Diabetes = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_hosp, main = "Diabetes = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_hosp, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_hosp, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_hosp, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# icu mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_icu, main = "Diabetes = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_icu, main = "Diabetes = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_icu, main = "Diabetes = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_icu, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_icu, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_icu, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_24h_aki, main = "Diabetes = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_48h_aki, main = "Diabetes = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_72h_aki, main = "Diabetes = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_48h_aki, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_24h_aki, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_72h_aki, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

dev.off()

png 
  2

In [116]:
df_sepsis1 <-df_mimic[df_mimic$diabetes==1,]
df_sepsis0 <-df_mimic[df_mimic$diabetes==0,]
# MIMIC SBP
# hospital mortality
gamfitMed_MIMIC_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_MIMIC_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_MIMIC_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/Diabetes_MIMIC_SBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(90, 140)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_hosp, main = "Diabetes = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_hosp, main = "Diabetes = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_hosp, main = "Diabetes = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_hosp, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_hosp, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_hosp, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# icu mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_icu, main = "Diabetes = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_icu, main = "Diabetes = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_icu, main = "Diabetes = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_icu, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_icu, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_icu, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_24h_aki, main = "Diabetes = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_48h_aki, main = "Diabetes = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_72h_aki, main = "Diabetes = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_48h_aki, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_24h_aki, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_72h_aki, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

dev.off()

png 
  2

In [117]:
df_sepsis1 <-df_mimic[df_mimic$diabetes==1,]
df_sepsis0 <-df_mimic[df_mimic$diabetes==0,]
# MIMIC DBP
# hospital mortality
gamfitMed_MIMIC_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_MIMIC_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_MIMIC_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/Diabetes_MIMIC_DBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(35, 75)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_hosp, main = "Diabetes = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_hosp, main = "Diabetes = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_hosp, main = "Diabetes = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_hosp, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_hosp, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_hosp, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# icu mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_icu, main = "Diabetes = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_icu, main = "Diabetes = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_icu, main = "Diabetes = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_icu, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_icu, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_icu, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_24h_aki, main = "Diabetes = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_48h_aki, main = "Diabetes = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_72h_aki, main = "Diabetes = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_48h_aki, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_24h_aki, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_72h_aki, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

dev.off()

png 
  2

In [173]:
df_sepsis1 <-df_eicu[df_eicu$diabetes==1,]
df_sepsis0 <-df_eicu[df_eicu$diabetes==0,]
# eICU MAP
# hospital mortality
gamfitMed_eICU_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_eICU_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_eICU_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/Diabetes_eICU_MAP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(55, 95)
# hospital mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_hosp, main = "Diabetes = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_hosp, main = "Diabetes = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_hosp, main = "Diabetes = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_hosp, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_hosp, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_hosp, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# icu mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_icu, main = "Diabetes = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_icu, main = "Diabetes = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_icu, main = "Diabetes = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_icu, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_icu, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_icu, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_24h_aki, main = "Diabetes = Yes", xRange = x_range,
                    yRange = c(0.0,1.0), label = "A3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_48h_aki, main = "Diabetes = Yes", xRange = x_range,
                    yRange = c(0.0,1.0), label = "C3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_72h_aki, main = "Diabetes = Yes", xRange = x_range,
                    yRange = c(0.0,1.0),label = "E3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_48h_aki, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_24h_aki, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_72h_aki, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

dev.off()

Warning message in newton(lsp = lsp, X = G$X, y = G$y, Eb = G$Eb, UrS = G$UrS, L = G$L, :
“Iteration limit reached without full convergence - check carefully”Warning message in min(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to max; returnin

png 
  2

In [174]:
df_sepsis1 <-df_eicu[df_eicu$diabetes==1,]
df_sepsis0 <-df_eicu[df_eicu$diabetes==0,]
# eICU SBP
# hospital mortality
gamfitMed_eICU_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_eICU_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_eICU_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/Diabetes_eICU_SBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(90, 140)
# hospital mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_hosp, main = "Diabetes = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_hosp, main = "Diabetes = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_hosp, main = "Diabetes = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_hosp, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_hosp, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_hosp, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# icu mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_icu, main = "Diabetes = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_icu, main = "Diabetes = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_icu, main = "Diabetes = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_icu, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_icu, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_icu, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# AKI

y_range = c(0.0,1.0)
gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_24h_aki, main = "Diabetes = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_48h_aki, main = "Diabetes = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_72h_aki, main = "Diabetes = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_48h_aki, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_24h_aki, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_72h_aki, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

dev.off()

Warning message in newton(lsp = lsp, X = G$X, y = G$y, Eb = G$Eb, UrS = G$UrS, L = G$L, :
“Iteration limit reached without full convergence - check carefully”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returnin

png 
  2

In [175]:
df_sepsis1 <-df_eicu[df_eicu$diabetes==1,]
df_sepsis0 <-df_eicu[df_eicu$diabetes==0,]
# eICU DBP
# hospital mortality
gamfitMed_eICU_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_eICU_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_eICU_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/Diabetes_eICU_DBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(35, 75)
# hospital mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_hosp, main = "Diabetes = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_hosp, main = "Diabetes = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_hosp, main = "Diabetes = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_hosp, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_hosp, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_hosp, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# icu mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_icu, main = "Diabetes = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_icu, main = "Diabetes = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_icu, main = "Diabetes = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_icu, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_icu, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_icu, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# AKI

y_range = c(0.0,1.0)
gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_24h_aki, main = "Diabetes = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_48h_aki, main = "Diabetes = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_72h_aki, main = "Diabetes = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_48h_aki, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_24h_aki, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_72h_aki, main = "Diabetes = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

dev.off()

Warning message in newton(lsp = lsp, X = G$X, y = G$y, Eb = G$Eb, UrS = G$UrS, L = G$L, :
“Iteration limit reached without full convergence - check carefully”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”

png 
  2

## chronic kidney diesase

In [121]:
df_sepsis1 <-df_mimic[df_mimic$ckd==1,]
df_sepsis0 <-df_mimic[df_mimic$ckd==0,]
# MIMIC MAP
# MIMIC MAP
# hospital mortality
gamfitMed_MIMIC_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_MIMIC_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_MIMIC_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/CKD_MIMIC_MAP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(55, 95)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_hosp, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_hosp, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_hosp, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_hosp, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_hosp, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_hosp, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# icu mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_icu, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_icu, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_icu, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_icu, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_icu, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_icu, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_24h_aki, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_48h_aki, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_72h_aki, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_48h_aki, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_24h_aki, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_72h_aki, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

dev.off()

Warning message in min(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning me

png 
  2

In [122]:
df_sepsis1 <-df_mimic[df_mimic$ckd==1,]
df_sepsis0 <-df_mimic[df_mimic$ckd==0,]
# MIMIC SBP
# hospital mortality
gamfitMed_MIMIC_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_MIMIC_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_MIMIC_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/CKD_MIMIC_SBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(90, 140)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_hosp, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_hosp, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_hosp, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_hosp, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_hosp, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_hosp, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# icu mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_icu, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_icu, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_icu, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_icu, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_icu, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_icu, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_24h_aki, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_48h_aki, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_72h_aki, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_48h_aki, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_24h_aki, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_72h_aki, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

dev.off()

Warning message in min(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning me

png 
  2

In [123]:
df_sepsis1 <-df_mimic[df_mimic$ckd==1,]
df_sepsis0 <-df_mimic[df_mimic$ckd==0,]
# MIMIC DBP
# hospital mortality
gamfitMed_MIMIC_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_MIMIC_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_MIMIC_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/CKD_MIMIC_DBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(35, 75)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_hosp, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_hosp, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_hosp, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_hosp, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_hosp, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_hosp, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# icu mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_icu, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_icu, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_icu, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_icu, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_icu, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_icu, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_24h_aki, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_48h_aki, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_72h_aki, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_48h_aki, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_24h_aki, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_72h_aki, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

dev.off()

Warning message in min(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning me

png 
  2

In [124]:
df_sepsis1 <-df_eicu[df_eicu$ckd==1,]
df_sepsis0 <-df_eicu[df_eicu$ckd==0,]
# eICU MAP
# hospital mortality
gamfitMed_eICU_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_eICU_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_eICU_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/CKD_eICU_MAP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(55, 95)
# hospital mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_hosp, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_hosp, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_hosp, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_hosp, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_hosp, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_hosp, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# icu mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_icu, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_icu, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_icu, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_icu, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_icu, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_icu, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_24h_aki, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_48h_aki, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_72h_aki, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_48h_aki, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_24h_aki, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_72h_aki, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

dev.off()

Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”

png 
  2

In [125]:
df_sepsis1 <-df_eicu[df_eicu$ckd==1,]
df_sepsis0 <-df_eicu[df_eicu$ckd==0,]
# eICU SBP
# hospital mortality
gamfitMed_eICU_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_eICU_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_eICU_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/CKD_eICU_SBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(90, 140)
# hospital mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_hosp, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_hosp, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_hosp, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_hosp, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_hosp, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_hosp, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# icu mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_icu, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_icu, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_icu, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_icu, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_icu, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_icu, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_24h_aki, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_48h_aki, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_72h_aki, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_48h_aki, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_24h_aki, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_72h_aki, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

dev.off()

Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”

png 
  2

In [126]:
df_sepsis1 <-df_eicu[df_eicu$ckd==1,]
df_sepsis0 <-df_eicu[df_eicu$ckd==0,]
# eICU DBP
# hospital mortality
gamfitMed_eICU_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_eICU_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_eICU_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/CKD_eICU_DBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(35, 75)
# hospital mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_hosp, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_hosp, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_hosp, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_hosp, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_hosp, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_hosp, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# icu mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_icu, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_icu, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_icu, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_icu, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_icu, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_icu, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_24h_aki, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_48h_aki, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_72h_aki, main = "Chronic kidney diesase = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_48h_aki, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_24h_aki, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_72h_aki, main = "Chronic kidney diesase = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

dev.off()

Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning me

png 
  2

## Chronic liver disease

In [127]:
df_sepsis1 <-df_mimic[df_mimic$cld==1,]
df_sepsis0 <-df_mimic[df_mimic$cld==0,]
# MIMIC MAP
# MIMIC MAP
# hospital mortality
gamfitMed_MIMIC_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_MIMIC_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_MIMIC_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/CLD_MIMIC_MAP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(55, 95)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_hosp, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_hosp, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_hosp, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_hosp, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_hosp, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_hosp, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# icu mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_icu, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_icu, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_icu, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_icu, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_icu, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_icu, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_24h_aki, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_48h_aki, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_72h_aki, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_48h_aki, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_24h_aki, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_72h_aki, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

dev.off()

Warning message in min(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning me

png 
  2

In [128]:
df_sepsis1 <-df_mimic[df_mimic$cld==1,]
df_sepsis0 <-df_mimic[df_mimic$cld==0,]
# MIMIC SBP
# hospital mortality
gamfitMed_MIMIC_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_MIMIC_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_MIMIC_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/CLD_MIMIC_SBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(90, 140)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_hosp, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_hosp, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_hosp, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_hosp, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_hosp, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_hosp, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# icu mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_icu, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_icu, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_icu, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_icu, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_icu, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_icu, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_24h_aki, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_48h_aki, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_72h_aki, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_48h_aki, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_24h_aki, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_72h_aki, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

dev.off()

Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”

png 
  2

In [129]:
df_sepsis1 <-df_mimic[df_mimic$cld==1,]
df_sepsis0 <-df_mimic[df_mimic$cld==0,]
# MIMIC DBP
# hospital mortality
gamfitMed_MIMIC_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_MIMIC_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_MIMIC_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/CLD_MIMIC_DBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(35, 75)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_hosp, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_hosp, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_hosp, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_hosp, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_hosp, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_hosp, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# icu mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_icu, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_icu, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_icu, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_icu, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_icu, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_icu, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_24h_aki, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_48h_aki, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_72h_aki, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_48h_aki, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_24h_aki, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_72h_aki, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

dev.off()

Warning message in min(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning me

png 
  2

In [130]:
df_sepsis1 <-df_eicu[df_eicu$ckd==1,]
df_sepsis0 <-df_eicu[df_eicu$ckd==0,]
# eICU MAP
# hospital mortality
gamfitMed_eICU_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_eICU_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_eICU_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/CLD_eICU_MAP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(55, 95)
# hospital mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_hosp, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_hosp, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_hosp, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_hosp, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_hosp, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_hosp, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# icu mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_icu, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_icu, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_icu, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_icu, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_icu, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_icu, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_24h_aki, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_48h_aki, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_72h_aki, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_48h_aki, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_24h_aki, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_72h_aki, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

dev.off()

Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”

png 
  2

In [131]:
df_sepsis1 <-df_eicu[df_eicu$ckd==1,]
df_sepsis0 <-df_eicu[df_eicu$ckd==0,]
# eICU SBP
# hospital mortality
gamfitMed_eICU_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_eICU_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_eICU_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/CLD_eICU_SBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(90, 140)
# hospital mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_hosp, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_hosp, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_hosp, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_hosp, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_hosp, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_hosp, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# icu mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_icu, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_icu, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_icu, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_icu, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_icu, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_icu, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_24h_aki, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_48h_aki, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_72h_aki, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_48h_aki, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_24h_aki, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_72h_aki, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

dev.off()

Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”

png 
  2

In [132]:
df_sepsis1 <-df_eicu[df_eicu$ckd==1,]
df_sepsis0 <-df_eicu[df_eicu$ckd==0,]
# eICU DBP
# hospital mortality
gamfitMed_eICU_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_eICU_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_eICU_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/CLD_eICU_DBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(35, 75)
# hospital mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_hosp, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_hosp, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_hosp, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_hosp, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_hosp, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_hosp, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# icu mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_icu, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_icu, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_icu, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_icu, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_icu, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_icu, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_24h_aki, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_48h_aki, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_72h_aki, main = "Chronic liver disease = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_48h_aki, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_24h_aki, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_72h_aki, main = "Chronic liver disease = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

dev.off()

Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning me

png 
  2

## Chronic obstructive pulmonary disease

In [176]:
df_sepsis1 <-df_mimic[df_mimic$copd==1,]
df_sepsis0 <-df_mimic[df_mimic$copd==0,]
# MIMIC MAP
# MIMIC MAP
# hospital mortality
gamfitMed_MIMIC_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_MIMIC_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_MIMIC_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/COPD_MIMIC_MAP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(55, 95)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_hosp, main = "COPD = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_hosp, main = "COPD = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_hosp, main = "COPD = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_hosp, main = "COPD = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_hosp, main = "COPD = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_hosp, main = "COPD = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# icu mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_icu, main = "COPD = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_icu, main = "COPD = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_icu, main = "COPD = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_icu, main = "COPD = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_icu, main = "COPD = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_icu, main = "COPD = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# AKI

y_range = c(0.0,1.0)
gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_24h_aki, main = "COPD = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_48h_aki, main = "COPD = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_72h_aki, main = "COPD = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_48h_aki, main = "COPD = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_24h_aki, main = "COPD = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_72h_aki, main = "COPD = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

dev.off()

Warning message in min(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning me

png 
  2

In [177]:
df_sepsis1 <-df_mimic[df_mimic$copd==1,]
df_sepsis0 <-df_mimic[df_mimic$copd==0,]
# MIMIC SBP
# hospital mortality
gamfitMed_MIMIC_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_MIMIC_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_MIMIC_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/COPD_MIMIC_SBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(90, 140)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_hosp, main = "COPD = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_hosp, main = "COPD = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_hosp, main = "COPD = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_hosp, main = "COPD = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_hosp, main = "COPD = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_hosp, main = "COPD = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# icu mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_icu, main = "COPD = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_icu, main = "COPD = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_icu, main = "COPD = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_icu, main = "COPD = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_icu, main = "COPD = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_icu, main = "COPD = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# AKI

y_range = c(0.0,1.0)
gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_24h_aki, main = "COPD = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_48h_aki, main = "COPD = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_72h_aki, main = "COPD = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_48h_aki, main = "COPD = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_24h_aki, main = "COPD = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_72h_aki, main = "COPD = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

dev.off()

Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning me

png 
  2

In [178]:
df_sepsis1 <-df_mimic[df_mimic$copd==1,]
df_sepsis0 <-df_mimic[df_mimic$copd==0,]
# MIMIC DBP
# hospital mortality
gamfitMed_MIMIC_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_MIMIC_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_MIMIC_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/COPD_MIMIC_DBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(35, 75)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_hosp, main = "COPD = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_hosp, main = "COPD = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_hosp, main = "COPD = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_hosp, main = "COPD = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_hosp, main = "COPD = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_hosp, main = "COPD = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# icu mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_icu, main = "COPD = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_icu, main = "COPD = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_icu, main = "COPD = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_icu, main = "COPD = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_icu, main = "COPD = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_icu, main = "COPD = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# AKI

y_range = c(0.0,1.0)
gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_24h_aki, main = "COPD = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_48h_aki, main = "COPD = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_72h_aki, main = "COPD = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_48h_aki, main = "COPD = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_24h_aki, main = "COPD = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_72h_aki, main = "COPD = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

dev.off()

Warning message in min(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning me

png 
  2

In [179]:
df_sepsis1 <-df_eicu[df_eicu$copd==1,]
df_sepsis0 <-df_eicu[df_eicu$copd==0,]

# eICU MAP
# hospital mortality
gamfitMed_eICU_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_eICU_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_eICU_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/COPD_eICU_MAP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(55, 95)
# hospital mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_hosp, main = "COPD = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_hosp, main = "COPD = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_hosp, main = "COPD = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_hosp, main = "COPD = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_hosp, main = "COPD = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_hosp, main = "COPD = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# icu mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_icu, main = "COPD = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_icu, main = "COPD = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_icu, main = "COPD = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_icu, main = "COPD = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_icu, main = "COPD = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_icu, main = "COPD = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# AKI

y_range = c(0.0,1.0)
gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_24h_aki, main = "COPD = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_48h_aki, main = "COPD = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_72h_aki, main = "COPD = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_48h_aki, main = "COPD = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_24h_aki, main = "COPD = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_72h_aki, main = "COPD = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

dev.off()

Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”

png 
  2

In [180]:
df_sepsis1 <-df_eicu[df_eicu$copd==1,]
df_sepsis0 <-df_eicu[df_eicu$copd==0,]

# eICU SBP
# hospital mortality
gamfitMed_eICU_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_eICU_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_eICU_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/COPD_eICU_SBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(90, 140)
# hospital mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_hosp, main = "COPD = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_hosp, main = "COPD = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_hosp, main = "COPD = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_hosp, main = "COPD = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_hosp, main = "COPD = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_hosp, main = "COPD = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# icu mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_icu, main = "COPD = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_icu, main = "COPD = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_icu, main = "COPD = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_icu, main = "COPD = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_icu, main = "COPD = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_icu, main = "COPD = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# AKI

y_range = c(0.0,1.0)
gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_24h_aki, main = "COPD = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_48h_aki, main = "COPD = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_72h_aki, main = "COPD = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_48h_aki, main = "COPD = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_24h_aki, main = "COPD = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_72h_aki, main = "COPD = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

dev.off()

png 
  2

In [181]:
df_sepsis1 <-df_eicu[df_eicu$copd==1,]
df_sepsis0 <-df_eicu[df_eicu$copd==0,]
# eICU DBP
# hospital mortality
gamfitMed_eICU_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_eICU_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_eICU_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/COPD_eICU_DBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(35, 75)
# hospital mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_hosp, main = "COPD = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_hosp, main = "COPD = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_hosp, main = "COPD = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_hosp, main = "COPD = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_hosp, main = "COPD = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_hosp, main = "COPD = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# icu mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_icu, main = "COPD = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_icu, main = "COPD = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_icu, main = "COPD = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_icu, main = "COPD = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_icu, main = "COPD = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_icu, main = "COPD = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# AKI

y_range = c(0.0,1.0)
gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_24h_aki, main = "COPD = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_48h_aki, main = "COPD = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_72h_aki, main = "COPD = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_48h_aki, main = "COPD = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_24h_aki, main = "COPD = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_72h_aki, main = "COPD = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

dev.off()


Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”

png 
  2

## stroke

In [182]:
df_sepsis1 <-df_mimic[df_mimic$stroke==1,]
df_sepsis0 <-df_mimic[df_mimic$stroke==0,]
# MIMIC MAP
# MIMIC MAP
# hospital mortality
gamfitMed_MIMIC_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_MIMIC_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_MIMIC_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/Stroke_MIMIC_MAP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(55, 95)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_hosp, main = "Stroke = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_hosp, main = "Stroke = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_hosp, main = "Stroke = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_hosp, main = "Stroke = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_hosp, main = "Stroke = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_hosp, main = "Stroke = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# icu mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_icu, main = "Stroke = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_icu, main = "Stroke = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_icu, main = "Stroke = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_icu, main = "Stroke = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_icu, main = "Stroke = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_icu, main = "Stroke = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# AKI

y_range = c(0.0,1.0)
gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_24h_aki, main = "Stroke = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_48h_aki, main = "Stroke = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_72h_aki, main = "Stroke = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_48h_aki, main = "Stroke = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_24h_aki, main = "Stroke = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_72h_aki, main = "Stroke = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

dev.off()

Warning message in min(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning me

png 
  2

In [183]:
df_sepsis1 <-df_mimic[df_mimic$stroke==1,]
df_sepsis0 <-df_mimic[df_mimic$stroke==0,]
# MIMIC SBP
# hospital mortality
gamfitMed_MIMIC_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_MIMIC_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_MIMIC_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/Stroke_MIMIC_SBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(90, 140)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_hosp, main = "Stroke = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_hosp, main = "Stroke = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_hosp, main = "Stroke = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_hosp, main = "Stroke = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_hosp, main = "Stroke = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_hosp, main = "Stroke = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# icu mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_icu, main = "Stroke = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_icu, main = "Stroke = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_icu, main = "Stroke = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_icu, main = "Stroke = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_icu, main = "Stroke = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_icu, main = "Stroke = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# AKI

y_range = c(0.0,1.0)
gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_24h_aki, main = "Stroke = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_48h_aki, main = "Stroke = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_72h_aki, main = "Stroke = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_48h_aki, main = "Stroke = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_24h_aki, main = "Stroke = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_72h_aki, main = "Stroke = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

dev.off()

png 
  2

In [184]:
df_sepsis1 <-df_mimic[df_mimic$stroke==1,]
df_sepsis0 <-df_mimic[df_mimic$stroke==0,]
# MIMIC DBP
# hospital mortality
gamfitMed_MIMIC_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_MIMIC_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_MIMIC_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/Stroke_MIMIC_DBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(35, 75)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_hosp, main = "Stroke = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_hosp, main = "Stroke = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_hosp, main = "Stroke = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_hosp, main = "Stroke = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_hosp, main = "Stroke = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_hosp, main = "Stroke = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# icu mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_icu, main = "Stroke = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_icu, main = "Stroke = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_icu, main = "Stroke = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_icu, main = "Stroke = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_icu, main = "Stroke = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_icu, main = "Stroke = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# AKI

y_range = c(0.0,1.0)
gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_24h_aki, main = "Stroke = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_48h_aki, main = "Stroke = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_72h_aki, main = "Stroke = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_48h_aki, main = "Stroke = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_24h_aki, main = "Stroke = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_72h_aki, main = "Stroke = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

dev.off()

Warning message in min(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning me

png 
  2

In [185]:
df_sepsis1 <-df_eicu[df_eicu$stroke==1,]
df_sepsis0 <-df_eicu[df_eicu$stroke==0,]
# eICU MAP
# hospital mortality
gamfitMed_eICU_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_eICU_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_eICU_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/Stroke_eICU_MAP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(55, 95)
# hospital mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_hosp, main = "Stroke = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_hosp, main = "Stroke = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_hosp, main = "Stroke = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_hosp, main = "Stroke = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_hosp, main = "Stroke = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_hosp, main = "Stroke = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# icu mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_icu, main = "Stroke = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_icu, main = "Stroke = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_icu, main = "Stroke = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_icu, main = "Stroke = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_icu, main = "Stroke = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_icu, main = "Stroke = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# AKI

y_range = c(0.0,1.0)
gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_24h_aki, main = "Stroke = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_48h_aki, main = "Stroke = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_72h_aki, main = "Stroke = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_48h_aki, main = "Stroke = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_24h_aki, main = "Stroke = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_72h_aki, main = "Stroke = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

dev.off()

png 
  2

In [186]:
df_sepsis1 <-df_eicu[df_eicu$stroke==1,]
df_sepsis0 <-df_eicu[df_eicu$stroke==0,]
# eICU SBP
# hospital mortality
gamfitMed_eICU_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_eICU_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_eICU_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/Stroke_eICU_SBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(90, 140)
# hospital mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_hosp, main = "Stroke = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_hosp, main = "Stroke = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_hosp, main = "Stroke = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_hosp, main = "Stroke = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_hosp, main = "Stroke = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_hosp, main = "Stroke = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# icu mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_icu, main = "Stroke = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_icu, main = "Stroke = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_icu, main = "Stroke = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_icu, main = "Stroke = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_icu, main = "Stroke = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_icu, main = "Stroke = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# AKI

y_range = c(0.0,1.0)
gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_24h_aki, main = "Stroke = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_48h_aki, main = "Stroke = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_72h_aki, main = "Stroke = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_48h_aki, main = "Stroke = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_24h_aki, main = "Stroke = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_72h_aki, main = "Stroke = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

dev.off()

png 
  2

In [187]:
df_sepsis1 <-df_eicu[df_eicu$stroke==1,]
df_sepsis0 <-df_eicu[df_eicu$stroke==0,]
# eICU DBP
# hospital mortality
gamfitMed_eICU_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_eICU_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_eICU_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/Stroke_eICU_DBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(35, 75)
# hospital mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_hosp, main = "Stroke = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_hosp, main = "Stroke = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_hosp, main = "Stroke = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_hosp, main = "Stroke = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_hosp, main = "Stroke = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_hosp, main = "Stroke = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# icu mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_icu, main = "Stroke = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_icu, main = "Stroke = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_icu, main = "Stroke = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_icu, main = "Stroke = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_icu, main = "Stroke = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_icu, main = "Stroke = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# AKI

y_range = c(0.0,1.0)
gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_24h_aki, main = "Stroke = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_48h_aki, main = "Stroke = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_72h_aki, main = "Stroke = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_48h_aki, main = "Stroke = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_24h_aki, main = "Stroke = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_72h_aki, main = "Stroke = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

dev.off()

png 
  2

## ischemic heart disease

In [145]:
df_sepsis1 <-df_mimic[df_mimic$ihd==1,]
df_sepsis0 <-df_mimic[df_mimic$ihd==0,]
# MIMIC MAP
# MIMIC MAP
# hospital mortality
gamfitMed_MIMIC_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_MIMIC_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_MIMIC_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/IHD_MIMIC_MAP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(55, 95)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_hosp, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_hosp, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_hosp, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_hosp, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_hosp, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_hosp, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# icu mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_icu, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_icu, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_icu, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_icu, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_icu, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_icu, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_24h_aki, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_48h_aki, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_72h_aki, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_48h_aki, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_24h_aki, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_72h_aki, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

dev.off()

png 
  2

In [146]:
df_sepsis1 <-df_mimic[df_mimic$ihd==1,]
df_sepsis0 <-df_mimic[df_mimic$ihd==0,]
# MIMIC SBP
# hospital mortality
gamfitMed_MIMIC_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_MIMIC_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_MIMIC_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/IHD_MIMIC_SBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(90, 140)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_hosp, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_hosp, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_hosp, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_hosp, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_hosp, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_hosp, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# icu mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_icu, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_icu, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_icu, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_icu, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_icu, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_icu, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_24h_aki, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_48h_aki, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_72h_aki, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_48h_aki, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_24h_aki, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_72h_aki, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

dev.off()

png 
  2

In [147]:
df_sepsis1 <-df_mimic[df_mimic$ihd==1,]
df_sepsis0 <-df_mimic[df_mimic$ihd==0,]
# MIMIC DBP
# hospital mortality
gamfitMed_MIMIC_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_MIMIC_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_MIMIC_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/IHD_MIMIC_DBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(35, 75)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_hosp, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_hosp, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_hosp, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_hosp, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_hosp, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_hosp, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# icu mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_icu, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_icu, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_icu, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_icu, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_icu, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_icu, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# AKI

y_range = c(0.7,1.0)
gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_24h_aki, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_48h_aki, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_72h_aki, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_48h_aki, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_24h_aki, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_72h_aki, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

dev.off()

png 
  2

In [188]:
df_sepsis1 <-df_eicu[df_eicu$ihd==1,]
df_sepsis0 <-df_eicu[df_eicu$ihd==0,]
# eICU MAP
# hospital mortality
gamfitMed_eICU_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_eICU_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_eICU_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/IHD_eICU_MAP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(55, 95)
# hospital mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_hosp, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_hosp, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_hosp, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_hosp, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_hosp, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_hosp, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# icu mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_icu, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_icu, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_icu, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_icu, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_icu, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_icu, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# AKI

y_range = c(0.0,1.0)
gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_24h_aki, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_48h_aki, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_72h_aki, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_48h_aki, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_24h_aki, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_72h_aki, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

dev.off()


Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”

png 
  2

In [189]:
df_sepsis1 <-df_eicu[df_eicu$ihd==1,]
df_sepsis0 <-df_eicu[df_eicu$ihd==0,]
# eICU SBP
# hospital mortality
gamfitMed_eICU_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_eICU_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_eICU_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/IHD_eICU_SBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(90, 140)
# hospital mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_hosp, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_hosp, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_hosp, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_hosp, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_hosp, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_hosp, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# icu mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_icu, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_icu, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_icu, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_icu, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_icu, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_icu, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# AKI

y_range = c(0.0,1.0)
gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_24h_aki, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_48h_aki, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_72h_aki, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_48h_aki, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_24h_aki, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_72h_aki, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

dev.off()


png 
  2

In [190]:
df_sepsis1 <-df_eicu[df_eicu$ihd==1,]
df_sepsis0 <-df_eicu[df_eicu$ihd==0,]

# eICU DBP
# hospital mortality
gamfitMed_eICU_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_eICU_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_eICU_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_eICU_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_eICU_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/IHD_eICU_DBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(35, 75)
# hospital mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_hosp, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_hosp, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_hosp, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_hosp, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_hosp, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_hosp, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# icu mortality
gam_plotMed(gamfitMed_eICU_sepsis1_24h_icu, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_48h_icu, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis1_72h_icu, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_48h_icu, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_24h_icu, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_sepsis0_72h_icu, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# AKI

y_range = c(0.0,1.0)
gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_24h_aki, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_48h_aki, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis1_72h_aki, main = "Ischemic heart disease = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_48h_aki, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_24h_aki, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_eICU_sepsis0_72h_aki, main = "Ischemic heart disease = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

dev.off()

Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”

png 
  2

## cancer

In [191]:
df_sepsis1 <-df_mimic[df_mimic$cancer==1,]
df_sepsis0 <-df_mimic[df_mimic$cancer==0,]
# MIMIC MAP
# MIMIC MAP
# hospital mortality
gamfitMed_MIMIC_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_MIMIC_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_MIMIC_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/Cancer_MIMIC_MAP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(55, 95)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_hosp, main = "Cancer = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_hosp, main = "Cancer = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_hosp, main = "Cancer = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_hosp, main = "Cancer = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_hosp, main = "Cancer = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_hosp, main = "Cancer = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# icu mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_icu, main = "Cancer = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_icu, main = "Cancer = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_icu, main = "Cancer = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_icu, main = "Cancer = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_icu, main = "Cancer = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_icu, main = "Cancer = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

# AKI

y_range = c(0.5,1.0)
gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_24h_aki, main = "Cancer = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_48h_aki, main = "Cancer = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_72h_aki, main = "Cancer = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_48h_aki, main = "Cancer = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_24h_aki, main = "Cancer = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_72h_aki, main = "Cancer = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

dev.off()

Warning message in min(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning me

png 
  2

In [192]:
df_sepsis1 <-df_mimic[df_mimic$cancer==1,]
df_sepsis0 <-df_mimic[df_mimic$cancer==0,]
# MIMIC SBP
# hospital mortality
gamfitMed_MIMIC_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_MIMIC_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_MIMIC_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/Cancer_MIMIC_SBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(90, 140)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_hosp, main = "Cancer = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_hosp, main = "Cancer = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_hosp, main = "Cancer = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_hosp, main = "Cancer = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_hosp, main = "Cancer = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_hosp, main = "Cancer = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# icu mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_icu, main = "Cancer = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_icu, main = "Cancer = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_icu, main = "Cancer = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_icu, main = "Cancer = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_icu, main = "Cancer = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_icu, main = "Cancer = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

# AKI

y_range = c(0.5,1.0)
gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_24h_aki, main = "Cancer = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_48h_aki, main = "Cancer = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_72h_aki, main = "Cancer = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_48h_aki, main = "Cancer = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_24h_aki, main = "Cancer = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_72h_aki, main = "Cancer = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

dev.off()

png 
  2

In [193]:
df_sepsis1 <-df_mimic[df_mimic$cancer==1,]
df_sepsis0 <-df_mimic[df_mimic$cancer==0,]
# MIMIC DBP
# hospital mortality
gamfitMed_MIMIC_sepsis1_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_hosp <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

# icu mortality
gamfitMed_MIMIC_sepsis1_24h_icu <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_icu  <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)


# AKI
gamfitMed_MIMIC_sepsis1_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_24h_aki <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_48h_aki <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

gamfitMed_MIMIC_sepsis1_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis1, family = binomial)
gamfitMed_MIMIC_sepsis0_72h_aki <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_sepsis0, family = binomial)

pdf(file = "figure/subgroup/medical_history/Cancer_MIMIC_DBP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(35, 75)
# hospital mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_hosp, main = "Cancer = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_hosp, main = "Cancer = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_hosp, main = "Cancer = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_hosp, main = "Cancer = No", xRange = x_range,
                    yRange = c(0, .50),label = "B1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_hosp, main = "Cancer = No", xRange = x_range,
                    yRange = c(0, .50), label = "D1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_hosp, main = "Cancer = No", xRange = x_range,
                    yRange = c(0, .50),label = "F1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# icu mortality
gam_plotMed(gamfitMed_MIMIC_sepsis1_24h_icu, main = "Cancer = Yes", xRange = x_range,
                    yRange = c(0, 0.5), label = "A2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_48h_icu, main = "Cancer = Yes", xRange = x_range,
                    yRange = c(0, .50), label = "C2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis1_72h_icu, main = "Cancer = Yes", xRange = x_range,
                    yRange = c(0, .50),label = "E2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_48h_icu, main = "Cancer = No", xRange = x_range,
                    yRange = c(0, .50),label = "B2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_24h_icu, main = "Cancer = No", xRange = x_range,
                    yRange = c(0, .50), label = "D2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_sepsis0_72h_icu, main = "Cancer = No", xRange = x_range,
                    yRange = c(0, .50),label = "F2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

# AKI

y_range = c(0.5,1.0)
gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_24h_aki, main = "Cancer = Yes", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_48h_aki, main = "Cancer = Yes", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis1_72h_aki, main = "Cancer = Yes", xRange = x_range,
                    yRange = y_range,label = "E3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_48h_aki, main = "Cancer = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "B3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_24h_aki, main = "Cancer = No", xRange = x_range,
                    yRange = c(0.5,1.0), label = "D3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_MIMIC_sepsis0_72h_aki, main = "Cancer = No", xRange = x_range,
                    yRange = c(0.5,1.0),label = "F3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

dev.off()

Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”Warning message in min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to min; returning Inf”Warning message in max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient < :
“no non-missing arguments to max; returning -Inf”

png 
  2

## ORs

In [395]:
c1 <- df_eicu_prop_AKI_72$vasso_duration_72h>=36
c2 <- df_eicu_prop_AKI_72$vasso_duration_72h<36
# Criterias <- data.frame( c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12,c13,c14,c15,c16)
Criterias <- data.frame(c1,c2)

Hosp_OR<- list()
Hosp_LCI<-list()
Hosp_UCI<-list()
Hosp_pvalue<-list()
i=1
j=1
while (i <= length(Criterias)){
    # MAP 24
    df_10<-df_eicu_prop_AKI_72[df_eicu_prop_AKI_72$prop_map_72_AKI>=0.05 & df_eicu_prop_AKI_72$prop_map_72_AKI<=0.15 & Criterias[,i],]
    df_40<-df_eicu_prop_AKI_72[df_eicu_prop_AKI_72$prop_map_72_AKI>=0.35 & df_eicu_prop_AKI_72$prop_map_72_AKI<=0.45 & Criterias[,i],]
    a<-length(df_10[df_10$AKI_7d==1,1]) # exposed
    c<-length(df_10[df_10$AKI_7d==0,1])
    b<-length(df_40[df_40$AKI_7d==1,1]) # control
    d<-length(df_40[df_40$AKI_7d==0,1])
    df_OR<-oddsratio(matrix(c(a,b,c,d), ncol=2, nrow=2))
    Hosp_OR[j]<-df_OR$measure[2]
    Hosp_LCI[j]<-df_OR$measure[4]
    Hosp_UCI[j]<-df_OR$measure[6]
    Hosp_pvalue[j]<-df_OR$p.value[4]

    # SBP 24
    df_10<-df_eicu_prop_AKI_72[df_eicu_prop_AKI_72$prop_asp_72_AKI>=0.05 & df_eicu_prop_AKI_72$prop_asp_72_AKI<=0.15 & Criterias[,i],]
    df_40<-df_eicu_prop_AKI_72[df_eicu_prop_AKI_72$prop_asp_72_AKI>=0.35 & df_eicu_prop_AKI_72$prop_asp_72_AKI<=0.45 & Criterias[,i],]

    a<-length(df_10[df_10$AKI_7d==1,1]) # exposed
    c<-length(df_10[df_10$AKI_7d==0,1])
    b<-length(df_40[df_40$AKI_7d==1,1]) # control
    d<-length(df_40[df_40$AKI_7d==0,1])
    df_OR<-oddsratio(matrix(c(a,b,c,d), ncol=2, nrow=2))
    Hosp_OR[j+1]<-df_OR$measure[2]
    Hosp_LCI[j+1]<-df_OR$measure[4]
    Hosp_UCI[j+1]<-df_OR$measure[6]
    Hosp_pvalue[j+1]<-df_OR$p.value[4]

    # DBP 24
    df_10<-df_eicu_prop_AKI_72[df_eicu_prop_AKI_72$prop_adp_72_AKI>=0.05 & df_eicu_prop_AKI_72$prop_adp_72_AKI<=0.15 & Criterias[,i],]
    df_40<-df_eicu_prop_AKI_72[df_eicu_prop_AKI_72$prop_adp_72_AKI>=0.35 & df_eicu_prop_AKI_72$prop_adp_72_AKI<=0.45 & Criterias[,i],]
    a<-length(df_10[df_10$AKI_7d==1,1]) # exposed
    c<-length(df_10[df_10$AKI_7d==0,1])
    b<-length(df_40[df_40$AKI_7d==1,1]) # control
    d<-length(df_40[df_40$AKI_7d==0,1])
    df_OR<-oddsratio(matrix(c(a,b,c,d), ncol=2, nrow=2))
    Hosp_OR[j+2]<-df_OR$measure[2]
    Hosp_LCI[j+2]<-df_OR$measure[4]
    Hosp_UCI[j+2]<-df_OR$measure[6]
    Hosp_pvalue[j+2]<-df_OR$p.value[4]
 
    df_OR_CI <- data.frame("OR" = unlist(Hosp_OR, use.names=FALSE), 
                        "LCI" = unlist(Hosp_LCI, use.names=FALSE)
                       , "UCI" = unlist(Hosp_UCI, use.names=FALSE)
                       , "pvalue" = unlist(Hosp_pvalue, use.names=FALSE))
    i <-i+1
    j <-j+9
    
}
df_OR_CI

OR,LCI,UCI,pvalue
1.578054,0.5910144,5.598471,0.47949827
2.015445,1.0016424,4.531207,0.05611684
1.792469,0.7557177,4.771859,0.21346700
1.985733,0.6771442,8.787164,0.32902903
1.140149,0.6823060,1.974557,0.69446607
0.837764,0.4306968,1.699895,0.59969383


## for small number

In [397]:
c0 <- df_eicu_prop_AKI$vasso_duration_24h >=12
c1 <- df_eicu_prop_AKI$ethnicity=="HISPANIC"
c2 <-df_eicu_prop_AKI$ethnicity=="BLACK"
c3 <-df_eicu_prop_AKI$ethnicity=="ASIAN"
c4 <-df_eicu_prop_AKI$ethnicity=="OTHERS"
c5 <-df_eicu_prop_AKI$afib ==1
c6 <-df_eicu_prop_AKI$ckd ==1
c7 <-df_eicu_prop_AKI$cld ==1
c8 <-df_eicu_prop_AKI$copd==1
c9 <-df_eicu_prop_AKI$stroke==1
c10 <-df_eicu_prop_AKI$unittype =="CCU"
c11<-df_eicu_prop_AKI$unittype =="CSRU"
c12<-df_eicu_prop_AKI$ethnicity=="Hispanic"

# Criterias <- data.frame( c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12,c13,c14,c15)
Criterias <- data.frame(c0)

Hosp_OR<- list()
Hosp_LCI<-list()
Hosp_UCI<-list()
Hosp_pvalue<-list()
i=1
j=1
while (i <= length(Criterias)){
    # MAP 24
    df_10<-df_eicu_prop_AKI[df_eicu_prop_AKI$prop_map_24_AKI>=0.001 & df_eicu_prop_AKI$prop_map_24_AKI<0.50 & Criterias[,i],]
    df_40<-df_eicu_prop_AKI[df_eicu_prop_AKI$prop_map_24_AKI>=0.50 & df_eicu_prop_AKI$prop_map_24_AKI<=1.00 & Criterias[,i],]
    a<-length(df_10[df_10$AKI_7d==1,1]) # exposed
    c<-length(df_10[df_10$AKI_7d==0,1])
    b<-length(df_40[df_40$AKI_7d==1,1]) # control
    d<-length(df_40[df_40$AKI_7d==0,1])
    df_OR<-oddsratio(matrix(c(a,b,c,d), ncol=2, nrow=2))
    Hosp_OR[j]<-df_OR$measure[2]
    Hosp_LCI[j]<-df_OR$measure[4]
    Hosp_UCI[j]<-df_OR$measure[6]
    Hosp_pvalue[j]<-df_OR$p.value[4]

    # SBP 24
    df_10<-df_eicu_prop_AKI[df_eicu_prop_AKI$prop_asp_24_AKI>=0.001 & df_eicu_prop_AKI$prop_asp_24_AKI<0.50 & Criterias[,i],]
    df_40<-df_eicu_prop_AKI[df_eicu_prop_AKI$prop_asp_24_AKI>=0.50 & df_eicu_prop_AKI$prop_asp_24_AKI<=1.00 & Criterias[,i],]

    a<-length(df_10[df_10$AKI_7d==1,1]) # exposed
    c<-length(df_10[df_10$AKI_7d==0,1])
    b<-length(df_40[df_40$AKI_7d==1,1]) # control
    d<-length(df_40[df_40$AKI_7d==0,1])
    df_OR<-oddsratio(matrix(c(a,b,c,d), ncol=2, nrow=2))
    Hosp_OR[j+1]<-df_OR$measure[2]
    Hosp_LCI[j+1]<-df_OR$measure[4]
    Hosp_UCI[j+1]<-df_OR$measure[6]
    Hosp_pvalue[j+1]<-df_OR$p.value[4]

    # DBP 24
    df_10<-df_eicu_prop_AKI[df_eicu_prop_AKI$prop_adp_24_AKI>=0.001 & df_eicu_prop_AKI$prop_adp_24_AKI<0.50 & Criterias[,i],]
    df_40<-df_eicu_prop_AKI[df_eicu_prop_AKI$prop_adp_24_AKI>=0.50 & df_eicu_prop_AKI$prop_adp_24_AKI<=1.00 & Criterias[,i],]
    a<-length(df_10[df_10$AKI_7d==1,1]) # exposed
    c<-length(df_10[df_10$AKI_7d==0,1])
    b<-length(df_40[df_40$AKI_7d==1,1]) # control
    d<-length(df_40[df_40$AKI_7d==0,1])
    df_OR<-oddsratio(matrix(c(a,b,c,d), ncol=2, nrow=2))
    Hosp_OR[j+2]<-df_OR$measure[2]
    Hosp_LCI[j+2]<-df_OR$measure[4]
    Hosp_UCI[j+2]<-df_OR$measure[6]
    Hosp_pvalue[j+2]<-df_OR$p.value[4]
 
    df_OR_CI <- data.frame("OR" = unlist(Hosp_OR, use.names=FALSE), 
                        "LCI" = unlist(Hosp_LCI, use.names=FALSE)
                       , "UCI" = unlist(Hosp_UCI, use.names=FALSE)
                       , "pvalue" = unlist(Hosp_pvalue, use.names=FALSE))
    i <-i+1
    j <-j+3
    
}
df_OR_CI

OR,LCI,UCI,pvalue
1.0011405,0.8221061,1.224684,1.0000000
0.9573257,0.7956549,1.154031,0.6686645
0.9841138,0.7898531,1.235556,0.9089509


In [208]:
Hosp_OR<- list()
Hosp_LCI<-list()
Hosp_UCI<-list()
Hosp_pvalue<-list()
i=1
j=1
while (i <= length(Criterias)){
    # MAP 24
    df_10<-df_eicu_prop[df_eicu_prop$prop_map_24>=0.05 & df_eicu_prop$prop_map_24<=0.15 ,]
    df_40<-df_eicu_prop[df_eicu_prop$prop_map_24>=0.35 & df_eicu_prop$prop_map_24<=0.45 ,]
    a<-length(df_10[df_10$hospital_expire_flag==1,1]) # exposed
    c<-length(df_10[df_10$hospital_expire_flag==0,1])
    b<-length(df_40[df_40$hospital_expire_flag==1,1]) # control
    d<-length(df_40[df_40$hospital_expire_flag==0,1])
    df_OR<-oddsratio(matrix(c(a,b,c,d), ncol=2, nrow=2))
    Hosp_OR[j]<-df_OR$measure[2]
    Hosp_LCI[j]<-df_OR$measure[4]
    Hosp_UCI[j]<-df_OR$measure[6]
    Hosp_pvalue[j]<-df_OR$p.value[4]

    # SBP 24
    df_10<-df_eicu_prop[df_eicu_prop$prop_asp_24>=0.05 & df_eicu_prop$prop_asp_24<=0.15 ,]
    df_40<-df_eicu_prop[df_eicu_prop$prop_asp_24>=0.35 & df_eicu_prop$prop_asp_24<=0.45 ,]

    a<-length(df_10[df_10$hospital_expire_flag==1,1]) # exposed
    c<-length(df_10[df_10$hospital_expire_flag==0,1])
    b<-length(df_40[df_40$hospital_expire_flag==1,1]) # control
    d<-length(df_40[df_40$hospital_expire_flag==0,1])
    df_OR<-oddsratio(matrix(c(a,b,c,d), ncol=2, nrow=2))
    Hosp_OR[j+1]<-df_OR$measure[2]
    Hosp_LCI[j+1]<-df_OR$measure[4]
    Hosp_UCI[j+1]<-df_OR$measure[6]
    Hosp_pvalue[j+1]<-df_OR$p.value[4]

    # DBP 24
    df_10<-df_eicu_prop[df_eicu_prop$prop_adp_24>=0.05 & df_eicu_prop$prop_adp_24<=0.15 ,]
    df_40<-df_eicu_prop[df_eicu_prop$prop_adp_24>=0.35 & df_eicu_prop$prop_adp_24<=0.45 ,]
    a<-length(df_10[df_10$hospital_expire_flag==1,1]) # exposed
    c<-length(df_10[df_10$hospital_expire_flag==0,1])
    b<-length(df_40[df_40$hospital_expire_flag==1,1]) # control
    d<-length(df_40[df_40$hospital_expire_flag==0,1])
    df_OR<-oddsratio(matrix(c(a,b,c,d), ncol=2, nrow=2))
    Hosp_OR[j+2]<-df_OR$measure[2]
    Hosp_LCI[j+2]<-df_OR$measure[4]
    Hosp_UCI[j+2]<-df_OR$measure[6]
    Hosp_pvalue[j+2]<-df_OR$p.value[4]
 
    df_OR_CI <- data.frame("OR" = unlist(Hosp_OR, use.names=FALSE), 
                        "LCI" = unlist(Hosp_LCI, use.names=FALSE)
                       , "UCI" = unlist(Hosp_UCI, use.names=FALSE)
                       , "pvalue" = unlist(Hosp_pvalue, use.names=FALSE))
    i <-i+1
    j <-j+9
    
}
df_OR_CI

OR,LCI,UCI,pvalue
1.295113,1.1024340,1.520825,0.0018561750
1.347493,1.1295274,1.613962,0.0009915676
1.176258,0.9833797,1.406213,0.0813327580
1.295113,1.1024340,1.520825,0.0018561750
1.347493,1.1295274,1.613962,0.0009915676
1.176258,0.9833797,1.406213,0.0813327580
